<a href="https://colab.research.google.com/github/seabhra/ChatBot_NewsgamesIA/blob/main/newsgames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
%run verificacao_simples.py

NameError: name 'resultados' is not defined

In [36]:
%%writefile verificacao_simples.py

import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse, urlencode
import hashlib
from collections import Counter
import time
import logging
import base64
import io
import nltk
import os
import sys
import subprocess

try:
    from dotenv import load_dotenv
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-dotenv"])
    from dotenv import load_dotenv




from concurrent.futures import ThreadPoolExecutor
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
from PIL import Image, UnidentifiedImageError
from datetime import datetime

# Configurar o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("VerificadorNoticias")

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticias:
    def __init__(self):
        logger.info("Verificador de Notícias inicializado!")

        # Carregar variáveis do ambiente
        load_dotenv()
        self.fact_check_api_key = os.getenv("FACT_CHECK_API_KEY")
        self.fact_check_api_url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]
        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5, "viral": 1.5,
            "bizarro": 1.5, "aterrorizante": 2, "assustador": 1.5, "terrível": 1.5,
            "estarrecedor": 2, "impossível": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors",
            "parece que", "há relatos", "não confirmado", "segundo fontes",
            "indícios", "sinalizam", "aparentemente", "estaria", "poderia",
            "teria", "estima-se"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe",
            "na visão de", "do meu ponto de vista", "considero", "entendo que",
            "me parece", "creio que", "penso que", "sinto que"
        ]

        # Tentar baixar recursos NLTK necessários para análise de texto
        try:
            nltk.download('punkt', quiet=True)
        except Exception as e:
            logger.warning(f"Não foi possível baixar recursos NLTK: {e}")

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        return parsed_url.netloc

    def verificar_url_simples(self, url):
        """Executa uma verificação simples em uma URL de notícia."""
        try:
            # Extrair domínio
            parsed_url = urlparse(url)
            dominio = parsed_url.netloc

            # Tentar acessar a URL
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10, verify=False)
            response.raise_for_status()  # Levantar exceção para status HTTP de erro

            # Verificar status HTTP
            if response.status_code == 200:
                logger.info(f"URL acessada com sucesso: {url}")
                logger.info(f"Domínio: {dominio}")
                logger.info(f"Tamanho do conteúdo: {len(response.text)} caracteres.")
                return {
                    "status": "sucesso",
                    "dominio": dominio,
                    "tamanho_conteudo": len(response.text),
                    "mensagem": "URL acessada com sucesso"
                }
            else:
                logger.warning(f"Erro ao acessar a URL. Código HTTP: {response.status_code}")
                return {
                    "status": "erro",
                    "mensagem": f"Erro ao acessar a URL. Código HTTP: {response.status_code}"
                }

        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao acessar a URL: {e}")
            return {
                "status": "erro",
                "mensagem": str(e)
            }

    def analisar_sentimento(self, texto):
        """Analisa o sentimento do texto usando TextBlob."""
        try:
            sentiment = TextBlob(texto).sentiment
            return {
                "polaridade": sentiment.polarity,
                "subjetividade": sentiment.subjectivity
            }
        except Exception as e:
            logger.error(f"Erro na análise de sentimento: {e}")
            return {
                "polaridade": 0,
                "subjetividade": 0
            }

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {"dominio": dominio}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
            return "Fonte confiável", 20, resultado
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
            return "Fonte suspeita", -30, resultado
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0
            return "Fonte não categorizada", 0, resultado

    def baixar_artigo(self, url):
        """Baixa o conteúdo do artigo a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""
            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)
                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date'] or meta.get('name') in ['publication_date', 'date', 'pubdate']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]', '[name=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(author_tag)
                else:
                    elements = soup.find_all(author_tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords,
                'html': response.text  # Manter o HTML para possível análise adicional
            }
        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            logger.error(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None
class VerificadorFactCheck:
    def __init__(self):
        """Inicializa a classe com os atributos necessários."""
        self.fact_check_api_url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

    def consultar_fact_check(self, query):
        """Consulta a API do Google Fact Check Tools."""
        # Restante do método
        pass

        # Carregar variáveis de ambiente
        load_dotenv()
        api_key = os.getenv("FACT_CHECK_API_KEY")

        if not api_key or api_key.strip() == "":
            raise ValueError("Chave de API não encontrada no arquivo .env. Certifique-se de definir FACT_CHECK_API_KEY corretamente.")

        # Verificar se a URL base da API está configurada
        if not self.fact_check_api_url.strip():
            logger.warning("URL base para a API de Fact Check não configurada.")
            return None

        # Construir a URL da API
        url_api = f"{self.fact_check_api_url}?key={api_key}&query={query}"

        try:
            response = requests.get(url_api)
            if response.status_code == 200:
                return response.json()
            else:
                logger.warning(f"Falha ao acessar API de Fact Check. Status: {response.status_code}")
                return None
        except Exception as e:
            logger.error(f"Erro na API de Fact Check: {e}")
            return None



    def analisar_linguagem(self, texto):
        """Analisa o conteúdo em busca de sensacionalismo, incerteza e opinatividade."""
        if not texto or len(texto) < 10:
            return {
                "sensacionalismo": 0,
                "incerteza": 0,
                "opinativo": 0,
                "exclamacoes": 0
            }

        resultado = {}

        # Análise de sensacionalismo
        indice_sensacionalismo = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower())) * peso
            for palavra, peso in self.palavras_sensacionalistas.items()
        )
        resultado["sensacionalismo"] = indice_sensacionalismo

        # Normalizar por tamanho do texto (por 1000 palavras)
        palavras_totais = len(texto.split())
        if palavras_totais > 0:
            indice_sensacionalismo_norm = (indice_sensacionalismo * 1000) / palavras_totais
            resultado["sensacionalismo_normalizado"] = indice_sensacionalismo_norm

            # Adicionar alerta se o índice for alto
            if indice_sensacionalismo_norm > 10:
                resultado["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista"
            elif indice_sensacionalismo_norm > 5:
                resultado["alerta_sensacionalismo"] = "Moderado índice de linguagem sensacionalista"

        # Análise de incerteza
        indice_incerteza = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower()))
            for palavra in self.palavras_incerteza
        )
        resultado["incerteza"] = indice_incerteza

        if palavras_totais > 0:
            indice_incerteza_norm = (indice_incerteza * 1000) / palavras_totais
            resultado["incerteza_normalizada"] = indice_incerteza_norm

            if indice_incerteza_norm > 8:
                resultado["alerta_incerteza"] = "Alto índice de expressões de incerteza"

        # Análise de opinatividade
        indice_opiniao = sum(
            len(re.findall(r'\b' + re.escape(expressao) + r'\b', texto.lower()))
            for expressao in self.expressoes_opinativas
        )
        resultado["opinativo"] = indice_opiniao

        if palavras_totais > 0:
            indice_opiniao_norm = (indice_opiniao * 1000) / palavras_totais
            resultado["opinativo_normalizado"] = indice_opiniao_norm

            if indice_opiniao_norm > 5:
                resultado["alerta_opiniao"] = "Alto índice de expressões opinativas"

        # Contagem de pontos de exclamação (indicador de sensacionalismo)
        exclamacoes = texto.count('!')
        resultado["exclamacoes"] = exclamacoes

        if palavras_totais > 0:
            exclamacoes_norm = (exclamacoes * 1000) / palavras_totais
            if exclamacoes_norm > 10:
                resultado["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        # Análise de sentimento usando TextBlob
        try:
            analise = TextBlob(texto)
            resultado["sentimento_polaridade"] = analise.sentiment.polarity  # -1 a 1 (negativo a positivo)
            resultado["sentimento_subjetividade"] = analise.sentiment.subjectivity  # 0 a 1 (objetivo a subjetivo)

            # Categorização baseada na polaridade
            if resultado["sentimento_polaridade"] < -0.5:
                resultado["sentimento_categoria"] = "muito negativo"
            elif resultado["sentimento_polaridade"] < -0.1:
                resultado["sentimento_categoria"] = "negativo"
            elif resultado["sentimento_polaridade"] <= 0.1:
                resultado["sentimento_categoria"] = "neutro"
            elif resultado["sentimento_polaridade"] <= 0.5:
                resultado["sentimento_categoria"] = "positivo"
            else:
                resultado["sentimento_categoria"] = "muito positivo"

            # Alerta para alto nível de subjetividade em notícias que deveriam ser objetivas
            if resultado["sentimento_subjetividade"] > 0.6:
                resultado["alerta_subjetividade"] = "Conteúdo altamente subjetivo"

        except Exception as e:
            logger.warning(f"Erro na análise de sentimento: {e}")

        return resultado

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}
        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        # Analisar metadados das primeiras 5 imagens (limitar para evitar muito processamento)
        try:
            with ThreadPoolExecutor(max_workers=3) as executor:
                metadados = list(executor.map(self.extrair_metadados_imagem, [img['url'] for img in imagens[:5]]))
                resultados["metadados_imagens"] = [m for m in metadados if m]  # Filtrar None
        except Exception as e:
            logger.error(f"Erro ao analisar metadados das imagens: {e}")

        return resultados

    def extrair_metadados_imagem(self, url_imagem):
        """Extrai metadados de uma imagem a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url_imagem, headers=headers, stream=True, timeout=5, verify=False)
            response.raise_for_status()

            # Abrir a imagem com PIL
            img = Image.open(io.BytesIO(response.content))

            # Coletar metadados básicos
            metadados = {
                "formato": img.format,
                "tamanho": img.size,
                "modo": img.mode,
                "url": url_imagem,
                "hash": hashlib.md5(response.content).hexdigest()
            }

            # Tentar extrair EXIF se disponível
            if hasattr(img, '_getexif') and img._getexif():
                exif = img._getexif()
                if exif:
                    exif_data = {}
                    for tag_id, value in exif.items():
                        # Converter para string para garantir serialização JSON
                        exif_data[str(tag_id)] = str(value)
                    metadados["exif"] = exif_data

            return metadados
        except (requests.RequestException, UnidentifiedImageError, IOError) as e:
            logger.warning(f"Erro ao extrair metadados da imagem {url_imagem}: {e}")
            return None

    def buscar_fact_check(self, texto, titulo=None):
        """Busca por verificações de fatos relacionadas ao conteúdo."""
        resultados = {}

        try:
            # Preparar termos de busca
            query = titulo if titulo else texto[:100]

            # Consultar API de fact check
            data = self.consultar_fact_check(query)

            if data:
                resultados["fact_checks"] = data.get('claims', [])
                resultados["num_fact_checks"] = len(resultados["fact_checks"])

                # Analisar resultados
                if resultados["num_fact_checks"] > 0:
                    # Verificar ratings médios
                    ratings = []
                    for claim in resultados["fact_checks"]:
                        for review in claim.get('claimReview', []):
                            if 'textualRating' in review:
                                ratings.append(review['textualRating'].lower())

                    # Contar frequência de cada rating
                    if ratings:
                        counter = Counter(ratings)
                        resultados["ratings_frequencia"] = counter

                        # Detectar termos comuns de falsidade
                        termos_falso = ['false', 'fake', 'falso', 'mentira', 'enganoso', 'misleading']
                        count_falso = sum(counter.get(termo, 0) for termo in termos_falso)

                        if count_falso > 0:
                            resultados["alerta_fact_check"] = "Conteúdo marcado como falso ou enganoso por verificadores de fatos"
                            resultados["pontos_fact_check"] = -25
                        else:
                            resultados["pontos_fact_check"] = 0
            else:
                resultados["fact_checks"] = []
                resultados["num_fact_checks"] = 0
                resultados["pontos_fact_check"] = 0

        except Exception as e:
            logger.error(f"Erro ao buscar fact checks: {e}")
            resultados["erro_fact_check"] = str(e)
            resultados["pontos_fact_check"] = 0

        return resultados

    def buscar_noticias_similares(self, titulo, keywords, conteudo=None):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}
        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta

            # Simular consistência entre fontes
            resultados["consistencia_entre_fontes"] = "alta"  # baixa, média, alta
            resultados["pontos_consistencia"] = 10  # Simulando pontos por alta consistência

        except Exception as e:
            logger.error(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)
            resultados["pontos_consistencia"] = 0

        return resultados

    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto

    def analisar_credibilidade(self, url):
        """Análise completa de credibilidade de uma URL de notícia."""
        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo or not artigo.get('conteudo'):
            logger.error(f"Não foi possível baixar o artigo: {url}")
            return {"erro": "Não foi possível baixar o artigo para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 2. Verificação de fonte/domínio
        status_fonte, pontos_fonte, info_fonte = self.verificar_fonte(url)
        resultados.update(info_fonte)

        # 3. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 4. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 5. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 6. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)
        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 7. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 8. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 9. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', []),
            conteudo
        )
        resultados.update(resultados_similares)

        # 10. Verificação em serviços de fact-checking
        resultados_fact_check = self.buscar_fact_check(conteudo, artigo['titulo'])
        resultados.update(resultados_fact_check)

        # Cálculo mais sofisticado usando vários fatores
# 11. Determinar um nível de veracidade
# Cálculo mais sofisticado usando vários fatores
base_pontuacao = 70  # Base inicial neutra

# Fatores que afetam a pontuação
pontuacao_fatores = [
    resultados.get("pontos_fonte", 0),
    resultados.get("pontos_consistencia", 0),
    resultados.get("pontos_evidencia", 0),
    resultados.get("pontos_conhecimento_previo", 0),
    resultados.get("pontos_contexto", 0),
    resultados.get("pontos_bias", 0)
]

# Aplicar pesos aos fatores
pesos = [1.5, 1.2, 2.0, 1.0, 1.0, 0.8]
pontuacao_ponderada = sum(p * f for p, f in zip(pesos, pontuacao_fatores))

# Calcular pontuação final
pontuacao_final = max(0, min(100, base_pontuacao + pontuacao_ponderada))

# Determinar nível de veracidade baseado na pontuação final
if pontuacao_final >= 90:
    nivel_veracidade = "Altamente verificado"
elif pontuacao_final >= 75:
    nivel_veracidade = "Provavelmente verdadeiro"
elif pontuacao_final >= 60:
    nivel_veracidade = "Parcialmente verificado"
elif pontuacao_final >= 40:
    nivel_veracidade = "Inconclusivo"
elif pontuacao_final >= 25:
    nivel_veracidade = "Provavelmente falso"
else:
    nivel_veracidade = "Falso"

resultados["pontuacao_veracidade"] = pontuacao_final
resultados["nivel_veracidade"] = nivel_veracidade

# 12. Sumarizar os resultados
resumo = {
    "nivel_veracidade": nivel_veracidade,
    "pontuacao": pontuacao_final,
    "analise_detalhada": resultados,
    "pontos_chave": {
        "fontes": resultados.get("qualidade_fonte", "Não analisado"),
        "consistencia": resultados.get("consistencia_narrativa", "Não analisado"),
        "evidencias": resultados.get("qualidade_evidencia", "Não analisado"),
        "contexto": resultados.get("analise_contexto", "Não analisado")
    },
    "recomendacao": ""
}

# Adicionar uma recomendação baseada no nível de veracidade
def adicionar_recomendacao(pontuacao_final):
    """Adiciona uma recomendação ao resumo com base no nível de veracidade."""
    resumo = {}
    if pontuacao_final >= 75:
        resumo["recomendacao"] = "Informação confiável para uso e disseminação."
    elif pontuacao_final >= 50:
        resumo["recomendacao"] = "Verificar informações adicionais antes de aceitar completamente."
    else:
        resumo["recomendacao"] = "Não recomendado para uso ou compartilhamento sem verificação adicional."

    return resumo





Overwriting verificacao_simples.py


In [35]:
%run verificacao_simples.py

IndentationError: unindent does not match any outer indentation level (verificacao_simples.py, line 274)

In [34]:
%%writefile verificacao_simples.py

import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse, urlencode
import hashlib
from collections import Counter
import time
import logging
import base64
import io
import nltk
import os
import sys
import subprocess

try:
    from dotenv import load_dotenv
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-dotenv"])
    from dotenv import load_dotenv




from concurrent.futures import ThreadPoolExecutor
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
from PIL import Image, UnidentifiedImageError
from datetime import datetime

# Configurar o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("VerificadorNoticias")

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticias:
    def __init__(self):
        logger.info("Verificador de Notícias inicializado!")

        # Carregar variáveis do ambiente
        load_dotenv()
        self.fact_check_api_key = os.getenv("FACT_CHECK_API_KEY")
        self.fact_check_api_url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]
        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5, "viral": 1.5,
            "bizarro": 1.5, "aterrorizante": 2, "assustador": 1.5, "terrível": 1.5,
            "estarrecedor": 2, "impossível": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors",
            "parece que", "há relatos", "não confirmado", "segundo fontes",
            "indícios", "sinalizam", "aparentemente", "estaria", "poderia",
            "teria", "estima-se"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe",
            "na visão de", "do meu ponto de vista", "considero", "entendo que",
            "me parece", "creio que", "penso que", "sinto que"
        ]

        # Tentar baixar recursos NLTK necessários para análise de texto
        try:
            nltk.download('punkt', quiet=True)
        except Exception as e:
            logger.warning(f"Não foi possível baixar recursos NLTK: {e}")

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        return parsed_url.netloc

    def verificar_url_simples(self, url):
        """Executa uma verificação simples em uma URL de notícia."""
        try:
            # Extrair domínio
            parsed_url = urlparse(url)
            dominio = parsed_url.netloc

            # Tentar acessar a URL
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10, verify=False)
            response.raise_for_status()  # Levantar exceção para status HTTP de erro

            # Verificar status HTTP
            if response.status_code == 200:
                logger.info(f"URL acessada com sucesso: {url}")
                logger.info(f"Domínio: {dominio}")
                logger.info(f"Tamanho do conteúdo: {len(response.text)} caracteres.")
                return {
                    "status": "sucesso",
                    "dominio": dominio,
                    "tamanho_conteudo": len(response.text),
                    "mensagem": "URL acessada com sucesso"
                }
            else:
                logger.warning(f"Erro ao acessar a URL. Código HTTP: {response.status_code}")
                return {
                    "status": "erro",
                    "mensagem": f"Erro ao acessar a URL. Código HTTP: {response.status_code}"
                }

        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao acessar a URL: {e}")
            return {
                "status": "erro",
                "mensagem": str(e)
            }

    def analisar_sentimento(self, texto):
        """Analisa o sentimento do texto usando TextBlob."""
        try:
            sentiment = TextBlob(texto).sentiment
            return {
                "polaridade": sentiment.polarity,
                "subjetividade": sentiment.subjectivity
            }
        except Exception as e:
            logger.error(f"Erro na análise de sentimento: {e}")
            return {
                "polaridade": 0,
                "subjetividade": 0
            }

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {"dominio": dominio}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
            return "Fonte confiável", 20, resultado
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
            return "Fonte suspeita", -30, resultado
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0
            return "Fonte não categorizada", 0, resultado

    def baixar_artigo(self, url):
        """Baixa o conteúdo do artigo a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""
            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)
                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date'] or meta.get('name') in ['publication_date', 'date', 'pubdate']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]', '[name=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(author_tag)
                else:
                    elements = soup.find_all(author_tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords,
                'html': response.text  # Manter o HTML para possível análise adicional
            }
        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            logger.error(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None

   class VerificadorFactCheck:
    def __init__(self):
        self.fact_check_api_url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

    def consultar_fact_check(self, query):
        """Consulta a API do Google Fact Check Tools."""

        # Carregar variáveis de ambiente
        load_dotenv()
        api_key = os.getenv("FACT_CHECK_API_KEY")

        if not api_key or api_key.strip() == "":
            raise ValueError("Chave de API não encontrada no arquivo .env. Certifique-se de definir FACT_CHECK_API_KEY corretamente.")

        # Verificar se a URL base da API está configurada
        if not self.fact_check_api_url.strip():
            logger.warning("URL base para a API de Fact Check não configurada.")
            return None

        # Construir a URL da API
        url_api = f"{self.fact_check_api_url}?key={api_key}&query={query}"

        try:
            response = requests.get(url_api)
            if response.status_code == 200:
                return response.json()
            else:
                logger.warning(f"Falha ao acessar API de Fact Check. Status: {response.status_code}")
                return None
        except Exception as e:
            logger.error(f"Erro na API de Fact Check: {e}")
            return None



    def analisar_linguagem(self, texto):
        """Analisa o conteúdo em busca de sensacionalismo, incerteza e opinatividade."""
        if not texto or len(texto) < 10:
            return {
                "sensacionalismo": 0,
                "incerteza": 0,
                "opinativo": 0,
                "exclamacoes": 0
            }

        resultado = {}

        # Análise de sensacionalismo
        indice_sensacionalismo = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower())) * peso
            for palavra, peso in self.palavras_sensacionalistas.items()
        )
        resultado["sensacionalismo"] = indice_sensacionalismo

        # Normalizar por tamanho do texto (por 1000 palavras)
        palavras_totais = len(texto.split())
        if palavras_totais > 0:
            indice_sensacionalismo_norm = (indice_sensacionalismo * 1000) / palavras_totais
            resultado["sensacionalismo_normalizado"] = indice_sensacionalismo_norm

            # Adicionar alerta se o índice for alto
            if indice_sensacionalismo_norm > 10:
                resultado["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista"
            elif indice_sensacionalismo_norm > 5:
                resultado["alerta_sensacionalismo"] = "Moderado índice de linguagem sensacionalista"

        # Análise de incerteza
        indice_incerteza = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower()))
            for palavra in self.palavras_incerteza
        )
        resultado["incerteza"] = indice_incerteza

        if palavras_totais > 0:
            indice_incerteza_norm = (indice_incerteza * 1000) / palavras_totais
            resultado["incerteza_normalizada"] = indice_incerteza_norm

            if indice_incerteza_norm > 8:
                resultado["alerta_incerteza"] = "Alto índice de expressões de incerteza"

        # Análise de opinatividade
        indice_opiniao = sum(
            len(re.findall(r'\b' + re.escape(expressao) + r'\b', texto.lower()))
            for expressao in self.expressoes_opinativas
        )
        resultado["opinativo"] = indice_opiniao

        if palavras_totais > 0:
            indice_opiniao_norm = (indice_opiniao * 1000) / palavras_totais
            resultado["opinativo_normalizado"] = indice_opiniao_norm

            if indice_opiniao_norm > 5:
                resultado["alerta_opiniao"] = "Alto índice de expressões opinativas"

        # Contagem de pontos de exclamação (indicador de sensacionalismo)
        exclamacoes = texto.count('!')
        resultado["exclamacoes"] = exclamacoes

        if palavras_totais > 0:
            exclamacoes_norm = (exclamacoes * 1000) / palavras_totais
            if exclamacoes_norm > 10:
                resultado["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        # Análise de sentimento usando TextBlob
        try:
            analise = TextBlob(texto)
            resultado["sentimento_polaridade"] = analise.sentiment.polarity  # -1 a 1 (negativo a positivo)
            resultado["sentimento_subjetividade"] = analise.sentiment.subjectivity  # 0 a 1 (objetivo a subjetivo)

            # Categorização baseada na polaridade
            if resultado["sentimento_polaridade"] < -0.5:
                resultado["sentimento_categoria"] = "muito negativo"
            elif resultado["sentimento_polaridade"] < -0.1:
                resultado["sentimento_categoria"] = "negativo"
            elif resultado["sentimento_polaridade"] <= 0.1:
                resultado["sentimento_categoria"] = "neutro"
            elif resultado["sentimento_polaridade"] <= 0.5:
                resultado["sentimento_categoria"] = "positivo"
            else:
                resultado["sentimento_categoria"] = "muito positivo"

            # Alerta para alto nível de subjetividade em notícias que deveriam ser objetivas
            if resultado["sentimento_subjetividade"] > 0.6:
                resultado["alerta_subjetividade"] = "Conteúdo altamente subjetivo"

        except Exception as e:
            logger.warning(f"Erro na análise de sentimento: {e}")

        return resultado

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}
        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        # Analisar metadados das primeiras 5 imagens (limitar para evitar muito processamento)
        try:
            with ThreadPoolExecutor(max_workers=3) as executor:
                metadados = list(executor.map(self.extrair_metadados_imagem, [img['url'] for img in imagens[:5]]))
                resultados["metadados_imagens"] = [m for m in metadados if m]  # Filtrar None
        except Exception as e:
            logger.error(f"Erro ao analisar metadados das imagens: {e}")

        return resultados

    def extrair_metadados_imagem(self, url_imagem):
        """Extrai metadados de uma imagem a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url_imagem, headers=headers, stream=True, timeout=5, verify=False)
            response.raise_for_status()

            # Abrir a imagem com PIL
            img = Image.open(io.BytesIO(response.content))

            # Coletar metadados básicos
            metadados = {
                "formato": img.format,
                "tamanho": img.size,
                "modo": img.mode,
                "url": url_imagem,
                "hash": hashlib.md5(response.content).hexdigest()
            }

            # Tentar extrair EXIF se disponível
            if hasattr(img, '_getexif') and img._getexif():
                exif = img._getexif()
                if exif:
                    exif_data = {}
                    for tag_id, value in exif.items():
                        # Converter para string para garantir serialização JSON
                        exif_data[str(tag_id)] = str(value)
                    metadados["exif"] = exif_data

            return metadados
        except (requests.RequestException, UnidentifiedImageError, IOError) as e:
            logger.warning(f"Erro ao extrair metadados da imagem {url_imagem}: {e}")
            return None

    def buscar_fact_check(self, texto, titulo=None):
        """Busca por verificações de fatos relacionadas ao conteúdo."""
        resultados = {}

        try:
            # Preparar termos de busca
            query = titulo if titulo else texto[:100]

            # Consultar API de fact check
            data = self.consultar_fact_check(query)

            if data:
                resultados["fact_checks"] = data.get('claims', [])
                resultados["num_fact_checks"] = len(resultados["fact_checks"])

                # Analisar resultados
                if resultados["num_fact_checks"] > 0:
                    # Verificar ratings médios
                    ratings = []
                    for claim in resultados["fact_checks"]:
                        for review in claim.get('claimReview', []):
                            if 'textualRating' in review:
                                ratings.append(review['textualRating'].lower())

                    # Contar frequência de cada rating
                    if ratings:
                        counter = Counter(ratings)
                        resultados["ratings_frequencia"] = counter

                        # Detectar termos comuns de falsidade
                        termos_falso = ['false', 'fake', 'falso', 'mentira', 'enganoso', 'misleading']
                        count_falso = sum(counter.get(termo, 0) for termo in termos_falso)

                        if count_falso > 0:
                            resultados["alerta_fact_check"] = "Conteúdo marcado como falso ou enganoso por verificadores de fatos"
                            resultados["pontos_fact_check"] = -25
                        else:
                            resultados["pontos_fact_check"] = 0
            else:
                resultados["fact_checks"] = []
                resultados["num_fact_checks"] = 0
                resultados["pontos_fact_check"] = 0

        except Exception as e:
            logger.error(f"Erro ao buscar fact checks: {e}")
            resultados["erro_fact_check"] = str(e)
            resultados["pontos_fact_check"] = 0

        return resultados

    def buscar_noticias_similares(self, titulo, keywords, conteudo=None):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}
        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta

            # Simular consistência entre fontes
            resultados["consistencia_entre_fontes"] = "alta"  # baixa, média, alta
            resultados["pontos_consistencia"] = 10  # Simulando pontos por alta consistência

        except Exception as e:
            logger.error(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)
            resultados["pontos_consistencia"] = 0

        return resultados

    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto

    def analisar_credibilidade(self, url):
        """Análise completa de credibilidade de uma URL de notícia."""
        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo or not artigo.get('conteudo'):
            logger.error(f"Não foi possível baixar o artigo: {url}")
            return {"erro": "Não foi possível baixar o artigo para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 2. Verificação de fonte/domínio
        status_fonte, pontos_fonte, info_fonte = self.verificar_fonte(url)
        resultados.update(info_fonte)

        # 3. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 4. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 5. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 6. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)
        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 7. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 8. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 9. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', []),
            conteudo
        )
        resultados.update(resultados_similares)

        # 10. Verificação em serviços de fact-checking
        resultados_fact_check = self.buscar_fact_check(conteudo, artigo['titulo'])
        resultados.update(resultados_fact_check)

        # Cálculo mais sofisticado usando vários fatores
# 11. Determinar um nível de veracidade
# Cálculo mais sofisticado usando vários fatores
base_pontuacao = 70  # Base inicial neutra

# Fatores que afetam a pontuação
pontuacao_fatores = [
    resultados.get("pontos_fonte", 0),
    resultados.get("pontos_consistencia", 0),
    resultados.get("pontos_evidencia", 0),
    resultados.get("pontos_conhecimento_previo", 0),
    resultados.get("pontos_contexto", 0),
    resultados.get("pontos_bias", 0)
]

# Aplicar pesos aos fatores
pesos = [1.5, 1.2, 2.0, 1.0, 1.0, 0.8]
pontuacao_ponderada = sum(p * f for p, f in zip(pesos, pontuacao_fatores))

# Calcular pontuação final
pontuacao_final = max(0, min(100, base_pontuacao + pontuacao_ponderada))

# Determinar nível de veracidade baseado na pontuação final
if pontuacao_final >= 90:
    nivel_veracidade = "Altamente verificado"
elif pontuacao_final >= 75:
    nivel_veracidade = "Provavelmente verdadeiro"
elif pontuacao_final >= 60:
    nivel_veracidade = "Parcialmente verificado"
elif pontuacao_final >= 40:
    nivel_veracidade = "Inconclusivo"
elif pontuacao_final >= 25:
    nivel_veracidade = "Provavelmente falso"
else:
    nivel_veracidade = "Falso"

resultados["pontuacao_veracidade"] = pontuacao_final
resultados["nivel_veracidade"] = nivel_veracidade

# 12. Sumarizar os resultados
resumo = {
    "nivel_veracidade": nivel_veracidade,
    "pontuacao": pontuacao_final,
    "analise_detalhada": resultados,
    "pontos_chave": {
        "fontes": resultados.get("qualidade_fonte", "Não analisado"),
        "consistencia": resultados.get("consistencia_narrativa", "Não analisado"),
        "evidencias": resultados.get("qualidade_evidencia", "Não analisado"),
        "contexto": resultados.get("analise_contexto", "Não analisado")
    },
    "recomendacao": ""
}

# Adicionar uma recomendação baseada no nível de veracidade
def adicionar_recomendacao(pontuacao_final):
    """Adiciona uma recomendação ao resumo com base no nível de veracidade."""
    resumo = {}
    if pontuacao_final >= 75:
        resumo["recomendacao"] = "Informação confiável para uso e disseminação."
    elif pontuacao_final >= 50:
        resumo["recomendacao"] = "Verificar informações adicionais antes de aceitar completamente."
    else:
        resumo["recomendacao"] = "Não recomendado para uso ou compartilhamento sem verificação adicional."

    return resumo





Overwriting verificacao_simples.py


In [33]:
%run verificacao_simples.py

IndentationError: unindent does not match any outer indentation level (verificacao_simples.py, line 273)

In [32]:
%%writefile verificacao_simples.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse, urlencode
import hashlib
from collections import Counter
import time
import logging
import base64
import io
import nltk
import os
import sys
import subprocess

try:
    from dotenv import load_dotenv
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-dotenv"])
    from dotenv import load_dotenv




from concurrent.futures import ThreadPoolExecutor
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
from PIL import Image, UnidentifiedImageError
from datetime import datetime

# Configurar o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("VerificadorNoticias")

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticias:
    def __init__(self):
        logger.info("Verificador de Notícias inicializado!")

        # Carregar variáveis do ambiente
        load_dotenv()
        self.fact_check_api_key = os.getenv("FACT_CHECK_API_KEY")
        self.fact_check_api_url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]
        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5, "viral": 1.5,
            "bizarro": 1.5, "aterrorizante": 2, "assustador": 1.5, "terrível": 1.5,
            "estarrecedor": 2, "impossível": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors",
            "parece que", "há relatos", "não confirmado", "segundo fontes",
            "indícios", "sinalizam", "aparentemente", "estaria", "poderia",
            "teria", "estima-se"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe",
            "na visão de", "do meu ponto de vista", "considero", "entendo que",
            "me parece", "creio que", "penso que", "sinto que"
        ]

        # Tentar baixar recursos NLTK necessários para análise de texto
        try:
            nltk.download('punkt', quiet=True)
        except Exception as e:
            logger.warning(f"Não foi possível baixar recursos NLTK: {e}")

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        return parsed_url.netloc

    def verificar_url_simples(self, url):
        """Executa uma verificação simples em uma URL de notícia."""
        try:
            # Extrair domínio
            parsed_url = urlparse(url)
            dominio = parsed_url.netloc

            # Tentar acessar a URL
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10, verify=False)
            response.raise_for_status()  # Levantar exceção para status HTTP de erro

            # Verificar status HTTP
            if response.status_code == 200:
                logger.info(f"URL acessada com sucesso: {url}")
                logger.info(f"Domínio: {dominio}")
                logger.info(f"Tamanho do conteúdo: {len(response.text)} caracteres.")
                return {
                    "status": "sucesso",
                    "dominio": dominio,
                    "tamanho_conteudo": len(response.text),
                    "mensagem": "URL acessada com sucesso"
                }
            else:
                logger.warning(f"Erro ao acessar a URL. Código HTTP: {response.status_code}")
                return {
                    "status": "erro",
                    "mensagem": f"Erro ao acessar a URL. Código HTTP: {response.status_code}"
                }

        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao acessar a URL: {e}")
            return {
                "status": "erro",
                "mensagem": str(e)
            }

    def analisar_sentimento(self, texto):
        """Analisa o sentimento do texto usando TextBlob."""
        try:
            sentiment = TextBlob(texto).sentiment
            return {
                "polaridade": sentiment.polarity,
                "subjetividade": sentiment.subjectivity
            }
        except Exception as e:
            logger.error(f"Erro na análise de sentimento: {e}")
            return {
                "polaridade": 0,
                "subjetividade": 0
            }

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {"dominio": dominio}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
            return "Fonte confiável", 20, resultado
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
            return "Fonte suspeita", -30, resultado
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0
            return "Fonte não categorizada", 0, resultado

    def baixar_artigo(self, url):
        """Baixa o conteúdo do artigo a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""
            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)
                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date'] or meta.get('name') in ['publication_date', 'date', 'pubdate']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]', '[name=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(author_tag)
                else:
                    elements = soup.find_all(author_tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords,
                'html': response.text  # Manter o HTML para possível análise adicional
            }
        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            logger.error(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None

   def consultar_fact_check(self, query):
    """Consulta a API do Google Fact Check Tools."""

    # Carregar variáveis de ambiente
    load_dotenv()
    api_key = os.getenv("FACT_CHECK_API_KEY")

    if not api_key or api_key.strip() == "":
        raise ValueError("Chave de API não encontrada no arquivo .env. Certifique-se de definir FACT_CHECK_API_KEY corretamente.")

    # Verificar se a URL base da API está configurada
    if not hasattr(self, 'fact_check_api_url') or not self.fact_check_api_url.strip():
        logger.warning("URL base para a API de Fact Check não configurada.")
        return None

    # Construir a URL da API
    url_api = f"{self.fact_check_api_url}?key={api_key}&query={query}"

    try:
        response = requests.get(url_api)
        if response.status_code == 200:
            return response.json()
        else:
            logger.warning(f"Falha ao acessar API de Fact Check. Status: {response.status_code}")
            return None
    except Exception as e:
        logger.error(f"Erro na API de Fact Check: {e}")
        return None


    def analisar_linguagem(self, texto):
        """Analisa o conteúdo em busca de sensacionalismo, incerteza e opinatividade."""
        if not texto or len(texto) < 10:
            return {
                "sensacionalismo": 0,
                "incerteza": 0,
                "opinativo": 0,
                "exclamacoes": 0
            }

        resultado = {}

        # Análise de sensacionalismo
        indice_sensacionalismo = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower())) * peso
            for palavra, peso in self.palavras_sensacionalistas.items()
        )
        resultado["sensacionalismo"] = indice_sensacionalismo

        # Normalizar por tamanho do texto (por 1000 palavras)
        palavras_totais = len(texto.split())
        if palavras_totais > 0:
            indice_sensacionalismo_norm = (indice_sensacionalismo * 1000) / palavras_totais
            resultado["sensacionalismo_normalizado"] = indice_sensacionalismo_norm

            # Adicionar alerta se o índice for alto
            if indice_sensacionalismo_norm > 10:
                resultado["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista"
            elif indice_sensacionalismo_norm > 5:
                resultado["alerta_sensacionalismo"] = "Moderado índice de linguagem sensacionalista"

        # Análise de incerteza
        indice_incerteza = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower()))
            for palavra in self.palavras_incerteza
        )
        resultado["incerteza"] = indice_incerteza

        if palavras_totais > 0:
            indice_incerteza_norm = (indice_incerteza * 1000) / palavras_totais
            resultado["incerteza_normalizada"] = indice_incerteza_norm

            if indice_incerteza_norm > 8:
                resultado["alerta_incerteza"] = "Alto índice de expressões de incerteza"

        # Análise de opinatividade
        indice_opiniao = sum(
            len(re.findall(r'\b' + re.escape(expressao) + r'\b', texto.lower()))
            for expressao in self.expressoes_opinativas
        )
        resultado["opinativo"] = indice_opiniao

        if palavras_totais > 0:
            indice_opiniao_norm = (indice_opiniao * 1000) / palavras_totais
            resultado["opinativo_normalizado"] = indice_opiniao_norm

            if indice_opiniao_norm > 5:
                resultado["alerta_opiniao"] = "Alto índice de expressões opinativas"

        # Contagem de pontos de exclamação (indicador de sensacionalismo)
        exclamacoes = texto.count('!')
        resultado["exclamacoes"] = exclamacoes

        if palavras_totais > 0:
            exclamacoes_norm = (exclamacoes * 1000) / palavras_totais
            if exclamacoes_norm > 10:
                resultado["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        # Análise de sentimento usando TextBlob
        try:
            analise = TextBlob(texto)
            resultado["sentimento_polaridade"] = analise.sentiment.polarity  # -1 a 1 (negativo a positivo)
            resultado["sentimento_subjetividade"] = analise.sentiment.subjectivity  # 0 a 1 (objetivo a subjetivo)

            # Categorização baseada na polaridade
            if resultado["sentimento_polaridade"] < -0.5:
                resultado["sentimento_categoria"] = "muito negativo"
            elif resultado["sentimento_polaridade"] < -0.1:
                resultado["sentimento_categoria"] = "negativo"
            elif resultado["sentimento_polaridade"] <= 0.1:
                resultado["sentimento_categoria"] = "neutro"
            elif resultado["sentimento_polaridade"] <= 0.5:
                resultado["sentimento_categoria"] = "positivo"
            else:
                resultado["sentimento_categoria"] = "muito positivo"

            # Alerta para alto nível de subjetividade em notícias que deveriam ser objetivas
            if resultado["sentimento_subjetividade"] > 0.6:
                resultado["alerta_subjetividade"] = "Conteúdo altamente subjetivo"

        except Exception as e:
            logger.warning(f"Erro na análise de sentimento: {e}")

        return resultado

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}
        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        # Analisar metadados das primeiras 5 imagens (limitar para evitar muito processamento)
        try:
            with ThreadPoolExecutor(max_workers=3) as executor:
                metadados = list(executor.map(self.extrair_metadados_imagem, [img['url'] for img in imagens[:5]]))
                resultados["metadados_imagens"] = [m for m in metadados if m]  # Filtrar None
        except Exception as e:
            logger.error(f"Erro ao analisar metadados das imagens: {e}")

        return resultados

    def extrair_metadados_imagem(self, url_imagem):
        """Extrai metadados de uma imagem a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url_imagem, headers=headers, stream=True, timeout=5, verify=False)
            response.raise_for_status()

            # Abrir a imagem com PIL
            img = Image.open(io.BytesIO(response.content))

            # Coletar metadados básicos
            metadados = {
                "formato": img.format,
                "tamanho": img.size,
                "modo": img.mode,
                "url": url_imagem,
                "hash": hashlib.md5(response.content).hexdigest()
            }

            # Tentar extrair EXIF se disponível
            if hasattr(img, '_getexif') and img._getexif():
                exif = img._getexif()
                if exif:
                    exif_data = {}
                    for tag_id, value in exif.items():
                        # Converter para string para garantir serialização JSON
                        exif_data[str(tag_id)] = str(value)
                    metadados["exif"] = exif_data

            return metadados
        except (requests.RequestException, UnidentifiedImageError, IOError) as e:
            logger.warning(f"Erro ao extrair metadados da imagem {url_imagem}: {e}")
            return None

    def buscar_fact_check(self, texto, titulo=None):
        """Busca por verificações de fatos relacionadas ao conteúdo."""
        resultados = {}

        try:
            # Preparar termos de busca
            query = titulo if titulo else texto[:100]

            # Consultar API de fact check
            data = self.consultar_fact_check(query)

            if data:
                resultados["fact_checks"] = data.get('claims', [])
                resultados["num_fact_checks"] = len(resultados["fact_checks"])

                # Analisar resultados
                if resultados["num_fact_checks"] > 0:
                    # Verificar ratings médios
                    ratings = []
                    for claim in resultados["fact_checks"]:
                        for review in claim.get('claimReview', []):
                            if 'textualRating' in review:
                                ratings.append(review['textualRating'].lower())

                    # Contar frequência de cada rating
                    if ratings:
                        counter = Counter(ratings)
                        resultados["ratings_frequencia"] = counter

                        # Detectar termos comuns de falsidade
                        termos_falso = ['false', 'fake', 'falso', 'mentira', 'enganoso', 'misleading']
                        count_falso = sum(counter.get(termo, 0) for termo in termos_falso)

                        if count_falso > 0:
                            resultados["alerta_fact_check"] = "Conteúdo marcado como falso ou enganoso por verificadores de fatos"
                            resultados["pontos_fact_check"] = -25
                        else:
                            resultados["pontos_fact_check"] = 0
            else:
                resultados["fact_checks"] = []
                resultados["num_fact_checks"] = 0
                resultados["pontos_fact_check"] = 0

        except Exception as e:
            logger.error(f"Erro ao buscar fact checks: {e}")
            resultados["erro_fact_check"] = str(e)
            resultados["pontos_fact_check"] = 0

        return resultados

    def buscar_noticias_similares(self, titulo, keywords, conteudo=None):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}
        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta

            # Simular consistência entre fontes
            resultados["consistencia_entre_fontes"] = "alta"  # baixa, média, alta
            resultados["pontos_consistencia"] = 10  # Simulando pontos por alta consistência

        except Exception as e:
            logger.error(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)
            resultados["pontos_consistencia"] = 0

        return resultados

    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto

    def analisar_credibilidade(self, url):
        """Análise completa de credibilidade de uma URL de notícia."""
        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo or not artigo.get('conteudo'):
            logger.error(f"Não foi possível baixar o artigo: {url}")
            return {"erro": "Não foi possível baixar o artigo para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 2. Verificação de fonte/domínio
        status_fonte, pontos_fonte, info_fonte = self.verificar_fonte(url)
        resultados.update(info_fonte)

        # 3. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 4. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 5. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 6. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)
        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 7. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 8. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 9. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', []),
            conteudo
        )
        resultados.update(resultados_similares)

        # 10. Verificação em serviços de fact-checking
        resultados_fact_check = self.buscar_fact_check(conteudo, artigo['titulo'])
        resultados.update(resultados_fact_check)

        # Cálculo mais sofisticado usando vários fatores
# 11. Determinar um nível de veracidade
# Cálculo mais sofisticado usando vários fatores
base_pontuacao = 70  # Base inicial neutra

# Fatores que afetam a pontuação
pontuacao_fatores = [
    resultados.get("pontos_fonte", 0),
    resultados.get("pontos_consistencia", 0),
    resultados.get("pontos_evidencia", 0),
    resultados.get("pontos_conhecimento_previo", 0),
    resultados.get("pontos_contexto", 0),
    resultados.get("pontos_bias", 0)
]

# Aplicar pesos aos fatores
pesos = [1.5, 1.2, 2.0, 1.0, 1.0, 0.8]
pontuacao_ponderada = sum(p * f for p, f in zip(pesos, pontuacao_fatores))

# Calcular pontuação final
pontuacao_final = max(0, min(100, base_pontuacao + pontuacao_ponderada))

# Determinar nível de veracidade baseado na pontuação final
if pontuacao_final >= 90:
    nivel_veracidade = "Altamente verificado"
elif pontuacao_final >= 75:
    nivel_veracidade = "Provavelmente verdadeiro"
elif pontuacao_final >= 60:
    nivel_veracidade = "Parcialmente verificado"
elif pontuacao_final >= 40:
    nivel_veracidade = "Inconclusivo"
elif pontuacao_final >= 25:
    nivel_veracidade = "Provavelmente falso"
else:
    nivel_veracidade = "Falso"

resultados["pontuacao_veracidade"] = pontuacao_final
resultados["nivel_veracidade"] = nivel_veracidade

# 12. Sumarizar os resultados
resumo = {
    "nivel_veracidade": nivel_veracidade,
    "pontuacao": pontuacao_final,
    "analise_detalhada": resultados,
    "pontos_chave": {
        "fontes": resultados.get("qualidade_fonte", "Não analisado"),
        "consistencia": resultados.get("consistencia_narrativa", "Não analisado"),
        "evidencias": resultados.get("qualidade_evidencia", "Não analisado"),
        "contexto": resultados.get("analise_contexto", "Não analisado")
    },
    "recomendacao": ""
}

# Adicionar uma recomendação baseada no nível de veracidade
def adicionar_recomendacao(pontuacao_final):
    """Adiciona uma recomendação ao resumo com base no nível de veracidade."""
    resumo = {}
    if pontuacao_final >= 75:
        resumo["recomendacao"] = "Informação confiável para uso e disseminação."
    elif pontuacao_final >= 50:
        resumo["recomendacao"] = "Verificar informações adicionais antes de aceitar completamente."
    else:
        resumo["recomendacao"] = "Não recomendado para uso ou compartilhamento sem verificação adicional."

    return resumo





Overwriting verificacao_simples.py


In [31]:
%run verificacao_simples.py

NameError: name 'resultados' is not defined

In [30]:
%%writefile verificacao_simples.py

import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse, urlencode
import hashlib
from collections import Counter
import time
import logging
import base64
import io
import nltk
import os
import sys
import subprocess

try:
    from dotenv import load_dotenv
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-dotenv"])
    from dotenv import load_dotenv




from concurrent.futures import ThreadPoolExecutor
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
from PIL import Image, UnidentifiedImageError
from datetime import datetime

# Configurar o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("VerificadorNoticias")

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticias:
    def __init__(self):
        logger.info("Verificador de Notícias inicializado!")

        # Carregar variáveis do ambiente
        load_dotenv()
        self.fact_check_api_key = os.getenv("FACT_CHECK_API_KEY")
        self.fact_check_api_url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]
        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5, "viral": 1.5,
            "bizarro": 1.5, "aterrorizante": 2, "assustador": 1.5, "terrível": 1.5,
            "estarrecedor": 2, "impossível": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors",
            "parece que", "há relatos", "não confirmado", "segundo fontes",
            "indícios", "sinalizam", "aparentemente", "estaria", "poderia",
            "teria", "estima-se"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe",
            "na visão de", "do meu ponto de vista", "considero", "entendo que",
            "me parece", "creio que", "penso que", "sinto que"
        ]

        # Tentar baixar recursos NLTK necessários para análise de texto
        try:
            nltk.download('punkt', quiet=True)
        except Exception as e:
            logger.warning(f"Não foi possível baixar recursos NLTK: {e}")

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        return parsed_url.netloc

    def verificar_url_simples(self, url):
        """Executa uma verificação simples em uma URL de notícia."""
        try:
            # Extrair domínio
            parsed_url = urlparse(url)
            dominio = parsed_url.netloc

            # Tentar acessar a URL
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10, verify=False)
            response.raise_for_status()  # Levantar exceção para status HTTP de erro

            # Verificar status HTTP
            if response.status_code == 200:
                logger.info(f"URL acessada com sucesso: {url}")
                logger.info(f"Domínio: {dominio}")
                logger.info(f"Tamanho do conteúdo: {len(response.text)} caracteres.")
                return {
                    "status": "sucesso",
                    "dominio": dominio,
                    "tamanho_conteudo": len(response.text),
                    "mensagem": "URL acessada com sucesso"
                }
            else:
                logger.warning(f"Erro ao acessar a URL. Código HTTP: {response.status_code}")
                return {
                    "status": "erro",
                    "mensagem": f"Erro ao acessar a URL. Código HTTP: {response.status_code}"
                }

        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao acessar a URL: {e}")
            return {
                "status": "erro",
                "mensagem": str(e)
            }

    def analisar_sentimento(self, texto):
        """Analisa o sentimento do texto usando TextBlob."""
        try:
            sentiment = TextBlob(texto).sentiment
            return {
                "polaridade": sentiment.polarity,
                "subjetividade": sentiment.subjectivity
            }
        except Exception as e:
            logger.error(f"Erro na análise de sentimento: {e}")
            return {
                "polaridade": 0,
                "subjetividade": 0
            }

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {"dominio": dominio}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
            return "Fonte confiável", 20, resultado
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
            return "Fonte suspeita", -30, resultado
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0
            return "Fonte não categorizada", 0, resultado

    def baixar_artigo(self, url):
        """Baixa o conteúdo do artigo a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""
            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)
                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date'] or meta.get('name') in ['publication_date', 'date', 'pubdate']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]', '[name=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(author_tag)
                else:
                    elements = soup.find_all(author_tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords,
                'html': response.text  # Manter o HTML para possível análise adicional
            }
        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            logger.error(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None

    def consultar_fact_check(self, query):
        """Consulta a API do Google Fact Check Tools."""
        if not self.fact_check_api_key:
            logger.warning("Chave de API para fact check não configurada")
            return None

        url_api = f"{self.fact_check_api_url}?key={self.fact_check_api_key}&query={query}"
        try:
            response = requests.get(url_api)
            if response.status_code == 200:
                return response.json()
            else:
                logger.warning(f"Falha ao acessar API de Fact Check. Status: {response.status_code}")
                return None
        except Exception as e:
            logger.error(f"Erro na API de Fact Check: {e}")
            return None

    def analisar_linguagem(self, texto):
        """Analisa o conteúdo em busca de sensacionalismo, incerteza e opinatividade."""
        if not texto or len(texto) < 10:
            return {
                "sensacionalismo": 0,
                "incerteza": 0,
                "opinativo": 0,
                "exclamacoes": 0
            }

        resultado = {}

        # Análise de sensacionalismo
        indice_sensacionalismo = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower())) * peso
            for palavra, peso in self.palavras_sensacionalistas.items()
        )
        resultado["sensacionalismo"] = indice_sensacionalismo

        # Normalizar por tamanho do texto (por 1000 palavras)
        palavras_totais = len(texto.split())
        if palavras_totais > 0:
            indice_sensacionalismo_norm = (indice_sensacionalismo * 1000) / palavras_totais
            resultado["sensacionalismo_normalizado"] = indice_sensacionalismo_norm

            # Adicionar alerta se o índice for alto
            if indice_sensacionalismo_norm > 10:
                resultado["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista"
            elif indice_sensacionalismo_norm > 5:
                resultado["alerta_sensacionalismo"] = "Moderado índice de linguagem sensacionalista"

        # Análise de incerteza
        indice_incerteza = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower()))
            for palavra in self.palavras_incerteza
        )
        resultado["incerteza"] = indice_incerteza

        if palavras_totais > 0:
            indice_incerteza_norm = (indice_incerteza * 1000) / palavras_totais
            resultado["incerteza_normalizada"] = indice_incerteza_norm

            if indice_incerteza_norm > 8:
                resultado["alerta_incerteza"] = "Alto índice de expressões de incerteza"

        # Análise de opinatividade
        indice_opiniao = sum(
            len(re.findall(r'\b' + re.escape(expressao) + r'\b', texto.lower()))
            for expressao in self.expressoes_opinativas
        )
        resultado["opinativo"] = indice_opiniao

        if palavras_totais > 0:
            indice_opiniao_norm = (indice_opiniao * 1000) / palavras_totais
            resultado["opinativo_normalizado"] = indice_opiniao_norm

            if indice_opiniao_norm > 5:
                resultado["alerta_opiniao"] = "Alto índice de expressões opinativas"

        # Contagem de pontos de exclamação (indicador de sensacionalismo)
        exclamacoes = texto.count('!')
        resultado["exclamacoes"] = exclamacoes

        if palavras_totais > 0:
            exclamacoes_norm = (exclamacoes * 1000) / palavras_totais
            if exclamacoes_norm > 10:
                resultado["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        # Análise de sentimento usando TextBlob
        try:
            analise = TextBlob(texto)
            resultado["sentimento_polaridade"] = analise.sentiment.polarity  # -1 a 1 (negativo a positivo)
            resultado["sentimento_subjetividade"] = analise.sentiment.subjectivity  # 0 a 1 (objetivo a subjetivo)

            # Categorização baseada na polaridade
            if resultado["sentimento_polaridade"] < -0.5:
                resultado["sentimento_categoria"] = "muito negativo"
            elif resultado["sentimento_polaridade"] < -0.1:
                resultado["sentimento_categoria"] = "negativo"
            elif resultado["sentimento_polaridade"] <= 0.1:
                resultado["sentimento_categoria"] = "neutro"
            elif resultado["sentimento_polaridade"] <= 0.5:
                resultado["sentimento_categoria"] = "positivo"
            else:
                resultado["sentimento_categoria"] = "muito positivo"

            # Alerta para alto nível de subjetividade em notícias que deveriam ser objetivas
            if resultado["sentimento_subjetividade"] > 0.6:
                resultado["alerta_subjetividade"] = "Conteúdo altamente subjetivo"

        except Exception as e:
            logger.warning(f"Erro na análise de sentimento: {e}")

        return resultado

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}
        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        # Analisar metadados das primeiras 5 imagens (limitar para evitar muito processamento)
        try:
            with ThreadPoolExecutor(max_workers=3) as executor:
                metadados = list(executor.map(self.extrair_metadados_imagem, [img['url'] for img in imagens[:5]]))
                resultados["metadados_imagens"] = [m for m in metadados if m]  # Filtrar None
        except Exception as e:
            logger.error(f"Erro ao analisar metadados das imagens: {e}")

        return resultados

    def extrair_metadados_imagem(self, url_imagem):
        """Extrai metadados de uma imagem a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url_imagem, headers=headers, stream=True, timeout=5, verify=False)
            response.raise_for_status()

            # Abrir a imagem com PIL
            img = Image.open(io.BytesIO(response.content))

            # Coletar metadados básicos
            metadados = {
                "formato": img.format,
                "tamanho": img.size,
                "modo": img.mode,
                "url": url_imagem,
                "hash": hashlib.md5(response.content).hexdigest()
            }

            # Tentar extrair EXIF se disponível
            if hasattr(img, '_getexif') and img._getexif():
                exif = img._getexif()
                if exif:
                    exif_data = {}
                    for tag_id, value in exif.items():
                        # Converter para string para garantir serialização JSON
                        exif_data[str(tag_id)] = str(value)
                    metadados["exif"] = exif_data

            return metadados
        except (requests.RequestException, UnidentifiedImageError, IOError) as e:
            logger.warning(f"Erro ao extrair metadados da imagem {url_imagem}: {e}")
            return None

    def buscar_fact_check(self, texto, titulo=None):
        """Busca por verificações de fatos relacionadas ao conteúdo."""
        resultados = {}

        try:
            # Preparar termos de busca
            query = titulo if titulo else texto[:100]

            # Consultar API de fact check
            data = self.consultar_fact_check(query)

            if data:
                resultados["fact_checks"] = data.get('claims', [])
                resultados["num_fact_checks"] = len(resultados["fact_checks"])

                # Analisar resultados
                if resultados["num_fact_checks"] > 0:
                    # Verificar ratings médios
                    ratings = []
                    for claim in resultados["fact_checks"]:
                        for review in claim.get('claimReview', []):
                            if 'textualRating' in review:
                                ratings.append(review['textualRating'].lower())

                    # Contar frequência de cada rating
                    if ratings:
                        counter = Counter(ratings)
                        resultados["ratings_frequencia"] = counter

                        # Detectar termos comuns de falsidade
                        termos_falso = ['false', 'fake', 'falso', 'mentira', 'enganoso', 'misleading']
                        count_falso = sum(counter.get(termo, 0) for termo in termos_falso)

                        if count_falso > 0:
                            resultados["alerta_fact_check"] = "Conteúdo marcado como falso ou enganoso por verificadores de fatos"
                            resultados["pontos_fact_check"] = -25
                        else:
                            resultados["pontos_fact_check"] = 0
            else:
                resultados["fact_checks"] = []
                resultados["num_fact_checks"] = 0
                resultados["pontos_fact_check"] = 0

        except Exception as e:
            logger.error(f"Erro ao buscar fact checks: {e}")
            resultados["erro_fact_check"] = str(e)
            resultados["pontos_fact_check"] = 0

        return resultados

    def buscar_noticias_similares(self, titulo, keywords, conteudo=None):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}
        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta

            # Simular consistência entre fontes
            resultados["consistencia_entre_fontes"] = "alta"  # baixa, média, alta
            resultados["pontos_consistencia"] = 10  # Simulando pontos por alta consistência

        except Exception as e:
            logger.error(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)
            resultados["pontos_consistencia"] = 0

        return resultados

    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto

    def analisar_credibilidade(self, url):
        """Análise completa de credibilidade de uma URL de notícia."""
        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo or not artigo.get('conteudo'):
            logger.error(f"Não foi possível baixar o artigo: {url}")
            return {"erro": "Não foi possível baixar o artigo para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 2. Verificação de fonte/domínio
        status_fonte, pontos_fonte, info_fonte = self.verificar_fonte(url)
        resultados.update(info_fonte)

        # 3. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 4. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 5. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 6. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)
        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 7. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 8. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 9. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', []),
            conteudo
        )
        resultados.update(resultados_similares)

        # 10. Verificação em serviços de fact-checking
        resultados_fact_check = self.buscar_fact_check(conteudo, artigo['titulo'])
        resultados.update(resultados_fact_check)

        # Cálculo mais sofisticado usando vários fatores
# 11. Determinar um nível de veracidade
# Cálculo mais sofisticado usando vários fatores
base_pontuacao = 70  # Base inicial neutra

# Fatores que afetam a pontuação
pontuacao_fatores = [
    resultados.get("pontos_fonte", 0),
    resultados.get("pontos_consistencia", 0),
    resultados.get("pontos_evidencia", 0),
    resultados.get("pontos_conhecimento_previo", 0),
    resultados.get("pontos_contexto", 0),
    resultados.get("pontos_bias", 0)
]

# Aplicar pesos aos fatores
pesos = [1.5, 1.2, 2.0, 1.0, 1.0, 0.8]
pontuacao_ponderada = sum(p * f for p, f in zip(pesos, pontuacao_fatores))

# Calcular pontuação final
pontuacao_final = max(0, min(100, base_pontuacao + pontuacao_ponderada))

# Determinar nível de veracidade baseado na pontuação final
if pontuacao_final >= 90:
    nivel_veracidade = "Altamente verificado"
elif pontuacao_final >= 75:
    nivel_veracidade = "Provavelmente verdadeiro"
elif pontuacao_final >= 60:
    nivel_veracidade = "Parcialmente verificado"
elif pontuacao_final >= 40:
    nivel_veracidade = "Inconclusivo"
elif pontuacao_final >= 25:
    nivel_veracidade = "Provavelmente falso"
else:
    nivel_veracidade = "Falso"

resultados["pontuacao_veracidade"] = pontuacao_final
resultados["nivel_veracidade"] = nivel_veracidade

# 12. Sumarizar os resultados
resumo = {
    "nivel_veracidade": nivel_veracidade,
    "pontuacao": pontuacao_final,
    "analise_detalhada": resultados,
    "pontos_chave": {
        "fontes": resultados.get("qualidade_fonte", "Não analisado"),
        "consistencia": resultados.get("consistencia_narrativa", "Não analisado"),
        "evidencias": resultados.get("qualidade_evidencia", "Não analisado"),
        "contexto": resultados.get("analise_contexto", "Não analisado")
    },
    "recomendacao": ""
}

# Adicionar uma recomendação baseada no nível de veracidade
def adicionar_recomendacao(pontuacao_final):
    """Adiciona uma recomendação ao resumo com base no nível de veracidade."""
    resumo = {}
    if pontuacao_final >= 75:
        resumo["recomendacao"] = "Informação confiável para uso e disseminação."
    elif pontuacao_final >= 50:
        resumo["recomendacao"] = "Verificar informações adicionais antes de aceitar completamente."
    else:
        resumo["recomendacao"] = "Não recomendado para uso ou compartilhamento sem verificação adicional."

    return resumo





Overwriting verificacao_simples.py


In [27]:
%%writefile verificacao_simples.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse, urlencode
import hashlib
from collections import Counter
import time
import logging
import base64
import io
import nltk
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
from PIL import Image, UnidentifiedImageError
from datetime import datetime

# Configurar o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("VerificadorNoticias")

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticias:
    def __init__(self):
        logger.info("Verificador de Notícias inicializado!")

        # Carregar variáveis do ambiente
        load_dotenv()
        self.fact_check_api_key = os.getenv("FACT_CHECK_API_KEY")
        self.fact_check_api_url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]
        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5, "viral": 1.5,
            "bizarro": 1.5, "aterrorizante": 2, "assustador": 1.5, "terrível": 1.5,
            "estarrecedor": 2, "impossível": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors",
            "parece que", "há relatos", "não confirmado", "segundo fontes",
            "indícios", "sinalizam", "aparentemente", "estaria", "poderia",
            "teria", "estima-se"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe",
            "na visão de", "do meu ponto de vista", "considero", "entendo que",
            "me parece", "creio que", "penso que", "sinto que"
        ]

        # Tentar baixar recursos NLTK necessários para análise de texto
        try:
            nltk.download('punkt', quiet=True)
        except Exception as e:
            logger.warning(f"Não foi possível baixar recursos NLTK: {e}")

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        return parsed_url.netloc

    def verificar_url_simples(self, url):
        """Executa uma verificação simples em uma URL de notícia."""
        try:
            # Extrair domínio
            parsed_url = urlparse(url)
            dominio = parsed_url.netloc

            # Tentar acessar a URL
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10, verify=False)
            response.raise_for_status()  # Levantar exceção para status HTTP de erro

            # Verificar status HTTP
            if response.status_code == 200:
                logger.info(f"URL acessada com sucesso: {url}")
                logger.info(f"Domínio: {dominio}")
                logger.info(f"Tamanho do conteúdo: {len(response.text)} caracteres.")
                return {
                    "status": "sucesso",
                    "dominio": dominio,
                    "tamanho_conteudo": len(response.text),
                    "mensagem": "URL acessada com sucesso"
                }
            else:
                logger.warning(f"Erro ao acessar a URL. Código HTTP: {response.status_code}")
                return {
                    "status": "erro",
                    "mensagem": f"Erro ao acessar a URL. Código HTTP: {response.status_code}"
                }

        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao acessar a URL: {e}")
            return {
                "status": "erro",
                "mensagem": str(e)
            }

    def analisar_sentimento(self, texto):
        """Analisa o sentimento do texto usando TextBlob."""
        try:
            sentiment = TextBlob(texto).sentiment
            return {
                "polaridade": sentiment.polarity,
                "subjetividade": sentiment.subjectivity
            }
        except Exception as e:
            logger.error(f"Erro na análise de sentimento: {e}")
            return {
                "polaridade": 0,
                "subjetividade": 0
            }

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {"dominio": dominio}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
            return "Fonte confiável", 20, resultado
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
            return "Fonte suspeita", -30, resultado
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0
            return "Fonte não categorizada", 0, resultado

    def baixar_artigo(self, url):
        """Baixa o conteúdo do artigo a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""
            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)
                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date'] or meta.get('name') in ['publication_date', 'date', 'pubdate']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]', '[name=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(author_tag)
                else:
                    elements = soup.find_all(author_tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords,
                'html': response.text  # Manter o HTML para possível análise adicional
            }
        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            logger.error(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None

    def consultar_fact_check(self, query):
        """Consulta a API do Google Fact Check Tools."""
        if not self.fact_check_api_key:
            logger.warning("Chave de API para fact check não configurada")
            return None

        url_api = f"{self.fact_check_api_url}?key={self.fact_check_api_key}&query={query}"
        try:
            response = requests.get(url_api)
            if response.status_code == 200:
                return response.json()
            else:
                logger.warning(f"Falha ao acessar API de Fact Check. Status: {response.status_code}")
                return None
        except Exception as e:
            logger.error(f"Erro na API de Fact Check: {e}")
            return None

    def analisar_linguagem(self, texto):
        """Analisa o conteúdo em busca de sensacionalismo, incerteza e opinatividade."""
        if not texto or len(texto) < 10:
            return {
                "sensacionalismo": 0,
                "incerteza": 0,
                "opinativo": 0,
                "exclamacoes": 0
            }

        resultado = {}

        # Análise de sensacionalismo
        indice_sensacionalismo = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower())) * peso
            for palavra, peso in self.palavras_sensacionalistas.items()
        )
        resultado["sensacionalismo"] = indice_sensacionalismo

        # Normalizar por tamanho do texto (por 1000 palavras)
        palavras_totais = len(texto.split())
        if palavras_totais > 0:
            indice_sensacionalismo_norm = (indice_sensacionalismo * 1000) / palavras_totais
            resultado["sensacionalismo_normalizado"] = indice_sensacionalismo_norm

            # Adicionar alerta se o índice for alto
            if indice_sensacionalismo_norm > 10:
                resultado["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista"
            elif indice_sensacionalismo_norm > 5:
                resultado["alerta_sensacionalismo"] = "Moderado índice de linguagem sensacionalista"

        # Análise de incerteza
        indice_incerteza = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower()))
            for palavra in self.palavras_incerteza
        )
        resultado["incerteza"] = indice_incerteza

        if palavras_totais > 0:
            indice_incerteza_norm = (indice_incerteza * 1000) / palavras_totais
            resultado["incerteza_normalizada"] = indice_incerteza_norm

            if indice_incerteza_norm > 8:
                resultado["alerta_incerteza"] = "Alto índice de expressões de incerteza"

        # Análise de opinatividade
        indice_opiniao = sum(
            len(re.findall(r'\b' + re.escape(expressao) + r'\b', texto.lower()))
            for expressao in self.expressoes_opinativas
        )
        resultado["opinativo"] = indice_opiniao

        if palavras_totais > 0:
            indice_opiniao_norm = (indice_opiniao * 1000) / palavras_totais
            resultado["opinativo_normalizado"] = indice_opiniao_norm

            if indice_opiniao_norm > 5:
                resultado["alerta_opiniao"] = "Alto índice de expressões opinativas"

        # Contagem de pontos de exclamação (indicador de sensacionalismo)
        exclamacoes = texto.count('!')
        resultado["exclamacoes"] = exclamacoes

        if palavras_totais > 0:
            exclamacoes_norm = (exclamacoes * 1000) / palavras_totais
            if exclamacoes_norm > 10:
                resultado["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        # Análise de sentimento usando TextBlob
        try:
            analise = TextBlob(texto)
            resultado["sentimento_polaridade"] = analise.sentiment.polarity  # -1 a 1 (negativo a positivo)
            resultado["sentimento_subjetividade"] = analise.sentiment.subjectivity  # 0 a 1 (objetivo a subjetivo)

            # Categorização baseada na polaridade
            if resultado["sentimento_polaridade"] < -0.5:
                resultado["sentimento_categoria"] = "muito negativo"
            elif resultado["sentimento_polaridade"] < -0.1:
                resultado["sentimento_categoria"] = "negativo"
            elif resultado["sentimento_polaridade"] <= 0.1:
                resultado["sentimento_categoria"] = "neutro"
            elif resultado["sentimento_polaridade"] <= 0.5:
                resultado["sentimento_categoria"] = "positivo"
            else:
                resultado["sentimento_categoria"] = "muito positivo"

            # Alerta para alto nível de subjetividade em notícias que deveriam ser objetivas
            if resultado["sentimento_subjetividade"] > 0.6:
                resultado["alerta_subjetividade"] = "Conteúdo altamente subjetivo"

        except Exception as e:
            logger.warning(f"Erro na análise de sentimento: {e}")

        return resultado

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}
        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        # Analisar metadados das primeiras 5 imagens (limitar para evitar muito processamento)
        try:
            with ThreadPoolExecutor(max_workers=3) as executor:
                metadados = list(executor.map(self.extrair_metadados_imagem, [img['url'] for img in imagens[:5]]))
                resultados["metadados_imagens"] = [m for m in metadados if m]  # Filtrar None
        except Exception as e:
            logger.error(f"Erro ao analisar metadados das imagens: {e}")

        return resultados

    def extrair_metadados_imagem(self, url_imagem):
        """Extrai metadados de uma imagem a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url_imagem, headers=headers, stream=True, timeout=5, verify=False)
            response.raise_for_status()

            # Abrir a imagem com PIL
            img = Image.open(io.BytesIO(response.content))

            # Coletar metadados básicos
            metadados = {
                "formato": img.format,
                "tamanho": img.size,
                "modo": img.mode,
                "url": url_imagem,
                "hash": hashlib.md5(response.content).hexdigest()
            }

            # Tentar extrair EXIF se disponível
            if hasattr(img, '_getexif') and img._getexif():
                exif = img._getexif()
                if exif:
                    exif_data = {}
                    for tag_id, value in exif.items():
                        # Converter para string para garantir serialização JSON
                        exif_data[str(tag_id)] = str(value)
                    metadados["exif"] = exif_data

            return metadados
        except (requests.RequestException, UnidentifiedImageError, IOError) as e:
            logger.warning(f"Erro ao extrair metadados da imagem {url_imagem}: {e}")
            return None

    def buscar_fact_check(self, texto, titulo=None):
        """Busca por verificações de fatos relacionadas ao conteúdo."""
        resultados = {}

        try:
            # Preparar termos de busca
            query = titulo if titulo else texto[:100]

            # Consultar API de fact check
            data = self.consultar_fact_check(query)

            if data:
                resultados["fact_checks"] = data.get('claims', [])
                resultados["num_fact_checks"] = len(resultados["fact_checks"])

                # Analisar resultados
                if resultados["num_fact_checks"] > 0:
                    # Verificar ratings médios
                    ratings = []
                    for claim in resultados["fact_checks"]:
                        for review in claim.get('claimReview', []):
                            if 'textualRating' in review:
                                ratings.append(review['textualRating'].lower())

                    # Contar frequência de cada rating
                    if ratings:
                        counter = Counter(ratings)
                        resultados["ratings_frequencia"] = counter

                        # Detectar termos comuns de falsidade
                        termos_falso = ['false', 'fake', 'falso', 'mentira', 'enganoso', 'misleading']
                        count_falso = sum(counter.get(termo, 0) for termo in termos_falso)

                        if count_falso > 0:
                            resultados["alerta_fact_check"] = "Conteúdo marcado como falso ou enganoso por verificadores de fatos"
                            resultados["pontos_fact_check"] = -25
                        else:
                            resultados["pontos_fact_check"] = 0
            else:
                resultados["fact_checks"] = []
                resultados["num_fact_checks"] = 0
                resultados["pontos_fact_check"] = 0

        except Exception as e:
            logger.error(f"Erro ao buscar fact checks: {e}")
            resultados["erro_fact_check"] = str(e)
            resultados["pontos_fact_check"] = 0

        return resultados

    def buscar_noticias_similares(self, titulo, keywords, conteudo=None):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}
        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta

            # Simular consistência entre fontes
            resultados["consistencia_entre_fontes"] = "alta"  # baixa, média, alta
            resultados["pontos_consistencia"] = 10  # Simulando pontos por alta consistência

        except Exception as e:
            logger.error(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)
            resultados["pontos_consistencia"] = 0

        return resultados

    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto

    def analisar_credibilidade(self, url):
        """Análise completa de credibilidade de uma URL de notícia."""
        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo or not artigo.get('conteudo'):
            logger.error(f"Não foi possível baixar o artigo: {url}")
            return {"erro": "Não foi possível baixar o artigo para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 2. Verificação de fonte/domínio
        status_fonte, pontos_fonte, info_fonte = self.verificar_fonte(url)
        resultados.update(info_fonte)

        # 3. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 4. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 5. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 6. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)
        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 7. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 8. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 9. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', []),
            conteudo
        )
        resultados.update(resultados_similares)

        # 10. Verificação em serviços de fact-checking
        resultados_fact_check = self.buscar_fact_check(conteudo, artigo['titulo'])
        resultados.update(resultados_fact_check)

        # Cálculo mais sofisticado usando vários fatores
# 11. Determinar um nível de veracidade
# Cálculo mais sofisticado usando vários fatores
base_pontuacao = 70  # Base inicial neutra

# Fatores que afetam a pontuação
pontuacao_fatores = [
    resultados.get("pontos_fonte", 0),
    resultados.get("pontos_consistencia", 0),
    resultados.get("pontos_evidencia", 0),
    resultados.get("pontos_conhecimento_previo", 0),
    resultados.get("pontos_contexto", 0),
    resultados.get("pontos_bias", 0)
]

# Aplicar pesos aos fatores
pesos = [1.5, 1.2, 2.0, 1.0, 1.0, 0.8]
pontuacao_ponderada = sum(p * f for p, f in zip(pesos, pontuacao_fatores))

# Calcular pontuação final
pontuacao_final = max(0, min(100, base_pontuacao + pontuacao_ponderada))

# Determinar nível de veracidade baseado na pontuação final
if pontuacao_final >= 90:
    nivel_veracidade = "Altamente verificado"
elif pontuacao_final >= 75:
    nivel_veracidade = "Provavelmente verdadeiro"
elif pontuacao_final >= 60:
    nivel_veracidade = "Parcialmente verificado"
elif pontuacao_final >= 40:
    nivel_veracidade = "Inconclusivo"
elif pontuacao_final >= 25:
    nivel_veracidade = "Provavelmente falso"
else:
    nivel_veracidade = "Falso"

resultados["pontuacao_veracidade"] = pontuacao_final
resultados["nivel_veracidade"] = nivel_veracidade

# 12. Sumarizar os resultados
resumo = {
    "nivel_veracidade": nivel_veracidade,
    "pontuacao": pontuacao_final,
    "analise_detalhada": resultados,
    "pontos_chave": {
        "fontes": resultados.get("qualidade_fonte", "Não analisado"),
        "consistencia": resultados.get("consistencia_narrativa", "Não analisado"),
        "evidencias": resultados.get("qualidade_evidencia", "Não analisado"),
        "contexto": resultados.get("analise_contexto", "Não analisado")
    },
    "recomendacao": ""
}

# Adicionar uma recomendação baseada no nível de veracidade
def adicionar_recomendacao(pontuacao_final):
    """Adiciona uma recomendação ao resumo com base no nível de veracidade."""
    resumo = {}
    if pontuacao_final >= 75:
        resumo["recomendacao"] = "Informação confiável para uso e disseminação."
    elif pontuacao_final >= 50:
        resumo["recomendacao"] = "Verificar informações adicionais antes de aceitar completamente."
    else:
        resumo["recomendacao"] = "Não recomendado para uso ou compartilhamento sem verificação adicional."

    return resumo

Overwriting verificacao_simples.py


In [26]:
%run verificacao_simples.py

SyntaxError: 'return' outside function (verificacao_simples.py, line 644)

In [25]:
%%writefile verificacao_simples.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse, urlencode
import hashlib
from collections import Counter
import time
import logging
import base64
import io
import nltk
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
from PIL import Image, UnidentifiedImageError
from datetime import datetime

# Configurar o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("VerificadorNoticias")

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticias:
    def __init__(self):
        logger.info("Verificador de Notícias inicializado!")

        # Carregar variáveis do ambiente
        load_dotenv()
        self.fact_check_api_key = os.getenv("FACT_CHECK_API_KEY")
        self.fact_check_api_url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]
        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5, "viral": 1.5,
            "bizarro": 1.5, "aterrorizante": 2, "assustador": 1.5, "terrível": 1.5,
            "estarrecedor": 2, "impossível": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors",
            "parece que", "há relatos", "não confirmado", "segundo fontes",
            "indícios", "sinalizam", "aparentemente", "estaria", "poderia",
            "teria", "estima-se"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe",
            "na visão de", "do meu ponto de vista", "considero", "entendo que",
            "me parece", "creio que", "penso que", "sinto que"
        ]

        # Tentar baixar recursos NLTK necessários para análise de texto
        try:
            nltk.download('punkt', quiet=True)
        except Exception as e:
            logger.warning(f"Não foi possível baixar recursos NLTK: {e}")

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        return parsed_url.netloc

    def verificar_url_simples(self, url):
        """Executa uma verificação simples em uma URL de notícia."""
        try:
            # Extrair domínio
            parsed_url = urlparse(url)
            dominio = parsed_url.netloc

            # Tentar acessar a URL
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10, verify=False)
            response.raise_for_status()  # Levantar exceção para status HTTP de erro

            # Verificar status HTTP
            if response.status_code == 200:
                logger.info(f"URL acessada com sucesso: {url}")
                logger.info(f"Domínio: {dominio}")
                logger.info(f"Tamanho do conteúdo: {len(response.text)} caracteres.")
                return {
                    "status": "sucesso",
                    "dominio": dominio,
                    "tamanho_conteudo": len(response.text),
                    "mensagem": "URL acessada com sucesso"
                }
            else:
                logger.warning(f"Erro ao acessar a URL. Código HTTP: {response.status_code}")
                return {
                    "status": "erro",
                    "mensagem": f"Erro ao acessar a URL. Código HTTP: {response.status_code}"
                }

        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao acessar a URL: {e}")
            return {
                "status": "erro",
                "mensagem": str(e)
            }

    def analisar_sentimento(self, texto):
        """Analisa o sentimento do texto usando TextBlob."""
        try:
            sentiment = TextBlob(texto).sentiment
            return {
                "polaridade": sentiment.polarity,
                "subjetividade": sentiment.subjectivity
            }
        except Exception as e:
            logger.error(f"Erro na análise de sentimento: {e}")
            return {
                "polaridade": 0,
                "subjetividade": 0
            }

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {"dominio": dominio}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
            return "Fonte confiável", 20, resultado
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
            return "Fonte suspeita", -30, resultado
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0
            return "Fonte não categorizada", 0, resultado

    def baixar_artigo(self, url):
        """Baixa o conteúdo do artigo a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""
            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)
                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date'] or meta.get('name') in ['publication_date', 'date', 'pubdate']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]', '[name=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(author_tag)
                else:
                    elements = soup.find_all(author_tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords,
                'html': response.text  # Manter o HTML para possível análise adicional
            }
        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            logger.error(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None

    def consultar_fact_check(self, query):
        """Consulta a API do Google Fact Check Tools."""
        if not self.fact_check_api_key:
            logger.warning("Chave de API para fact check não configurada")
            return None

        url_api = f"{self.fact_check_api_url}?key={self.fact_check_api_key}&query={query}"
        try:
            response = requests.get(url_api)
            if response.status_code == 200:
                return response.json()
            else:
                logger.warning(f"Falha ao acessar API de Fact Check. Status: {response.status_code}")
                return None
        except Exception as e:
            logger.error(f"Erro na API de Fact Check: {e}")
            return None

    def analisar_linguagem(self, texto):
        """Analisa o conteúdo em busca de sensacionalismo, incerteza e opinatividade."""
        if not texto or len(texto) < 10:
            return {
                "sensacionalismo": 0,
                "incerteza": 0,
                "opinativo": 0,
                "exclamacoes": 0
            }

        resultado = {}

        # Análise de sensacionalismo
        indice_sensacionalismo = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower())) * peso
            for palavra, peso in self.palavras_sensacionalistas.items()
        )
        resultado["sensacionalismo"] = indice_sensacionalismo

        # Normalizar por tamanho do texto (por 1000 palavras)
        palavras_totais = len(texto.split())
        if palavras_totais > 0:
            indice_sensacionalismo_norm = (indice_sensacionalismo * 1000) / palavras_totais
            resultado["sensacionalismo_normalizado"] = indice_sensacionalismo_norm

            # Adicionar alerta se o índice for alto
            if indice_sensacionalismo_norm > 10:
                resultado["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista"
            elif indice_sensacionalismo_norm > 5:
                resultado["alerta_sensacionalismo"] = "Moderado índice de linguagem sensacionalista"

        # Análise de incerteza
        indice_incerteza = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower()))
            for palavra in self.palavras_incerteza
        )
        resultado["incerteza"] = indice_incerteza

        if palavras_totais > 0:
            indice_incerteza_norm = (indice_incerteza * 1000) / palavras_totais
            resultado["incerteza_normalizada"] = indice_incerteza_norm

            if indice_incerteza_norm > 8:
                resultado["alerta_incerteza"] = "Alto índice de expressões de incerteza"

        # Análise de opinatividade
        indice_opiniao = sum(
            len(re.findall(r'\b' + re.escape(expressao) + r'\b', texto.lower()))
            for expressao in self.expressoes_opinativas
        )
        resultado["opinativo"] = indice_opiniao

        if palavras_totais > 0:
            indice_opiniao_norm = (indice_opiniao * 1000) / palavras_totais
            resultado["opinativo_normalizado"] = indice_opiniao_norm

            if indice_opiniao_norm > 5:
                resultado["alerta_opiniao"] = "Alto índice de expressões opinativas"

        # Contagem de pontos de exclamação (indicador de sensacionalismo)
        exclamacoes = texto.count('!')
        resultado["exclamacoes"] = exclamacoes

        if palavras_totais > 0:
            exclamacoes_norm = (exclamacoes * 1000) / palavras_totais
            if exclamacoes_norm > 10:
                resultado["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        # Análise de sentimento usando TextBlob
        try:
            analise = TextBlob(texto)
            resultado["sentimento_polaridade"] = analise.sentiment.polarity  # -1 a 1 (negativo a positivo)
            resultado["sentimento_subjetividade"] = analise.sentiment.subjectivity  # 0 a 1 (objetivo a subjetivo)

            # Categorização baseada na polaridade
            if resultado["sentimento_polaridade"] < -0.5:
                resultado["sentimento_categoria"] = "muito negativo"
            elif resultado["sentimento_polaridade"] < -0.1:
                resultado["sentimento_categoria"] = "negativo"
            elif resultado["sentimento_polaridade"] <= 0.1:
                resultado["sentimento_categoria"] = "neutro"
            elif resultado["sentimento_polaridade"] <= 0.5:
                resultado["sentimento_categoria"] = "positivo"
            else:
                resultado["sentimento_categoria"] = "muito positivo"

            # Alerta para alto nível de subjetividade em notícias que deveriam ser objetivas
            if resultado["sentimento_subjetividade"] > 0.6:
                resultado["alerta_subjetividade"] = "Conteúdo altamente subjetivo"

        except Exception as e:
            logger.warning(f"Erro na análise de sentimento: {e}")

        return resultado

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}
        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        # Analisar metadados das primeiras 5 imagens (limitar para evitar muito processamento)
        try:
            with ThreadPoolExecutor(max_workers=3) as executor:
                metadados = list(executor.map(self.extrair_metadados_imagem, [img['url'] for img in imagens[:5]]))
                resultados["metadados_imagens"] = [m for m in metadados if m]  # Filtrar None
        except Exception as e:
            logger.error(f"Erro ao analisar metadados das imagens: {e}")

        return resultados

    def extrair_metadados_imagem(self, url_imagem):
        """Extrai metadados de uma imagem a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url_imagem, headers=headers, stream=True, timeout=5, verify=False)
            response.raise_for_status()

            # Abrir a imagem com PIL
            img = Image.open(io.BytesIO(response.content))

            # Coletar metadados básicos
            metadados = {
                "formato": img.format,
                "tamanho": img.size,
                "modo": img.mode,
                "url": url_imagem,
                "hash": hashlib.md5(response.content).hexdigest()
            }

            # Tentar extrair EXIF se disponível
            if hasattr(img, '_getexif') and img._getexif():
                exif = img._getexif()
                if exif:
                    exif_data = {}
                    for tag_id, value in exif.items():
                        # Converter para string para garantir serialização JSON
                        exif_data[str(tag_id)] = str(value)
                    metadados["exif"] = exif_data

            return metadados
        except (requests.RequestException, UnidentifiedImageError, IOError) as e:
            logger.warning(f"Erro ao extrair metadados da imagem {url_imagem}: {e}")
            return None

    def buscar_fact_check(self, texto, titulo=None):
        """Busca por verificações de fatos relacionadas ao conteúdo."""
        resultados = {}

        try:
            # Preparar termos de busca
            query = titulo if titulo else texto[:100]

            # Consultar API de fact check
            data = self.consultar_fact_check(query)

            if data:
                resultados["fact_checks"] = data.get('claims', [])
                resultados["num_fact_checks"] = len(resultados["fact_checks"])

                # Analisar resultados
                if resultados["num_fact_checks"] > 0:
                    # Verificar ratings médios
                    ratings = []
                    for claim in resultados["fact_checks"]:
                        for review in claim.get('claimReview', []):
                            if 'textualRating' in review:
                                ratings.append(review['textualRating'].lower())

                    # Contar frequência de cada rating
                    if ratings:
                        counter = Counter(ratings)
                        resultados["ratings_frequencia"] = counter

                        # Detectar termos comuns de falsidade
                        termos_falso = ['false', 'fake', 'falso', 'mentira', 'enganoso', 'misleading']
                        count_falso = sum(counter.get(termo, 0) for termo in termos_falso)

                        if count_falso > 0:
                            resultados["alerta_fact_check"] = "Conteúdo marcado como falso ou enganoso por verificadores de fatos"
                            resultados["pontos_fact_check"] = -25
                        else:
                            resultados["pontos_fact_check"] = 0
            else:
                resultados["fact_checks"] = []
                resultados["num_fact_checks"] = 0
                resultados["pontos_fact_check"] = 0

        except Exception as e:
            logger.error(f"Erro ao buscar fact checks: {e}")
            resultados["erro_fact_check"] = str(e)
            resultados["pontos_fact_check"] = 0

        return resultados

    def buscar_noticias_similares(self, titulo, keywords, conteudo=None):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}
        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta

            # Simular consistência entre fontes
            resultados["consistencia_entre_fontes"] = "alta"  # baixa, média, alta
            resultados["pontos_consistencia"] = 10  # Simulando pontos por alta consistência

        except Exception as e:
            logger.error(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)
            resultados["pontos_consistencia"] = 0

        return resultados

    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto

    def analisar_credibilidade(self, url):
        """Análise completa de credibilidade de uma URL de notícia."""
        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo or not artigo.get('conteudo'):
            logger.error(f"Não foi possível baixar o artigo: {url}")
            return {"erro": "Não foi possível baixar o artigo para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 2. Verificação de fonte/domínio
        status_fonte, pontos_fonte, info_fonte = self.verificar_fonte(url)
        resultados.update(info_fonte)

        # 3. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 4. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 5. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 6. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)
        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 7. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 8. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 9. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', []),
            conteudo
        )
        resultados.update(resultados_similares)

        # 10. Verificação em serviços de fact-checking
        resultados_fact_check = self.buscar_fact_check(conteudo, artigo['titulo'])
        resultados.update(resultados_fact_check)

        # Cálculo mais sofisticado usando vários fatores
# 11. Determinar um nível de veracidade
# Cálculo mais sofisticado usando vários fatores
base_pontuacao = 70  # Base inicial neutra

# Fatores que afetam a pontuação
pontuacao_fatores = [
    resultados.get("pontos_fonte", 0),
    resultados.get("pontos_consistencia", 0),
    resultados.get("pontos_evidencia", 0),
    resultados.get("pontos_conhecimento_previo", 0),
    resultados.get("pontos_contexto", 0),
    resultados.get("pontos_bias", 0)
]

# Aplicar pesos aos fatores
pesos = [1.5, 1.2, 2.0, 1.0, 1.0, 0.8]
pontuacao_ponderada = sum(p * f for p, f in zip(pesos, pontuacao_fatores))

# Calcular pontuação final
pontuacao_final = max(0, min(100, base_pontuacao + pontuacao_ponderada))

# Determinar nível de veracidade baseado na pontuação final
if pontuacao_final >= 90:
    nivel_veracidade = "Altamente verificado"
elif pontuacao_final >= 75:
    nivel_veracidade = "Provavelmente verdadeiro"
elif pontuacao_final >= 60:
    nivel_veracidade = "Parcialmente verificado"
elif pontuacao_final >= 40:
    nivel_veracidade = "Inconclusivo"
elif pontuacao_final >= 25:
    nivel_veracidade = "Provavelmente falso"
else:
    nivel_veracidade = "Falso"

resultados["pontuacao_veracidade"] = pontuacao_final
resultados["nivel_veracidade"] = nivel_veracidade

# 12. Sumarizar os resultados
resumo = {
    "nivel_veracidade": nivel_veracidade,
    "pontuacao": pontuacao_final,
    "analise_detalhada": resultados,
    "pontos_chave": {
        "fontes": resultados.get("qualidade_fonte", "Não analisado"),
        "consistencia": resultados.get("consistencia_narrativa", "Não analisado"),
        "evidencias": resultados.get("qualidade_evidencia", "Não analisado"),
        "contexto": resultados.get("analise_contexto", "Não analisado")
    },
    "recomendacao": ""
}

# Adicionar uma recomendação baseada no nível de veracidade
if pontuacao_final >= 75:
    resumo["recomendacao"] = "Informação confiável para uso e disseminação."
elif pontuacao_final >= 50:
    resumo["recomendacao"] = "Verificar informações adicionais antes de aceitar completamente."
else:
    resumo["recomendacao"] = "Não recomendado para uso ou compartilhamento sem verificação adicional."
    return resumo

Overwriting verificacao_simples.py


In [23]:
%%writefile verificacao_simples.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse, urlencode
import hashlib
from collections import Counter
import time
import logging
import base64
import io
import nltk
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
from PIL import Image, UnidentifiedImageError
from datetime import datetime

# Configurar o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("VerificadorNoticias")

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticias:
    def __init__(self):
        logger.info("Verificador de Notícias inicializado!")

        # Carregar variáveis do ambiente
        load_dotenv()
        self.fact_check_api_key = os.getenv("FACT_CHECK_API_KEY")
        self.fact_check_api_url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]
        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5, "viral": 1.5,
            "bizarro": 1.5, "aterrorizante": 2, "assustador": 1.5, "terrível": 1.5,
            "estarrecedor": 2, "impossível": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors",
            "parece que", "há relatos", "não confirmado", "segundo fontes",
            "indícios", "sinalizam", "aparentemente", "estaria", "poderia",
            "teria", "estima-se"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe",
            "na visão de", "do meu ponto de vista", "considero", "entendo que",
            "me parece", "creio que", "penso que", "sinto que"
        ]

        # Tentar baixar recursos NLTK necessários para análise de texto
        try:
            nltk.download('punkt', quiet=True)
        except Exception as e:
            logger.warning(f"Não foi possível baixar recursos NLTK: {e}")

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        return parsed_url.netloc

    def verificar_url_simples(self, url):
        """Executa uma verificação simples em uma URL de notícia."""
        try:
            # Extrair domínio
            parsed_url = urlparse(url)
            dominio = parsed_url.netloc

            # Tentar acessar a URL
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10, verify=False)
            response.raise_for_status()  # Levantar exceção para status HTTP de erro

            # Verificar status HTTP
            if response.status_code == 200:
                logger.info(f"URL acessada com sucesso: {url}")
                logger.info(f"Domínio: {dominio}")
                logger.info(f"Tamanho do conteúdo: {len(response.text)} caracteres.")
                return {
                    "status": "sucesso",
                    "dominio": dominio,
                    "tamanho_conteudo": len(response.text),
                    "mensagem": "URL acessada com sucesso"
                }
            else:
                logger.warning(f"Erro ao acessar a URL. Código HTTP: {response.status_code}")
                return {
                    "status": "erro",
                    "mensagem": f"Erro ao acessar a URL. Código HTTP: {response.status_code}"
                }

        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao acessar a URL: {e}")
            return {
                "status": "erro",
                "mensagem": str(e)
            }

    def analisar_sentimento(self, texto):
        """Analisa o sentimento do texto usando TextBlob."""
        try:
            sentiment = TextBlob(texto).sentiment
            return {
                "polaridade": sentiment.polarity,
                "subjetividade": sentiment.subjectivity
            }
        except Exception as e:
            logger.error(f"Erro na análise de sentimento: {e}")
            return {
                "polaridade": 0,
                "subjetividade": 0
            }

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {"dominio": dominio}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
            return "Fonte confiável", 20, resultado
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
            return "Fonte suspeita", -30, resultado
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0
            return "Fonte não categorizada", 0, resultado

    def baixar_artigo(self, url):
        """Baixa o conteúdo do artigo a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""
            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)
                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date'] or meta.get('name') in ['publication_date', 'date', 'pubdate']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]', '[name=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(author_tag)
                else:
                    elements = soup.find_all(author_tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords,
                'html': response.text  # Manter o HTML para possível análise adicional
            }
        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            logger.error(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None

    def consultar_fact_check(self, query):
        """Consulta a API do Google Fact Check Tools."""
        if not self.fact_check_api_key:
            logger.warning("Chave de API para fact check não configurada")
            return None

        url_api = f"{self.fact_check_api_url}?key={self.fact_check_api_key}&query={query}"
        try:
            response = requests.get(url_api)
            if response.status_code == 200:
                return response.json()
            else:
                logger.warning(f"Falha ao acessar API de Fact Check. Status: {response.status_code}")
                return None
        except Exception as e:
            logger.error(f"Erro na API de Fact Check: {e}")
            return None

    def analisar_linguagem(self, texto):
        """Analisa o conteúdo em busca de sensacionalismo, incerteza e opinatividade."""
        if not texto or len(texto) < 10:
            return {
                "sensacionalismo": 0,
                "incerteza": 0,
                "opinativo": 0,
                "exclamacoes": 0
            }

        resultado = {}

        # Análise de sensacionalismo
        indice_sensacionalismo = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower())) * peso
            for palavra, peso in self.palavras_sensacionalistas.items()
        )
        resultado["sensacionalismo"] = indice_sensacionalismo

        # Normalizar por tamanho do texto (por 1000 palavras)
        palavras_totais = len(texto.split())
        if palavras_totais > 0:
            indice_sensacionalismo_norm = (indice_sensacionalismo * 1000) / palavras_totais
            resultado["sensacionalismo_normalizado"] = indice_sensacionalismo_norm

            # Adicionar alerta se o índice for alto
            if indice_sensacionalismo_norm > 10:
                resultado["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista"
            elif indice_sensacionalismo_norm > 5:
                resultado["alerta_sensacionalismo"] = "Moderado índice de linguagem sensacionalista"

        # Análise de incerteza
        indice_incerteza = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower()))
            for palavra in self.palavras_incerteza
        )
        resultado["incerteza"] = indice_incerteza

        if palavras_totais > 0:
            indice_incerteza_norm = (indice_incerteza * 1000) / palavras_totais
            resultado["incerteza_normalizada"] = indice_incerteza_norm

            if indice_incerteza_norm > 8:
                resultado["alerta_incerteza"] = "Alto índice de expressões de incerteza"

        # Análise de opinatividade
        indice_opiniao = sum(
            len(re.findall(r'\b' + re.escape(expressao) + r'\b', texto.lower()))
            for expressao in self.expressoes_opinativas
        )
        resultado["opinativo"] = indice_opiniao

        if palavras_totais > 0:
            indice_opiniao_norm = (indice_opiniao * 1000) / palavras_totais
            resultado["opinativo_normalizado"] = indice_opiniao_norm

            if indice_opiniao_norm > 5:
                resultado["alerta_opiniao"] = "Alto índice de expressões opinativas"

        # Contagem de pontos de exclamação (indicador de sensacionalismo)
        exclamacoes = texto.count('!')
        resultado["exclamacoes"] = exclamacoes

        if palavras_totais > 0:
            exclamacoes_norm = (exclamacoes * 1000) / palavras_totais
            if exclamacoes_norm > 10:
                resultado["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        # Análise de sentimento usando TextBlob
        try:
            analise = TextBlob(texto)
            resultado["sentimento_polaridade"] = analise.sentiment.polarity  # -1 a 1 (negativo a positivo)
            resultado["sentimento_subjetividade"] = analise.sentiment.subjectivity  # 0 a 1 (objetivo a subjetivo)

            # Categorização baseada na polaridade
            if resultado["sentimento_polaridade"] < -0.5:
                resultado["sentimento_categoria"] = "muito negativo"
            elif resultado["sentimento_polaridade"] < -0.1:
                resultado["sentimento_categoria"] = "negativo"
            elif resultado["sentimento_polaridade"] <= 0.1:
                resultado["sentimento_categoria"] = "neutro"
            elif resultado["sentimento_polaridade"] <= 0.5:
                resultado["sentimento_categoria"] = "positivo"
            else:
                resultado["sentimento_categoria"] = "muito positivo"

            # Alerta para alto nível de subjetividade em notícias que deveriam ser objetivas
            if resultado["sentimento_subjetividade"] > 0.6:
                resultado["alerta_subjetividade"] = "Conteúdo altamente subjetivo"

        except Exception as e:
            logger.warning(f"Erro na análise de sentimento: {e}")

        return resultado

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}
        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        # Analisar metadados das primeiras 5 imagens (limitar para evitar muito processamento)
        try:
            with ThreadPoolExecutor(max_workers=3) as executor:
                metadados = list(executor.map(self.extrair_metadados_imagem, [img['url'] for img in imagens[:5]]))
                resultados["metadados_imagens"] = [m for m in metadados if m]  # Filtrar None
        except Exception as e:
            logger.error(f"Erro ao analisar metadados das imagens: {e}")

        return resultados

    def extrair_metadados_imagem(self, url_imagem):
        """Extrai metadados de uma imagem a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url_imagem, headers=headers, stream=True, timeout=5, verify=False)
            response.raise_for_status()

            # Abrir a imagem com PIL
            img = Image.open(io.BytesIO(response.content))

            # Coletar metadados básicos
            metadados = {
                "formato": img.format,
                "tamanho": img.size,
                "modo": img.mode,
                "url": url_imagem,
                "hash": hashlib.md5(response.content).hexdigest()
            }

            # Tentar extrair EXIF se disponível
            if hasattr(img, '_getexif') and img._getexif():
                exif = img._getexif()
                if exif:
                    exif_data = {}
                    for tag_id, value in exif.items():
                        # Converter para string para garantir serialização JSON
                        exif_data[str(tag_id)] = str(value)
                    metadados["exif"] = exif_data

            return metadados
        except (requests.RequestException, UnidentifiedImageError, IOError) as e:
            logger.warning(f"Erro ao extrair metadados da imagem {url_imagem}: {e}")
            return None

    def buscar_fact_check(self, texto, titulo=None):
        """Busca por verificações de fatos relacionadas ao conteúdo."""
        resultados = {}

        try:
            # Preparar termos de busca
            query = titulo if titulo else texto[:100]

            # Consultar API de fact check
            data = self.consultar_fact_check(query)

            if data:
                resultados["fact_checks"] = data.get('claims', [])
                resultados["num_fact_checks"] = len(resultados["fact_checks"])

                # Analisar resultados
                if resultados["num_fact_checks"] > 0:
                    # Verificar ratings médios
                    ratings = []
                    for claim in resultados["fact_checks"]:
                        for review in claim.get('claimReview', []):
                            if 'textualRating' in review:
                                ratings.append(review['textualRating'].lower())

                    # Contar frequência de cada rating
                    if ratings:
                        counter = Counter(ratings)
                        resultados["ratings_frequencia"] = counter

                        # Detectar termos comuns de falsidade
                        termos_falso = ['false', 'fake', 'falso', 'mentira', 'enganoso', 'misleading']
                        count_falso = sum(counter.get(termo, 0) for termo in termos_falso)

                        if count_falso > 0:
                            resultados["alerta_fact_check"] = "Conteúdo marcado como falso ou enganoso por verificadores de fatos"
                            resultados["pontos_fact_check"] = -25
                        else:
                            resultados["pontos_fact_check"] = 0
            else:
                resultados["fact_checks"] = []
                resultados["num_fact_checks"] = 0
                resultados["pontos_fact_check"] = 0

        except Exception as e:
            logger.error(f"Erro ao buscar fact checks: {e}")
            resultados["erro_fact_check"] = str(e)
            resultados["pontos_fact_check"] = 0

        return resultados

    def buscar_noticias_similares(self, titulo, keywords, conteudo=None):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}
        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta

            # Simular consistência entre fontes
            resultados["consistencia_entre_fontes"] = "alta"  # baixa, média, alta
            resultados["pontos_consistencia"] = 10  # Simulando pontos por alta consistência

        except Exception as e:
            logger.error(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)
            resultados["pontos_consistencia"] = 0

        return resultados

    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto

    def analisar_credibilidade(self, url):
        """Análise completa de credibilidade de uma URL de notícia."""
        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo or not artigo.get('conteudo'):
            logger.error(f"Não foi possível baixar o artigo: {url}")
            return {"erro": "Não foi possível baixar o artigo para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 2. Verificação de fonte/domínio
        status_fonte, pontos_fonte, info_fonte = self.verificar_fonte(url)
        resultados.update(info_fonte)

        # 3. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 4. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 5. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 6. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)
        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 7. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 8. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 9. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', []),
            conteudo
        )
        resultados.update(resultados_similares)

        # 10. Verificação em serviços de fact-checking
        resultados_fact_check = self.buscar_fact_check(conteudo, artigo['titulo'])
        resultados.update(resultados_fact_check)

        # Cálculo mais sofisticado usando vários fatores
# 11. Determinar um nível de veracidade
# Cálculo mais sofisticado usando vários fatores
base_pontuacao = 70  # Base inicial neutra

# Fatores que afetam a pontuação
pontuacao_fatores = [
    resultados.get("pontos_fonte", 0),
    resultados.get("pontos_consistencia", 0),
    resultados.get("pontos_evidencia", 0),
    resultados.get("pontos_conhecimento_previo", 0),
    resultados.get("pontos_contexto", 0),
    resultados.get("pontos_bias", 0)
]

# Aplicar pesos aos fatores
pesos = [1.5, 1.2, 2.0, 1.0, 1.0, 0.8]
pontuacao_ponderada = sum(p * f for p, f in zip(pesos, pontuacao_fatores))

# Calcular pontuação final
pontuacao_final = max(0, min(100, base_pontuacao + pontuacao_ponderada))

# Determinar nível de veracidade baseado na pontuação final
if pontuacao_final >= 90:
    nivel_veracidade = "Altamente verificado"
elif pontuacao_final >= 75:
    nivel_veracidade = "Provavelmente verdadeiro"
elif pontuacao_final >= 60:
    nivel_veracidade = "Parcialmente verificado"
elif pontuacao_final >= 40:
    nivel_veracidade = "Inconclusivo"
elif pontuacao_final >= 25:
    nivel_veracidade = "Provavelmente falso"
else:
    nivel_veracidade = "Falso"

resultados["pontuacao_veracidade"] = pontuacao_final
resultados["nivel_veracidade"] = nivel_veracidade

# 12. Sumarizar os resultados
resumo = {
    "nivel_veracidade": nivel_veracidade,
    "pontuacao": pontuacao_final,
    "analise_detalhada": resultados,
    "pontos_chave": {
        "fontes": resultados.get("qualidade_fonte", "Não analisado"),
        "consistencia": resultados.get("consistencia_narrativa", "Não analisado"),
        "evidencias": resultados.get("qualidade_evidencia", "Não analisado"),
        "contexto": resultados.get("analise_contexto", "Não analisado")
    },
    "recomendacao": ""
}

# Adicionar uma recomendação baseada no nível de veracidade
if pontuacao_final >= 75:
    resumo["recomendacao"] = "Informação confiável para uso e disseminação."
elif pontuacao_final >= 50:
    resumo["recomendacao"] = "Verificar informações adicionais antes de aceitar completamente."
else:
    resumo["recomendacao"] = "Não recomendado para uso ou compartilhamento sem verificação adicional."

return resumo

Overwriting verificacao_simples.py


In [21]:
%%writefile verificacao_simples.py

import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse, urlencode
import hashlib
from collections import Counter
import time
import logging
import base64
import io
import nltk
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
from PIL import Image, UnidentifiedImageError
from datetime import datetime

# Configurar o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("VerificadorNoticias")

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticias:
    def __init__(self):
        logger.info("Verificador de Notícias inicializado!")

        # Carregar variáveis do ambiente
        load_dotenv()
        self.fact_check_api_key = os.getenv("FACT_CHECK_API_KEY")
        self.fact_check_api_url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]
        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5, "viral": 1.5,
            "bizarro": 1.5, "aterrorizante": 2, "assustador": 1.5, "terrível": 1.5,
            "estarrecedor": 2, "impossível": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors",
            "parece que", "há relatos", "não confirmado", "segundo fontes",
            "indícios", "sinalizam", "aparentemente", "estaria", "poderia",
            "teria", "estima-se"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe",
            "na visão de", "do meu ponto de vista", "considero", "entendo que",
            "me parece", "creio que", "penso que", "sinto que"
        ]

        # Tentar baixar recursos NLTK necessários para análise de texto
        try:
            nltk.download('punkt', quiet=True)
        except Exception as e:
            logger.warning(f"Não foi possível baixar recursos NLTK: {e}")

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        return parsed_url.netloc

    def verificar_url_simples(self, url):
        """Executa uma verificação simples em uma URL de notícia."""
        try:
            # Extrair domínio
            parsed_url = urlparse(url)
            dominio = parsed_url.netloc

            # Tentar acessar a URL
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10, verify=False)
            response.raise_for_status()  # Levantar exceção para status HTTP de erro

            # Verificar status HTTP
            if response.status_code == 200:
                logger.info(f"URL acessada com sucesso: {url}")
                logger.info(f"Domínio: {dominio}")
                logger.info(f"Tamanho do conteúdo: {len(response.text)} caracteres.")
                return {
                    "status": "sucesso",
                    "dominio": dominio,
                    "tamanho_conteudo": len(response.text),
                    "mensagem": "URL acessada com sucesso"
                }
            else:
                logger.warning(f"Erro ao acessar a URL. Código HTTP: {response.status_code}")
                return {
                    "status": "erro",
                    "mensagem": f"Erro ao acessar a URL. Código HTTP: {response.status_code}"
                }

        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao acessar a URL: {e}")
            return {
                "status": "erro",
                "mensagem": str(e)
            }

    def analisar_sentimento(self, texto):
        """Analisa o sentimento do texto usando TextBlob."""
        try:
            sentiment = TextBlob(texto).sentiment
            return {
                "polaridade": sentiment.polarity,
                "subjetividade": sentiment.subjectivity
            }
        except Exception as e:
            logger.error(f"Erro na análise de sentimento: {e}")
            return {
                "polaridade": 0,
                "subjetividade": 0
            }

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {"dominio": dominio}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
            return "Fonte confiável", 20, resultado
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
            return "Fonte suspeita", -30, resultado
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0
            return "Fonte não categorizada", 0, resultado

    def baixar_artigo(self, url):
        """Baixa o conteúdo do artigo a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""
            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)
                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date'] or meta.get('name') in ['publication_date', 'date', 'pubdate']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]', '[name=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(author_tag)
                else:
                    elements = soup.find_all(author_tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords,
                'html': response.text  # Manter o HTML para possível análise adicional
            }
        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            logger.error(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None

    def consultar_fact_check(self, query):
        """Consulta a API do Google Fact Check Tools."""
        if not self.fact_check_api_key:
            logger.warning("Chave de API para fact check não configurada")
            return None

        url_api = f"{self.fact_check_api_url}?key={self.fact_check_api_key}&query={query}"
        try:
            response = requests.get(url_api)
            if response.status_code == 200:
                return response.json()
            else:
                logger.warning(f"Falha ao acessar API de Fact Check. Status: {response.status_code}")
                return None
        except Exception as e:
            logger.error(f"Erro na API de Fact Check: {e}")
            return None

    def analisar_linguagem(self, texto):
        """Analisa o conteúdo em busca de sensacionalismo, incerteza e opinatividade."""
        if not texto or len(texto) < 10:
            return {
                "sensacionalismo": 0,
                "incerteza": 0,
                "opinativo": 0,
                "exclamacoes": 0
            }

        resultado = {}

        # Análise de sensacionalismo
        indice_sensacionalismo = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower())) * peso
            for palavra, peso in self.palavras_sensacionalistas.items()
        )
        resultado["sensacionalismo"] = indice_sensacionalismo

        # Normalizar por tamanho do texto (por 1000 palavras)
        palavras_totais = len(texto.split())
        if palavras_totais > 0:
            indice_sensacionalismo_norm = (indice_sensacionalismo * 1000) / palavras_totais
            resultado["sensacionalismo_normalizado"] = indice_sensacionalismo_norm

            # Adicionar alerta se o índice for alto
            if indice_sensacionalismo_norm > 10:
                resultado["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista"
            elif indice_sensacionalismo_norm > 5:
                resultado["alerta_sensacionalismo"] = "Moderado índice de linguagem sensacionalista"

        # Análise de incerteza
        indice_incerteza = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower()))
            for palavra in self.palavras_incerteza
        )
        resultado["incerteza"] = indice_incerteza

        if palavras_totais > 0:
            indice_incerteza_norm = (indice_incerteza * 1000) / palavras_totais
            resultado["incerteza_normalizada"] = indice_incerteza_norm

            if indice_incerteza_norm > 8:
                resultado["alerta_incerteza"] = "Alto índice de expressões de incerteza"

        # Análise de opinatividade
        indice_opiniao = sum(
            len(re.findall(r'\b' + re.escape(expressao) + r'\b', texto.lower()))
            for expressao in self.expressoes_opinativas
        )
        resultado["opinativo"] = indice_opiniao

        if palavras_totais > 0:
            indice_opiniao_norm = (indice_opiniao * 1000) / palavras_totais
            resultado["opinativo_normalizado"] = indice_opiniao_norm

            if indice_opiniao_norm > 5:
                resultado["alerta_opiniao"] = "Alto índice de expressões opinativas"

        # Contagem de pontos de exclamação (indicador de sensacionalismo)
        exclamacoes = texto.count('!')
        resultado["exclamacoes"] = exclamacoes

        if palavras_totais > 0:
            exclamacoes_norm = (exclamacoes * 1000) / palavras_totais
            if exclamacoes_norm > 10:
                resultado["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        # Análise de sentimento usando TextBlob
        try:
            analise = TextBlob(texto)
            resultado["sentimento_polaridade"] = analise.sentiment.polarity  # -1 a 1 (negativo a positivo)
            resultado["sentimento_subjetividade"] = analise.sentiment.subjectivity  # 0 a 1 (objetivo a subjetivo)

            # Categorização baseada na polaridade
            if resultado["sentimento_polaridade"] < -0.5:
                resultado["sentimento_categoria"] = "muito negativo"
            elif resultado["sentimento_polaridade"] < -0.1:
                resultado["sentimento_categoria"] = "negativo"
            elif resultado["sentimento_polaridade"] <= 0.1:
                resultado["sentimento_categoria"] = "neutro"
            elif resultado["sentimento_polaridade"] <= 0.5:
                resultado["sentimento_categoria"] = "positivo"
            else:
                resultado["sentimento_categoria"] = "muito positivo"

            # Alerta para alto nível de subjetividade em notícias que deveriam ser objetivas
            if resultado["sentimento_subjetividade"] > 0.6:
                resultado["alerta_subjetividade"] = "Conteúdo altamente subjetivo"

        except Exception as e:
            logger.warning(f"Erro na análise de sentimento: {e}")

        return resultado

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}
        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        # Analisar metadados das primeiras 5 imagens (limitar para evitar muito processamento)
        try:
            with ThreadPoolExecutor(max_workers=3) as executor:
                metadados = list(executor.map(self.extrair_metadados_imagem, [img['url'] for img in imagens[:5]]))
                resultados["metadados_imagens"] = [m for m in metadados if m]  # Filtrar None
        except Exception as e:
            logger.error(f"Erro ao analisar metadados das imagens: {e}")

        return resultados

    def extrair_metadados_imagem(self, url_imagem):
        """Extrai metadados de uma imagem a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url_imagem, headers=headers, stream=True, timeout=5, verify=False)
            response.raise_for_status()

            # Abrir a imagem com PIL
            img = Image.open(io.BytesIO(response.content))

            # Coletar metadados básicos
            metadados = {
                "formato": img.format,
                "tamanho": img.size,
                "modo": img.mode,
                "url": url_imagem,
                "hash": hashlib.md5(response.content).hexdigest()
            }

            # Tentar extrair EXIF se disponível
            if hasattr(img, '_getexif') and img._getexif():
                exif = img._getexif()
                if exif:
                    exif_data = {}
                    for tag_id, value in exif.items():
                        # Converter para string para garantir serialização JSON
                        exif_data[str(tag_id)] = str(value)
                    metadados["exif"] = exif_data

            return metadados
        except (requests.RequestException, UnidentifiedImageError, IOError) as e:
            logger.warning(f"Erro ao extrair metadados da imagem {url_imagem}: {e}")
            return None

    def buscar_fact_check(self, texto, titulo=None):
        """Busca por verificações de fatos relacionadas ao conteúdo."""
        resultados = {}

        try:
            # Preparar termos de busca
            query = titulo if titulo else texto[:100]

            # Consultar API de fact check
            data = self.consultar_fact_check(query)

            if data:
                resultados["fact_checks"] = data.get('claims', [])
                resultados["num_fact_checks"] = len(resultados["fact_checks"])

                # Analisar resultados
                if resultados["num_fact_checks"] > 0:
                    # Verificar ratings médios
                    ratings = []
                    for claim in resultados["fact_checks"]:
                        for review in claim.get('claimReview', []):
                            if 'textualRating' in review:
                                ratings.append(review['textualRating'].lower())

                    # Contar frequência de cada rating
                    if ratings:
                        counter = Counter(ratings)
                        resultados["ratings_frequencia"] = counter

                        # Detectar termos comuns de falsidade
                        termos_falso = ['false', 'fake', 'falso', 'mentira', 'enganoso', 'misleading']
                        count_falso = sum(counter.get(termo, 0) for termo in termos_falso)

                        if count_falso > 0:
                            resultados["alerta_fact_check"] = "Conteúdo marcado como falso ou enganoso por verificadores de fatos"
                            resultados["pontos_fact_check"] = -25
                        else:
                            resultados["pontos_fact_check"] = 0
            else:
                resultados["fact_checks"] = []
                resultados["num_fact_checks"] = 0
                resultados["pontos_fact_check"] = 0

        except Exception as e:
            logger.error(f"Erro ao buscar fact checks: {e}")
            resultados["erro_fact_check"] = str(e)
            resultados["pontos_fact_check"] = 0

        return resultados

    def buscar_noticias_similares(self, titulo, keywords, conteudo=None):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}
        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta

            # Simular consistência entre fontes
            resultados["consistencia_entre_fontes"] = "alta"  # baixa, média, alta
            resultados["pontos_consistencia"] = 10  # Simulando pontos por alta consistência

        except Exception as e:
            logger.error(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)
            resultados["pontos_consistencia"] = 0

        return resultados

    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto

    def analisar_credibilidade(self, url):
        """Análise completa de credibilidade de uma URL de notícia."""
        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo or not artigo.get('conteudo'):
            logger.error(f"Não foi possível baixar o artigo: {url}")
            return {"erro": "Não foi possível baixar o artigo para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 2. Verificação de fonte/domínio
        status_fonte, pontos_fonte, info_fonte = self.verificar_fonte(url)
        resultados.update(info_fonte)

        # 3. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 4. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 5. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 6. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)
        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 7. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 8. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 9. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', []),
            conteudo
        )
        resultados.update(resultados_similares)

        # 10. Verificação em serviços de fact-checking
        resultados_fact_check = self.buscar_fact_check(conteudo, artigo['titulo'])
        resultados.update(resultados_fact_check)

        # Cálculo mais sofisticado usando vários fatores
# 11. Determinar um nível de veracidade
        # Cálculo mais sofisticado usando vários fatores
        base_pontuacao = 70  # Base inicial neutra
               # Fatores que afetam a pontuação
        pontuacao_fatores = [
            resultados.get("pontos_fonte", 0),
            resultados.get("pontos_consistencia", 0),
            resultados.get("pontos_evidencia", 0),
            resultados.get("pontos_conhecimento_previo", 0),
            resultados.get("pontos_contexto", 0),
            resultados.get("pontos_bias", 0)
        ]
                # Aplicar pesos aos fatores
        pesos = [1.5, 1.2, 2.0, 1.0, 1.0, 0.8]
        pontuacao_ponderada = sum(p * f for p, f in zip(pesos, pontuacao_fatores))
                # Calcular pontuação final
        pontuacao_final = max(0, min(100, base_pontuacao + pontuacao_ponderada))
                # Determinar nível de veracidade baseado na pontuação final
        if pontuacao_final >= 90:
            nivel_veracidade = "Altamente verificado"
        elif pontuacao_final >= 75:
            nivel_veracidade = "Provavelmente verdadeiro"
        elif pontuacao_final >= 60:
            nivel_veracidade = "Parcialmente verificado"
        elif pontuacao_final >= 40:
            nivel_veracidade = "Inconclusivo"
        elif pontuacao_final >= 25:
            nivel_veracidade = "Provavelmente falso"
        else:
            nivel_veracidade = "Falso"

        resultados["pontuacao_veracidade"] = pontuacao_final
        resultados["nivel_veracidade"] = nivel_veracidade
                # 12. Sumarizar os resultados
        resumo = {
            "nivel_veracidade": nivel_veracidade,
            "pontuacao": pontuacao_final,
            "analise_detalhada": resultados,
            "pontos_chave": {
                "fontes": resultados.get("qualidade_fonte", "Não analisado"),
                "consistencia": resultados.get("consistencia_narrativa", "Não analisado"),
                "evidencias": resultados.get("qualidade_evidencia", "Não analisado"),
                "contexto": resultados.get("analise_contexto", "Não analisado")
            },
            "recomendacao": ""
        }

        # Adicionar uma recomendação baseada no nível de veracidade
        if pontuacao_final >= 75:
            resumo["recomendacao"] = "Informação confiável para uso e disseminação."
        elif pontuacao_final >= 50:
            resumo["recomendacao"] = "Verificar informações adicionais antes de aceitar completamente."
        else:
            resumo["recomendacao"] = "Não recomendado para uso ou compartilhamento sem verificação adicional."
                    return resumo


Overwriting verificacao_simples.py


In [20]:
%run verificacao_simples.py

In [16]:
%%writefile verificacao_simples.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse, urlencode
import hashlib
from collections import Counter
import time
import logging
import base64
import io
import nltk
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
from textblob import TextBlob  # Para análise de sentimento
from nltk.tokenize import sent_tokenize
from PIL import Image, UnidentifiedImageError
from datetime import datetime
from textblob import TextBlob

# Configurar o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("VerificadorNoticias")

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticias:
    def __init__(self):
        logger.info("Verificador de Notícias inicializado!")

        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]
        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5, "viral": 1.5,
            "bizarro": 1.5, "aterrorizante": 2, "assustador": 1.5, "terrível": 1.5,
            "estarrecedor": 2, "impossível": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors",
            "parece que", "há relatos", "não confirmado", "segundo fontes",
            "indícios", "sinalizam", "aparentemente", "estaria", "poderia",
            "teria", "estima-se"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe",
            "na visão de", "do meu ponto de vista", "considero", "entendo que",
            "me parece", "creio que", "penso que", "sinto que"
        ]

        # URLs de APIs para verificação de fatos
        self.fact_check_api_url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"
        self.fact_check_api_key = "SUA_API_KEY_AQUI"  # Substitua pela sua chave de API real

        # Tentar baixar recursos NLTK necessários para análise de texto
        try:
            nltk.download('punkt', quiet=True)
        except Exception as e:
            logger.warning(f"Não foi possível baixar recursos NLTK: {e}")

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        return parsed_url.netloc

    def verificar_url_simples(self, url):
        """Executa uma verificação simples em uma URL de notícia."""
        try:
            # Extrair domínio
            parsed_url = urlparse(url)
            dominio = parsed_url.netloc

            # Tentar acessar a URL
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10, verify=False)
            response.raise_for_status()  # Levantar exceção para status HTTP de erro

            # Verificar status HTTP
            if response.status_code == 200:
                logger.info(f"URL acessada com sucesso: {url}")
                logger.info(f"Domínio: {dominio}")
                logger.info(f"Tamanho do conteúdo: {len(response.text)} caracteres.")
                return {
                    "status": "sucesso",
                    "dominio": dominio,
                    "tamanho_conteudo": len(response.text),
                    "mensagem": "URL acessada com sucesso"
                }
            else:
                logger.warning(f"Erro ao acessar a URL. Código HTTP: {response.status_code}")
                return {
                    "status": "erro",
                    "mensagem": f"Erro ao acessar a URL. Código HTTP: {response.status_code}"
                }

        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao acessar a URL: {e}")
            return {
                "status": "erro",
                "mensagem": str(e)
            }
def analisar_sentimento(self, texto):
    sentiment = TextBlob(texto).sentiment
    return {
        "polaridade": sentiment.polarity,
        "subjetividade": sentiment.subjectivity
    }

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {"dominio": dominio}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
            return "Fonte confiável", 20, resultado
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
            return "Fonte suspeita", -30, resultado
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0
            return "Fonte não categorizada", 0, resultado

    def baixar_artigo(self, url):
        """Baixa o conteúdo do artigo a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""
            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)
                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date'] or meta.get('name') in ['publication_date', 'date', 'pubdate']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]', '[name=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(author_tag)
                else:
                    elements = soup.find_all(author_tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords,
                'html': response.text  # Manter o HTML para possível análise adicional
            }
        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            logger.error(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None
# Integra API do Google Fact Check.
def consultar_fact_check(self, query):
    url_api = f"https://factchecktools.googleapis.com/v1alpha1/claims:search?query={query}"
    try:
        response = requests.get(url_api)
        if response.status_code == 200:
            return response.json()
        else:
            logger.warning("Falha ao acessar API de Fact Check.")
            return None
    except Exception as e:
        logger.error(f"Erro na API de Fact Check: {e}")
        return None
    def analisar_linguagem(self, texto):
        """Analisa o conteúdo em busca de sensacionalismo, incerteza e opinatividade."""
        if not texto or len(texto) < 10:
            return {
                "sensacionalismo": 0,
                "incerteza": 0,
                "opinativo": 0,
                "exclamacoes": 0
            }

        resultado = {}

        # Análise de sensacionalismo
        indice_sensacionalismo = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower())) * peso
            for palavra, peso in self.palavras_sensacionalistas.items()
        )
        resultado["sensacionalismo"] = indice_sensacionalismo

        # Normalizar por tamanho do texto (por 1000 palavras)
        palavras_totais = len(texto.split())
        if palavras_totais > 0:
            indice_sensacionalismo_norm = (indice_sensacionalismo * 1000) / palavras_totais
            resultado["sensacionalismo_normalizado"] = indice_sensacionalismo_norm

            # Adicionar alerta se o índice for alto
            if indice_sensacionalismo_norm > 10:
                resultado["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista"
            elif indice_sensacionalismo_norm > 5:
                resultado["alerta_sensacionalismo"] = "Moderado índice de linguagem sensacionalista"

        # Análise de incerteza
        indice_incerteza = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower()))
            for palavra in self.palavras_incerteza
        )
        resultado["incerteza"] = indice_incerteza

        if palavras_totais > 0:
            indice_incerteza_norm = (indice_incerteza * 1000) / palavras_totais
            resultado["incerteza_normalizada"] = indice_incerteza_norm

            if indice_incerteza_norm > 8:
                resultado["alerta_incerteza"] = "Alto índice de expressões de incerteza"

        # Análise de opinatividade
        indice_opiniao = sum(
            len(re.findall(r'\b' + re.escape(expressao) + r'\b', texto.lower()))
            for expressao in self.expressoes_opinativas
        )
        resultado["opinativo"] = indice_opiniao

        if palavras_totais > 0:
            indice_opiniao_norm = (indice_opiniao * 1000) / palavras_totais
            resultado["opinativo_normalizado"] = indice_opiniao_norm

            if indice_opiniao_norm > 5:
                resultado["alerta_opiniao"] = "Alto índice de expressões opinativas"

        # Contagem de pontos de exclamação (indicador de sensacionalismo)
        exclamacoes = texto.count('!')
        resultado["exclamacoes"] = exclamacoes

        if palavras_totais > 0:
            exclamacoes_norm = (exclamacoes * 1000) / palavras_totais
            if exclamacoes_norm > 10:
                resultado["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        # Análise de sentimento usando TextBlob
        try:
            analise = TextBlob(texto)
            resultado["sentimento_polaridade"] = analise.sentiment.polarity  # -1 a 1 (negativo a positivo)
            resultado["sentimento_subjetividade"] = analise.sentiment.subjectivity  # 0 a 1 (objetivo a subjetivo)

            # Categorização baseada na polaridade
            if resultado["sentimento_polaridade"] < -0.5:
                resultado["sentimento_categoria"] = "muito negativo"
            elif resultado["sentimento_polaridade"] < -0.1:
                resultado["sentimento_categoria"] = "negativo"
            elif resultado["sentimento_polaridade"] <= 0.1:
                resultado["sentimento_categoria"] = "neutro"
            elif resultado["sentimento_polaridade"] <= 0.5:
                resultado["sentimento_categoria"] = "positivo"
            else:
                resultado["sentimento_categoria"] = "muito positivo"

            # Alerta para alto nível de subjetividade em notícias que deveriam ser objetivas
            if resultado["sentimento_subjetividade"] > 0.6:
                resultado["alerta_subjetividade"] = "Conteúdo altamente subjetivo"

        except Exception as e:
            logger.warning(f"Erro na análise de sentimento: {e}")

        return resultado

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}
        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        # Analisar metadados das primeiras 5 imagens (limitar para evitar muito processamento)
        try:
            with ThreadPoolExecutor(max_workers=3) as executor:
                metadados = list(executor.map(self.extrair_metadados_imagem, [img['url'] for img in imagens[:5]]))
                resultados["metadados_imagens"] = [m for m in metadados if m]  # Filtrar None
        except Exception as e:
            logger.error(f"Erro ao analisar metadados das imagens: {e}")

        return resultados

    def extrair_metadados_imagem(self, url_imagem):
        """Extrai metadados de uma imagem a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url_imagem, headers=headers, stream=True, timeout=5, verify=False)
            response.raise_for_status()

            # Abrir a imagem com PIL
            img = Image.open(io.BytesIO(response.content))

            # Coletar metadados básicos
            metadados = {
                "formato": img.format,
                "tamanho": img.size,
                "modo": img.mode,
                "url": url_imagem,
                "hash": hashlib.md5(response.content).hexdigest()
            }

            # Tentar extrair EXIF se disponível
            if hasattr(img, '_getexif') and img._getexif():
                exif = img._getexif()
                if exif:
                    exif_data = {}
                    for tag_id, value in exif.items():
                        # Converter para string para garantir serialização JSON
                        exif_data[str(tag_id)] = str(value)
                    metadados["exif"] = exif_data

            return metadados
        except (requests.RequestException, UnidentifiedImageError, IOError) as e:
            logger.warning(f"Erro ao extrair metadados da imagem {url_imagem}: {e}")
            return None

class SistemaVerificacaoUnificado:
    def __init__(self):
        # Carregar variáveis do arquivo .env
        load_dotenv()
        self.fact_check_api_key = os.getenv("FACT_CHECK_API_KEY")

        # Verificar se a chave de API está disponível
        if not self.fact_check_api_key or self.fact_check_api_key.strip() == "":
            raise ValueError("Chave de API de verificação de fatos não encontrada. Verifique o arquivo .env.")

            return resultados
                    try:
            # Preparar termos de busca
            query = titulo if titulo else texto[:100]
                        # Parâmetros da consulta
            params = {
                'key': self.fact_check_api_key,
                'query': query,
                'languageCode': 'pt'  # Você pode ajustar ou detectar automaticamente
            }

            # Fazer a requisição
            response = requests.get(
                self.fact_check_api_url,
                params=params
            )

            if response.status_code == 200:
                data = response.json()
                resultados["fact_checks"] = data.get('claims', [])
                resultados["num_fact_checks"] = len(resultados["fact_checks"])

                # Analisar resultados
                if resultados["num_fact_checks"] > 0:
                    # Verificar ratings médios
                    ratings = []
                    for claim in resultados["fact_checks"]:
                        for review in claim.get('claimReview', []):
                            if 'textualRating' in review:
                                ratings.append(review['textualRating'].lower())

                    # Contar frequência de cada rating
                    if ratings:
                        counter = Counter(ratings)
                        resultados["ratings_frequencia"] = counter

                        # Detectar termos comuns de falsidade
                        termos_falso = ['false', 'fake', 'falso', 'mentira', 'enganoso', 'misleading']
                        count_falso = sum(counter.get(termo, 0) for termo in termos_falso)

                        if count_falso > 0:
                            resultados["alerta_fact_check"] = "Conteúdo marcado como falso ou enganoso por verificadores de fatos"
                            resultados["pontos_fact_check"] = -25
                        else:
                            resultados["pontos_fact_check"] = 0
            else:
                logger.warning(f"Erro na API de Fact Check. Status: {response.status_code}")
                resultados["erro_fact_check"] = f"Erro na API. Status: {response.status_code}"

        except Exception as e:
            logger.error(f"Erro ao buscar fact checks: {e}")
            resultados["erro_fact_check"] = str(e)

        return resultados

    def buscar_noticias_similares(self, titulo, keywords, conteudo=None):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}
        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta

            # Simular consistência entre fontes
            resultados["consistencia_entre_fontes"] = "alta"  # baixa, média, alta
            resultados["pontos_consistencia"] = 10  # Simulando pontos por alta consistência

        except Exception as e:
            logger.error(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)

        return resultados

    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto

    def analisar_credibilidade(self, artigo):
        """Análise combinada de credibilidade baseada em características do texto e fonte."""
        if not artigo or not artigo.get('conteudo'):
            return {"erro": "Artigo não disponível para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 1. Verificação de fonte/domínio
        _, _, info_fonte = self.verificar_fonte(artigo['url'])
        resultados.update(info_fonte)

        # 2. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 3. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 4. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 5. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)
        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 6. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 7. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 8. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', []),
            conteudo
        )
        resultados.update(resultados_similares)

        # 9. Verificação em serviços de fact-checking
        resultados_fact_check = self.buscar_fact_check(conteudo, artigo['titulo'])
        resultados.update(resultados_fact_check)
                # 10. Determinar um nível de veracidade
        # Cálculo mais sofisticado usando vários fatores
        base_pontuacao = 70  # Base inicial neutra
                # Fatores que afetam a pontuação
        pontuacao_fatores = [
            resultados.get("pontos_fonte", 0),
            resultados.get("pontos_fact_check", 0),
            resultados.get("pontos_consistencia", 0)
        ]

        # Penalidades
        if "alerta_tamanho" in resultados:
            pontuacao_fatores.append(-15)
        if "alerta_sensacionalismo" in resultados:
            if "Alto índice" in resultados["alerta_sensacionalismo"]:
                pontuacao_fatores.append(-20)
            else:
                pontuacao_fatores.append(-10)
        if "alerta_fontes" in resultados:
            pontuacao_fatores.append(-10)
        if "alerta_opiniao" in resultados:
            pontuacao_fatores.append(-15)
        if "alerta_incerteza" in resultados and resultados["incerteza"] > 3:
            pontuacao_fatores.append(-10)
        if "alerta_exclamacoes" in resultados:
            pontuacao_fatores.append(-5)
        if "alerta_imagens_descricao" in resultados:
            pontuacao_fatores.append(-5)
        if "alerta_fact_check" in resultados:
            pontuacao_fatores.append(-25)
        if resultados.get("sentimento_subjetividade", 0) > 0.7:
            pontuacao_fatores.append(-15)

        # Bônus
        if resultados.get("num_citacoes", 0) > 3:
            pontuacao_fatores.append(10)
        elif resultados.get("num_citacoes", 0) > 0:
            pontuacao_fatores.append(5)
        if resultados.get("num_urls", 0) > 2:
            pontuacao_fatores.append(5)
        if resultados.get("num_imagens", 0) > 1:
            pontuacao_fatores.append(5)
        if resultados.get("artigos_similares", 0) > 3:
            pontuacao_fatores.append(5)

        # Calcular pontuação final
        pontuacao_final = base_pontuacao + sum(pontuacao_fatores)
        pontuacao_final = max(0, min(100, pontuacao_final))
        resultados["pontuacao_credibilidade"] = pontuacao_final
# Converter para nível de veracidade
nivel_veracidade = 3  # Default: Inconclusivo
if pontuacao_final >= 95:
    nivel_veracidade = 1  # Confirmado
elif pontuacao_final >= 85:
    nivel_veracidade = 2  # Provavelmente verdadeiro
elif pontuacao_final <= 30:
    nivel_veracidade = 4  # Provavelmente falso
elif pontuacao_final < 85 and pontuacao_final > 30:
    nivel_veracidade = 3  # Inconclusivo
# Adicionar o nível de veracidade aos resultados
resultados["nivel_veracidade"] = nivel_veracidade
# Exibir os resultados (opcional)
print(f"Pontuação de credibilidade: {resultados['pontuacao_credibilidade']}")
print(f"Nível de veracidade: {resultados['nivel_veracidade']}")


Overwriting verificacao_simples.py


In [14]:
%%writefile verificacao_simples.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse, urlencode
import hashlib
from collections import Counter
import time
import logging
import base64
import io
import nltk
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
from textblob import TextBlob  # Para análise de sentimento
from nltk.tokenize import sent_tokenize
from PIL import Image, UnidentifiedImageError
from datetime import datetime
from textblob import TextBlob

# Configurar o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("VerificadorNoticias")

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticias:
    def __init__(self):
        logger.info("Verificador de Notícias inicializado!")

        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]
                self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5, "viral": 1.5,
            "bizarro": 1.5, "aterrorizante": 2, "assustador": 1.5, "terrível": 1.5,
            "estarrecedor": 2, "impossível": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors",
            "parece que", "há relatos", "não confirmado", "segundo fontes",
            "indícios", "sinalizam", "aparentemente", "estaria", "poderia",
            "teria", "estima-se"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe",
            "na visão de", "do meu ponto de vista", "considero", "entendo que",
            "me parece", "creio que", "penso que", "sinto que"
        ]

        # URLs de APIs para verificação de fatos
        self.fact_check_api_url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"
        self.fact_check_api_key = "SUA_API_KEY_AQUI"  # Substitua pela sua chave de API real

        # Tentar baixar recursos NLTK necessários para análise de texto
        try:
            nltk.download('punkt', quiet=True)
        except Exception as e:
            logger.warning(f"Não foi possível baixar recursos NLTK: {e}")

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        return parsed_url.netloc

    def verificar_url_simples(self, url):
        """Executa uma verificação simples em uma URL de notícia."""
        try:
            # Extrair domínio
            parsed_url = urlparse(url)
            dominio = parsed_url.netloc

            # Tentar acessar a URL
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10, verify=False)
            response.raise_for_status()  # Levantar exceção para status HTTP de erro

            # Verificar status HTTP
            if response.status_code == 200:
                logger.info(f"URL acessada com sucesso: {url}")
                logger.info(f"Domínio: {dominio}")
                logger.info(f"Tamanho do conteúdo: {len(response.text)} caracteres.")
                return {
                    "status": "sucesso",
                    "dominio": dominio,
                    "tamanho_conteudo": len(response.text),
                    "mensagem": "URL acessada com sucesso"
                }
            else:
                logger.warning(f"Erro ao acessar a URL. Código HTTP: {response.status_code}")
                return {
                    "status": "erro",
                    "mensagem": f"Erro ao acessar a URL. Código HTTP: {response.status_code}"
                }

        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao acessar a URL: {e}")
            return {
                "status": "erro",
                "mensagem": str(e)
            }
def analisar_sentimento(self, texto):
    sentiment = TextBlob(texto).sentiment
    return {
        "polaridade": sentiment.polarity,
        "subjetividade": sentiment.subjectivity
    }

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {"dominio": dominio}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
            return "Fonte confiável", 20, resultado
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
            return "Fonte suspeita", -30, resultado
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0
            return "Fonte não categorizada", 0, resultado

    def baixar_artigo(self, url):
        """Baixa o conteúdo do artigo a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""
            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)
                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date'] or meta.get('name') in ['publication_date', 'date', 'pubdate']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]', '[name=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(author_tag)
                else:
                    elements = soup.find_all(author_tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords,
                'html': response.text  # Manter o HTML para possível análise adicional
            }
        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            logger.error(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None
# Integra API do Google Fact Check.
def consultar_fact_check(self, query):
    url_api = f"https://factchecktools.googleapis.com/v1alpha1/claims:search?query={query}"
    try:
        response = requests.get(url_api)
        if response.status_code == 200:
            return response.json()
        else:
            logger.warning("Falha ao acessar API de Fact Check.")
            return None
    except Exception as e:
        logger.error(f"Erro na API de Fact Check: {e}")
        return None
    def analisar_linguagem(self, texto):
        """Analisa o conteúdo em busca de sensacionalismo, incerteza e opinatividade."""
        if not texto or len(texto) < 10:
            return {
                "sensacionalismo": 0,
                "incerteza": 0,
                "opinativo": 0,
                "exclamacoes": 0
            }

        resultado = {}

        # Análise de sensacionalismo
        indice_sensacionalismo = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower())) * peso
            for palavra, peso in self.palavras_sensacionalistas.items()
        )
        resultado["sensacionalismo"] = indice_sensacionalismo

        # Normalizar por tamanho do texto (por 1000 palavras)
        palavras_totais = len(texto.split())
        if palavras_totais > 0:
            indice_sensacionalismo_norm = (indice_sensacionalismo * 1000) / palavras_totais
            resultado["sensacionalismo_normalizado"] = indice_sensacionalismo_norm

            # Adicionar alerta se o índice for alto
            if indice_sensacionalismo_norm > 10:
                resultado["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista"
            elif indice_sensacionalismo_norm > 5:
                resultado["alerta_sensacionalismo"] = "Moderado índice de linguagem sensacionalista"

        # Análise de incerteza
        indice_incerteza = sum(
            len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto.lower()))
            for palavra in self.palavras_incerteza
        )
        resultado["incerteza"] = indice_incerteza

        if palavras_totais > 0:
            indice_incerteza_norm = (indice_incerteza * 1000) / palavras_totais
            resultado["incerteza_normalizada"] = indice_incerteza_norm

            if indice_incerteza_norm > 8:
                resultado["alerta_incerteza"] = "Alto índice de expressões de incerteza"

        # Análise de opinatividade
        indice_opiniao = sum(
            len(re.findall(r'\b' + re.escape(expressao) + r'\b', texto.lower()))
            for expressao in self.expressoes_opinativas
        )
        resultado["opinativo"] = indice_opiniao

        if palavras_totais > 0:
            indice_opiniao_norm = (indice_opiniao * 1000) / palavras_totais
            resultado["opinativo_normalizado"] = indice_opiniao_norm

            if indice_opiniao_norm > 5:
                resultado["alerta_opiniao"] = "Alto índice de expressões opinativas"

        # Contagem de pontos de exclamação (indicador de sensacionalismo)
        exclamacoes = texto.count('!')
        resultado["exclamacoes"] = exclamacoes

        if palavras_totais > 0:
            exclamacoes_norm = (exclamacoes * 1000) / palavras_totais
            if exclamacoes_norm > 10:
                resultado["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        # Análise de sentimento usando TextBlob
        try:
            analise = TextBlob(texto)
            resultado["sentimento_polaridade"] = analise.sentiment.polarity  # -1 a 1 (negativo a positivo)
            resultado["sentimento_subjetividade"] = analise.sentiment.subjectivity  # 0 a 1 (objetivo a subjetivo)

            # Categorização baseada na polaridade
            if resultado["sentimento_polaridade"] < -0.5:
                resultado["sentimento_categoria"] = "muito negativo"
            elif resultado["sentimento_polaridade"] < -0.1:
                resultado["sentimento_categoria"] = "negativo"
            elif resultado["sentimento_polaridade"] <= 0.1:
                resultado["sentimento_categoria"] = "neutro"
            elif resultado["sentimento_polaridade"] <= 0.5:
                resultado["sentimento_categoria"] = "positivo"
            else:
                resultado["sentimento_categoria"] = "muito positivo"

            # Alerta para alto nível de subjetividade em notícias que deveriam ser objetivas
            if resultado["sentimento_subjetividade"] > 0.6:
                resultado["alerta_subjetividade"] = "Conteúdo altamente subjetivo"

        except Exception as e:
            logger.warning(f"Erro na análise de sentimento: {e}")

        return resultado

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}
        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        # Analisar metadados das primeiras 5 imagens (limitar para evitar muito processamento)
        try:
            with ThreadPoolExecutor(max_workers=3) as executor:
                metadados = list(executor.map(self.extrair_metadados_imagem, [img['url'] for img in imagens[:5]]))
                resultados["metadados_imagens"] = [m for m in metadados if m]  # Filtrar None
        except Exception as e:
            logger.error(f"Erro ao analisar metadados das imagens: {e}")

        return resultados

    def extrair_metadados_imagem(self, url_imagem):
        """Extrai metadados de uma imagem a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url_imagem, headers=headers, stream=True, timeout=5, verify=False)
            response.raise_for_status()

            # Abrir a imagem com PIL
            img = Image.open(io.BytesIO(response.content))

            # Coletar metadados básicos
            metadados = {
                "formato": img.format,
                "tamanho": img.size,
                "modo": img.mode,
                "url": url_imagem,
                "hash": hashlib.md5(response.content).hexdigest()
            }

            # Tentar extrair EXIF se disponível
            if hasattr(img, '_getexif') and img._getexif():
                exif = img._getexif()
                if exif:
                    exif_data = {}
                    for tag_id, value in exif.items():
                        # Converter para string para garantir serialização JSON
                        exif_data[str(tag_id)] = str(value)
                    metadados["exif"] = exif_data

            return metadados
        except (requests.RequestException, UnidentifiedImageError, IOError) as e:
            logger.warning(f"Erro ao extrair metadados da imagem {url_imagem}: {e}")
            return None

class SistemaVerificacaoUnificado:
    def __init__(self):
        # Carregar variáveis do arquivo .env
        load_dotenv()
        self.fact_check_api_key = os.getenv("FACT_CHECK_API_KEY")

        # Verificar se a chave de API está disponível
        if not self.fact_check_api_key or self.fact_check_api_key.strip() == "":
            raise ValueError("Chave de API de verificação de fatos não encontrada. Verifique o arquivo .env.")

            return resultados
                    try:
            # Preparar termos de busca
            query = titulo if titulo else texto[:100]
                        # Parâmetros da consulta
            params = {
                'key': self.fact_check_api_key,
                'query': query,
                'languageCode': 'pt'  # Você pode ajustar ou detectar automaticamente
            }

            # Fazer a requisição
            response = requests.get(
                self.fact_check_api_url,
                params=params
            )

            if response.status_code == 200:
                data = response.json()
                resultados["fact_checks"] = data.get('claims', [])
                resultados["num_fact_checks"] = len(resultados["fact_checks"])

                # Analisar resultados
                if resultados["num_fact_checks"] > 0:
                    # Verificar ratings médios
                    ratings = []
                    for claim in resultados["fact_checks"]:
                        for review in claim.get('claimReview', []):
                            if 'textualRating' in review:
                                ratings.append(review['textualRating'].lower())

                    # Contar frequência de cada rating
                    if ratings:
                        counter = Counter(ratings)
                        resultados["ratings_frequencia"] = counter

                        # Detectar termos comuns de falsidade
                        termos_falso = ['false', 'fake', 'falso', 'mentira', 'enganoso', 'misleading']
                        count_falso = sum(counter.get(termo, 0) for termo in termos_falso)

                        if count_falso > 0:
                            resultados["alerta_fact_check"] = "Conteúdo marcado como falso ou enganoso por verificadores de fatos"
                            resultados["pontos_fact_check"] = -25
                        else:
                            resultados["pontos_fact_check"] = 0
            else:
                logger.warning(f"Erro na API de Fact Check. Status: {response.status_code}")
                resultados["erro_fact_check"] = f"Erro na API. Status: {response.status_code}"

        except Exception as e:
            logger.error(f"Erro ao buscar fact checks: {e}")
            resultados["erro_fact_check"] = str(e)

        return resultados

    def buscar_noticias_similares(self, titulo, keywords, conteudo=None):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}
        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta

            # Simular consistência entre fontes
            resultados["consistencia_entre_fontes"] = "alta"  # baixa, média, alta
            resultados["pontos_consistencia"] = 10  # Simulando pontos por alta consistência

        except Exception as e:
            logger.error(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)

        return resultados

    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto

    def analisar_credibilidade(self, artigo):
        """Análise combinada de credibilidade baseada em características do texto e fonte."""
        if not artigo or not artigo.get('conteudo'):
            return {"erro": "Artigo não disponível para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 1. Verificação de fonte/domínio
        _, _, info_fonte = self.verificar_fonte(artigo['url'])
        resultados.update(info_fonte)

        # 2. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 3. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 4. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 5. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)
        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 6. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 7. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 8. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', []),
            conteudo
        )
        resultados.update(resultados_similares)

        # 9. Verificação em serviços de fact-checking
        resultados_fact_check = self.buscar_fact_check(conteudo, artigo['titulo'])
        resultados.update(resultados_fact_check)
                # 10. Determinar um nível de veracidade
        # Cálculo mais sofisticado usando vários fatores
        base_pontuacao = 70  # Base inicial neutra
                # Fatores que afetam a pontuação
        pontuacao_fatores = [
            resultados.get("pontos_fonte", 0),
            resultados.get("pontos_fact_check", 0),
            resultados.get("pontos_consistencia", 0)
        ]

        # Penalidades
        if "alerta_tamanho" in resultados:
            pontuacao_fatores.append(-15)
        if "alerta_sensacionalismo" in resultados:
            if "Alto índice" in resultados["alerta_sensacionalismo"]:
                pontuacao_fatores.append(-20)
            else:
                pontuacao_fatores.append(-10)
        if "alerta_fontes" in resultados:
            pontuacao_fatores.append(-10)
        if "alerta_opiniao" in resultados:
            pontuacao_fatores.append(-15)
        if "alerta_incerteza" in resultados and resultados["incerteza"] > 3:
            pontuacao_fatores.append(-10)
        if "alerta_exclamacoes" in resultados:
            pontuacao_fatores.append(-5)
        if "alerta_imagens_descricao" in resultados:
            pontuacao_fatores.append(-5)
        if "alerta_fact_check" in resultados:
            pontuacao_fatores.append(-25)
        if resultados.get("sentimento_subjetividade", 0) > 0.7:
            pontuacao_fatores.append(-15)

        # Bônus
        if resultados.get("num_citacoes", 0) > 3:
            pontuacao_fatores.append(10)
        elif resultados.get("num_citacoes", 0) > 0:
            pontuacao_fatores.append(5)
        if resultados.get("num_urls", 0) > 2:
            pontuacao_fatores.append(5)
        if resultados.get("num_imagens", 0) > 1:
            pontuacao_fatores.append(5)
        if resultados.get("artigos_similares", 0) > 3:
            pontuacao_fatores.append(5)

        # Calcular pontuação final
        pontuacao_final = base_pontuacao + sum(pontuacao_fatores)
        pontuacao_final = max(0, min(100, pontuacao_final))
        resultados["pontuacao_credibilidade"] = pontuacao_final
# Converter para nível de veracidade
nivel_veracidade = 3  # Default: Inconclusivo
if pontuacao_final >= 95:
    nivel_veracidade = 1  # Confirmado
elif pontuacao_final >= 85:
    nivel_veracidade = 2  # Provavelmente verdadeiro
elif pontuacao_final <= 30:
    nivel_veracidade = 4  # Provavelmente falso
elif pontuacao_final < 85 and pontuacao_final > 30:
    nivel_veracidade = 3  # Inconclusivo
# Adicionar o nível de veracidade aos resultados
resultados["nivel_veracidade"] = nivel_veracidade
# Exibir os resultados (opcional)
print(f"Pontuação de credibilidade: {resultados['pontuacao_credibilidade']}")
print(f"Nível de veracidade: {resultados['nivel_veracidade']}")


Overwriting verificacao_simples.py


In [13]:
%run verificacao_simples.py

Analisando a URL: https://ultimosegundo.ig.com.br/mundo/2025-04-19/casa-branca--site-defende-teoria-de-que-coronavirus-vazou-de-laboratorio-na-china.html
Verificador de Notícias Avançado inicializado!
{'titulo': 'Trump defende que coronavírus teve origem em laboratório chinês', 'fonte_status': 'Fonte não categorizada', 'analise_linguagem': {'sensacionalismo': 0.0, 'incerteza': 0, 'opinativo': 0}, 'pontuacao': 50.0}


In [12]:
%%writefile verificacao_simples.py

import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse
import hashlib
from collections import Counter
import time

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticiasAvancado:
    def __init__(self):
        print("Verificador de Notícias Avançado inicializado!")
        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]
        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]
        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5
        }
        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors"
        ]
        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe"
        ]

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        return parsed_url.netloc

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
            return "Fonte confiável", 20
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
            return "Fonte suspeita", -30
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0
            return "Fonte não categorizada", 0

        resultado["dominio"] = dominio
        return resultado

    def baixar_artigo(self, url):
        """Baixa o conteúdo do artigo a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""
            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)
                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(author_tag)
                else:
                    elements = soup.find_all(author_tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords
            }
        except requests.exceptions.RequestException as e:
            print(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            print(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None

    def analisar_linguagem(self, texto):
        """Analisa o conteúdo em busca de sensacionalismo, incerteza e opinatividade."""
        indice_sensacionalismo = sum(
            len(re.findall(r'\b' + palavra + r'\b', texto.lower())) * peso
            for palavra, peso in self.palavras_sensacionalistas.items()
        )
        indice_incerteza = sum(
            len(re.findall(r'\b' + palavra + r'\b', texto.lower())) for palavra in self.palavras_incerteza
        )
        indice_opiniao = sum(
            len(re.findall(r'\b' + expressao + r'\b', texto.lower())) for expressao in self.expressoes_opinativas
        )
        return {
            "sensacionalismo": indice_sensacionalismo,
            "incerteza": indice_incerteza,
            "opinativo": indice_opiniao
        }

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}
        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados
        resultados["num_imagens"] = len(imagens)
        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao
        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"
        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls
        return resultados

    def verificar_noticia(self, url):
        """Realiza a verificação completa da notícia."""
        artigo = self.baixar_artigo(url)
        if not artigo:
            return {"erro": "Não foi possível processar o artigo."}

        fonte_status, pontos_fonte = self.verificar_fonte(url)
        analise_linguagem = self.analisar_linguagem(artigo["conteudo"])
        pontuacao = 50 + pontos_fonte - analise_linguagem["sensacionalismo"] - analise_linguagem["incerteza"]

        return {
            "titulo": artigo["titulo"],
            "fonte_status": fonte_status,
            "analise_linguagem": analise_linguagem,
            "pontuacao": max(0, min(100, pontuacao))
        }

    def buscar_noticias_similares(self, titulo, keywords):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}
        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta
        except Exception as e:
            print(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)
        return resultados

    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto

    def analisar_credibilidade(self, artigo):
        """Análise combinada de credibilidade baseada em características do texto e fonte."""
        if not artigo or not artigo.get('conteudo'):
            return {"erro": "Artigo não disponível para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 1. Verificação de fonte/domínio
        info_fonte = self.verificar_fonte(artigo['url'])
        resultados.update(info_fonte)

        # 2. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 3. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 4. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 5. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)
        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 6. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 7. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 8. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', [])
        )
        resultados.update(resultados_similares)

        # 9. Determinar um nível de veracidade (ESTA LÓGICA PRECISA SER IMPLEMENTADA)
        nivel_de_veracidade_numerico = 3  # Exemplo: Artigo considerado inconclusivo
        resultados["nivel_veracidade_numerico"] = nivel_de_veracidade_numerico

        # Converter o nível numérico para texto
        resultados["nivel_veracidade"] = self.converter_nivel_para_texto(nivel_de_veracidade_numerico)

        return resultados

    def verificar_url(self, url):
        """Função principal para verificar a credibilidade de uma notícia a partir da URL."""
        resultados = {"url": url}

        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo:
            resultados["erro"] = "Não foi possível baixar o artigo."
            return resultados

        # 2. Analisar a credibilidade do artigo
        analise_credibilidade = self.analisar_credibilidade(artigo)
        resultados.update(analise_credibilidade)

        return resultados


def analisar_url(url):
    """Função para analisar uma URL específica."""
    verificador = VerificadorNoticiasAvancado()
    resultados = verificador.verificar_url(url)
    return resultados


# Exemplo de uso (para testar no mesmo arquivo)
def verificar_url(url_para):
    # Aqui, insira o código da função `verificar_url` ou chame o método da sua classe
    print(f"Analisando a URL: {url_para}")
    # Você pode utilizar o método da classe `VerificadorNoticiasAvancado`
    verificador = VerificadorNoticiasAvancado()
    resultado = verificador.verificar_noticia(url_para)
    print(resultado)  # Mostre o relatório de análise ou como você preferir formatar a saída


if __name__ == "__main__":
    url_para_analisar = "https://ultimosegundo.ig.com.br/mundo/2025-04-19/casa-branca--site-defende-teoria-de-que-coronavirus-vazou-de-laboratorio-na-china.html"
    verificar_url(url_para_analisar)

Overwriting verificacao_simples.py


In [11]:
%run verificacao_simples.py

IndentationError: unindent does not match any outer indentation level (verificacao_simples.py, line 48)

In [10]:
%%writefile verificacao_simples.py
# verificacao_simples.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse
import hashlib
from collections import Counter
import time
# Desabilitar avisos SSL
urllib3.disable_warnings()
class VerificadorNoticiasAvancado:
    def __init__(self):
        print("Verificador de Notícias Avançado inicializado!")
           # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]
        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]
        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5
        }
        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors"
        ]
        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe"
        ]
 def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        return parsed_url.netloc



 def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            return "Fonte confiável", 20
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            return "Fonte suspeita", -30
        else:
            return "Fonte não categorizada", 0
        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0
        resultado["dominio"] = dominio
        return resultado
   def baixar_artigo(self, url):
        """Baixa o conteúdo do artigo a partir da URL."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"
            conteudo = ' '.join([p.get_text(strip=True) for p in soup.find_all('p')])
            return {"titulo": titulo, "conteudo": conteudo}
        except Exception as e:
            print(f"Erro ao baixar artigo: {e}")
            return None
            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"
            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""
            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)
                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])
            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date']:
                    data = meta.get('content')
                    break
            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)
            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })
            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]
            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords
            }
        except requests.exceptions.RequestException as e:
            print(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            print(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None

def analisar_linguagem(self, texto):
        """Analisa o conteúdo em busca de sensacionalismo, incerteza e opinatividade."""
        indice_sensacionalismo = sum(
            len(re.findall(r'\b' + palavra + r'\b', texto.lower())) * peso
            for palavra, peso in self.palavras_sensacionalistas.items()
        )
        indice_incerteza = sum(
            len(re.findall(r'\b' + palavra + r'\b', texto.lower())) for palavra in self.palavras_incerteza
        )
        indice_opiniao = sum(
            len(re.findall(r'\b' + expressao + r'\b', texto.lower())) for expressao in self.expressoes_opinativas
        )
        return {
            "sensacionalismo": indice_sensacionalismo,
            "incerteza": indice_incerteza,
            "opinativo": indice_opiniao
        }


    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}
        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados
        resultados["num_imagens"] = len(imagens)
        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao
        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"
        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls
        return resultados
def verificar_noticia(self, url):
        """Realiza a verificação completa da notícia."""
        artigo = self.baixar_artigo(url)
        if not artigo:
            return {"erro": "Não foi possível processar o artigo."}
                fonte_status, pontos_fonte = self.verificar_fonte(url)
        analise_linguagem = self.analisar_linguagem(artigo["conteudo"])
        pontuacao = 50 + pontos_fonte - analise_linguagem["sensacionalismo"] - analise_linguagem["incerteza"]
        return {
            "titulo": artigo["titulo"],
            "fonte_status": fonte_status,
            "analise_linguagem": analise_linguagem,
            "pontuacao": max(0, min(100, pontuacao))
        }
    def buscar_noticias_similares(self, titulo, keywords):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}
        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta
        except Exception as e:
            print(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)
        return resultados
    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto
    def analisar_credibilidade(self, artigo):
        """Análise combinada de credibilidade baseada em características do texto e fonte."""
        if not artigo or not artigo.get('conteudo'):
            return {"erro": "Artigo não disponível para análise"}
        resultados = {}
        conteudo = artigo['conteudo']
        # 1. Verificação de fonte/domínio
        info_fonte = self.verificar_fonte(artigo['url'])
        resultados.update(info_fonte)
        # 2. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']
        # 3. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"
        # 4. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)
        # 5. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)
        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"
        # 6. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)
        # 7. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)
        # 8. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', [])
        )
        resultados.update(resultados_similares)
        # 9. Determinar um nível de veracidade (ESTA LÓGICA PRECISA SER IMPLEMENTADA)
        nivel_de_veracidade_numerico = 3  # Exemplo: Artigo considerado inconclusivo
        resultados["nivel_veracidade_numerico"] = nivel_de_veracidade_numerico
        # Converter o nível numérico para texto
        resultados["nivel_veracidade"] = self.converter_nivel_para_texto(nivel_de_veracidade_numerico)
        return resultados
    def verificar_url(self, url):
        """Função principal para verificar a credibilidade de uma notícia a partir da URL."""
        resultados = {"url": url}
        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo:
            resultados["erro"] = "Não foi possível baixar o artigo."
            return resultados
        # 2. Analisar a credibilidade do artigo
        analise_credibilidade = self.analisar_credibilidade(artigo)
        resultados.update(analise_credibilidade)
        return resultados
def analisar_url(url):
    """Função para analisar uma URL específica."""
    verificador = VerificadorNoticiasAvancado()
    resultados = verificador.verificar_url(url)
    return resultados
# Exemplo de uso (para testar no mesmo arquivo)
def verificar_url(url_para):
    # Aqui, insira o código da função `verificar_url` ou chame o método da sua classe
    print(f"Analisando a URL: {url_para}")
    # Você pode utilizar o método da classe `VerificadorNoticiasAvancado`
    verificador = VerificadorNoticiasAvancado()
    resultado = verificador.verificar_noticia(url_para)
    print(resultado)  # Mostre o relatório de análise ou como você preferir formatar a saída
if __name__ == "__main__":
    url_para_analisar = "https://ultimosegundo.ig.com.br/mundo/2025-04-19/casa-branca--site-defende-teoria-de-que-coronavirus-vazou-de-laboratorio-na-china.html"
    verificar_url(url_para_analisar)


Overwriting verificacao_simples.py


In [9]:
%run verificacao_simples.py

Analisando a URL: https://ultimosegundo.ig.com.br/mundo/2025-04-19/casa-branca--site-defende-teoria-de-que-coronavirus-vazou-de-laboratorio-na-china.html
Verificador de Notícias Avançado inicializado!


AttributeError: 'VerificadorNoticiasAvancado' object has no attribute 'verificar_noticia'

In [8]:
%%writefile verificacao_simples.py
# verificacao_simples.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse
import hashlib
from collections import Counter
import time

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticiasAvancado:
    def __init__(self):
        print("Verificador de Notícias Avançado inicializado!")
        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]

        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe"
        ]

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        domain = parsed_url.netloc
        return domain

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0

        resultado["dominio"] = dominio
        return resultado

    def baixar_artigo(self, url):
        """Baixa e extrai o conteúdo básico de um artigo a partir da URL"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=10, allow_redirects=True)
            response.raise_for_status()  # Lança uma exceção para códigos de status HTTP ruins

            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""

            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)

                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords
            }

        except requests.exceptions.RequestException as e:
            print(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            print(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None

    def analisar_linguagem(self, texto):
        """Analisa o tom e a linguagem usada no texto."""
        resultados = {}

        # Análise de sensacionalismo com pesos
        count_sensacionalismo = 0
        palavras_encontradas = []

        for palavra, peso in self.palavras_sensacionalistas.items():
            ocorrencias = len(re.findall(r'\b' + palavra + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_sensacionalismo += ocorrencias * peso
                palavras_encontradas.append(f"{palavra} ({ocorrencias}x)")

        resultados["indice_sensacionalismo"] = count_sensacionalismo
        resultados["palavras_sensacionalistas"] = palavras_encontradas

        if count_sensacionalismo > 3:
            resultados["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista detectado"
        elif count_sensacionalismo > 1.5:
            resultados["alerta_sensacionalismo"] = "Linguagem potencialmente sensacionalista detectada"

        # Análise de incerteza
        count_incerteza = 0
        palavras_incerteza = []

        for palavra in self.palavras_incerteza:
            ocorrencias = len(re.findall(r'\b' + palavra + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_incerteza += ocorrencias
                palavras_incerteza.append(palavra)

        resultados["indice_incerteza"] = count_incerteza
        resultados["palavras_incerteza"] = palavras_incerteza

        if count_incerteza > 3:
            resultados["alerta_incerteza"] = "Alto uso de termos de incerteza detectado"

        # Análise de opinião
        count_opiniao = 0
        expressoes_opiniao = []

        for expressao in self.expressoes_opinativas:
            ocorrencias = len(re.findall(r'\b' + expressao + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_opiniao += ocorrencias
                expressoes_opiniao.append(expressao)

        resultados["indice_opiniao"] = count_opiniao
        resultados["expressoes_opiniao"] = expressoes_opiniao

        if count_opiniao > 2:
            resultados["alerta_opiniao"] = "Texto com forte caráter opinativo detectado"

        # Análise de exclamações (indicativo de sensacionalismo)
        exclamacoes = len(re.findall(r'!', texto))
        resultados["num_exclamacoes"] = exclamacoes

        if exclamacoes > 5:
            resultados["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        return resultados

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}

        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        return resultados

    def buscar_noticias_similares(self, titulo, keywords):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}

        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta
        except Exception as e:
            print(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)

        return resultados

    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto

    def analisar_credibilidade(self, artigo):
        """Análise combinada de credibilidade baseada em características do texto e fonte."""
        if not artigo or not artigo.get('conteudo'):
            return {"erro": "Artigo não disponível para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 1. Verificação de fonte/domínio
        info_fonte = self.verificar_fonte(artigo['url'])
        resultados.update(info_fonte)

        # 2. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 3. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 4. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 5. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)

        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 6. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 7. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 8. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', [])
        )
        resultados.update(resultados_similares)

        # 9. Determinar um nível de veracidade (ESTA LÓGICA PRECISA SER IMPLEMENTADA)
        nivel_de_veracidade_numerico = 3  # Exemplo: Artigo considerado inconclusivo
        resultados["nivel_veracidade_numerico"] = nivel_de_veracidade_numerico

        # Converter o nível numérico para texto
        resultados["nivel_veracidade"] = self.converter_nivel_para_texto(nivel_de_veracidade_numerico)

        return resultados

    def verificar_url(self, url):
        """Função principal para verificar a credibilidade de uma notícia a partir da URL."""
        resultados = {"url": url}

        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo:
            resultados["erro"] = "Não foi possível baixar o artigo."
            return resultados

        # 2. Analisar a credibilidade do artigo
        analise_credibilidade = self.analisar_credibilidade(artigo)
        resultados.update(analise_credibilidade)

        return resultados

def analisar_url(url):
    """Função para analisar uma URL específica."""

    verificador = VerificadorNoticiasAvancado()
    resultados = verificador.verificar_url(url)

    return resultados

# Exemplo de uso (para testar no mesmo arquivo)
def verificar_url(url_para):
    # Aqui, insira o código da função `verificar_url` ou chame o método da sua classe
    print(f"Analisando a URL: {url_para}")
    # Você pode utilizar o método da classe `VerificadorNoticiasAvancado`
    verificador = VerificadorNoticiasAvancado()
    resultado = verificador.verificar_noticia(url_para)
    print(resultado)  # Mostre o relatório de análise ou como você preferir formatar a saída

if __name__ == "__main__":
    url_para_analisar = "https://ultimosegundo.ig.com.br/mundo/2025-04-19/casa-branca--site-defende-teoria-de-que-coronavirus-vazou-de-laboratorio-na-china.html"
    verificar_url(url_para_analisar)


Overwriting verificacao_simples.py


In [7]:
%run verificacao_simples.py

NameError: name 'url_para' is not defined

In [6]:
%%writefile verificacao_simples.py
# verificacao_simples.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse
import hashlib
from collections import Counter
import time

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticiasAvancado:
    def __init__(self):
        print("Verificador de Notícias Avançado inicializado!")
        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]

        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe"
        ]

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        domain = parsed_url.netloc
        return domain

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0

        resultado["dominio"] = dominio
        return resultado

    def baixar_artigo(self, url):
        """Baixa e extrai o conteúdo básico de um artigo a partir da URL"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=10, allow_redirects=True)
            response.raise_for_status()  # Lança uma exceção para códigos de status HTTP ruins

            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""

            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)

                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords
            }

        except requests.exceptions.RequestException as e:
            print(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            print(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None

    def analisar_linguagem(self, texto):
        """Analisa o tom e a linguagem usada no texto."""
        resultados = {}

        # Análise de sensacionalismo com pesos
        count_sensacionalismo = 0
        palavras_encontradas = []

        for palavra, peso in self.palavras_sensacionalistas.items():
            ocorrencias = len(re.findall(r'\b' + palavra + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_sensacionalismo += ocorrencias * peso
                palavras_encontradas.append(f"{palavra} ({ocorrencias}x)")

        resultados["indice_sensacionalismo"] = count_sensacionalismo
        resultados["palavras_sensacionalistas"] = palavras_encontradas

        if count_sensacionalismo > 3:
            resultados["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista detectado"
        elif count_sensacionalismo > 1.5:
            resultados["alerta_sensacionalismo"] = "Linguagem potencialmente sensacionalista detectada"

        # Análise de incerteza
        count_incerteza = 0
        palavras_incerteza = []

        for palavra in self.palavras_incerteza:
            ocorrencias = len(re.findall(r'\b' + palavra + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_incerteza += ocorrencias
                palavras_incerteza.append(palavra)

        resultados["indice_incerteza"] = count_incerteza
        resultados["palavras_incerteza"] = palavras_incerteza

        if count_incerteza > 3:
            resultados["alerta_incerteza"] = "Alto uso de termos de incerteza detectado"

        # Análise de opinião
        count_opiniao = 0
        expressoes_opiniao = []

        for expressao in self.expressoes_opinativas:
            ocorrencias = len(re.findall(r'\b' + expressao + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_opiniao += ocorrencias
                expressoes_opiniao.append(expressao)

        resultados["indice_opiniao"] = count_opiniao
        resultados["expressoes_opiniao"] = expressoes_opiniao

        if count_opiniao > 2:
            resultados["alerta_opiniao"] = "Texto com forte caráter opinativo detectado"

        # Análise de exclamações (indicativo de sensacionalismo)
        exclamacoes = len(re.findall(r'!', texto))
        resultados["num_exclamacoes"] = exclamacoes

        if exclamacoes > 5:
            resultados["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        return resultados

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}

        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        return resultados

    def buscar_noticias_similares(self, titulo, keywords):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}

        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta
        except Exception as e:
            print(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)

        return resultados

    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto

    def analisar_credibilidade(self, artigo):
        """Análise combinada de credibilidade baseada em características do texto e fonte."""
        if not artigo or not artigo.get('conteudo'):
            return {"erro": "Artigo não disponível para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 1. Verificação de fonte/domínio
        info_fonte = self.verificar_fonte(artigo['url'])
        resultados.update(info_fonte)

        # 2. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 3. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 4. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 5. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)

        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 6. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 7. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 8. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', [])
        )
        resultados.update(resultados_similares)

        # 9. Determinar um nível de veracidade (ESTA LÓGICA PRECISA SER IMPLEMENTADA)
        nivel_de_veracidade_numerico = 3  # Exemplo: Artigo considerado inconclusivo
        resultados["nivel_veracidade_numerico"] = nivel_de_veracidade_numerico

        # Converter o nível numérico para texto
        resultados["nivel_veracidade"] = self.converter_nivel_para_texto(nivel_de_veracidade_numerico)

        return resultados

    def verificar_url(self, url):
        """Função principal para verificar a credibilidade de uma notícia a partir da URL."""
        resultados = {"url": url}

        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo:
            resultados["erro"] = "Não foi possível baixar o artigo."
            return resultados

        # 2. Analisar a credibilidade do artigo
        analise_credibilidade = self.analisar_credibilidade(artigo)
        resultados.update(analise_credibilidade)

        return resultados

def analisar_url(url):
    """Função para analisar uma URL específica."""
    verificador = VerificadorNoticiasAvancado()
    resultados = verificador.verificar_url(url)

    return resultados

# Exemplo de uso (para testar no mesmo arquivo)
if __name__ == "__main__":
    url_para

Overwriting verificacao_simples.py


In [5]:
%run verificacao_simples.py

NameError: name 'url_para' is not defined

In [4]:
%%writefile verificacao_simples.py
# verificacao_simples.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse
import hashlib
from collections import Counter
import time

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticiasAvancado:
    def __init__(self):
        print("Verificador de Notícias Avançado inicializado!")
        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]

        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe"
        ]

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        domain = parsed_url.netloc
        return domain

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0

        resultado["dominio"] = dominio
        return resultado

    def baixar_artigo(self, url):
        """Baixa e extrai o conteúdo básico de um artigo a partir da URL"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=10, allow_redirects=True)
            response.raise_for_status()  # Lança uma exceção para códigos de status HTTP ruins

            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""

            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)

                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': response.url,  # Use a URL final após redirecionamentos
                'imagens': imagens,
                'keywords': keywords
            }

        except requests.exceptions.RequestException as e:
            print(f"Erro ao baixar artigo ({url}): {e}")
            return None
        except Exception as e:
            print(f"Erro inesperado ao processar artigo ({url}): {e}")
            return None

    def analisar_linguagem(self, texto):
        """Analisa o tom e a linguagem usada no texto."""
        resultados = {}

        # Análise de sensacionalismo com pesos
        count_sensacionalismo = 0
        palavras_encontradas = []

        for palavra, peso in self.palavras_sensacionalistas.items():
            ocorrencias = len(re.findall(r'\b' + palavra + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_sensacionalismo += ocorrencias * peso
                palavras_encontradas.append(f"{palavra} ({ocorrencias}x)")

        resultados["indice_sensacionalismo"] = count_sensacionalismo
        resultados["palavras_sensacionalistas"] = palavras_encontradas

        if count_sensacionalismo > 3:
            resultados["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista detectado"
        elif count_sensacionalismo > 1.5:
            resultados["alerta_sensacionalismo"] = "Linguagem potencialmente sensacionalista detectada"

        # Análise de incerteza
        count_incerteza = 0
        palavras_incerteza = []

        for palavra in self.palavras_incerteza:
            ocorrencias = len(re.findall(r'\b' + palavra + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_incerteza += ocorrencias
                palavras_incerteza.append(palavra)

        resultados["indice_incerteza"] = count_incerteza
        resultados["palavras_incerteza"] = palavras_incerteza

        if count_incerteza > 3:
            resultados["alerta_incerteza"] = "Alto uso de termos de incerteza detectado"

        # Análise de opinião
        count_opiniao = 0
        expressoes_opiniao = []

        for expressao in self.expressoes_opinativas:
            ocorrencias = len(re.findall(r'\b' + expressao + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_opiniao += ocorrencias
                expressoes_opiniao.append(expressao)

        resultados["indice_opiniao"] = count_opiniao
        resultados["expressoes_opiniao"] = expressoes_opiniao

        if count_opiniao > 2:
            resultados["alerta_opiniao"] = "Texto com forte caráter opinativo detectado"

        # Análise de exclamações (indicativo de sensacionalismo)
        exclamacoes = len(re.findall(r'!', texto))
        resultados["num_exclamacoes"] = exclamacoes

        if exclamacoes > 5:
            resultados["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        return resultados

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}

        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        return resultados

    def buscar_noticias_similares(self, titulo, keywords):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}

        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta
        except Exception as e:
            print(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)

        return resultados

    def converter_nivel_para_texto(self, nivel):
        """Converte um nível numérico de veracidade para texto."""
        nivel_texto = {
            1: "CONFIRMADO",
            2: "PROVAVELMENTE VERDADEIRO",
            3: "INCONCLUSIVO",
            4: "PROVAVELMENTE FALSO",
            5: "FALSO"
        }.get(nivel, "INCONCLUSIVO")
        return nivel_texto

    def analisar_credibilidade(self, artigo):
        """Análise combinada de credibilidade baseada em características do texto e fonte."""
        if not artigo or not artigo.get('conteudo'):
            return {"erro": "Artigo não disponível para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 1. Verificação de fonte/domínio
        info_fonte = self.verificar_fonte(artigo['url'])
        resultados.update(info_fonte)

        # 2. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 3. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 4. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 5. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)

        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 6. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 7. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 8. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', [])
        )
        resultados.update(resultados_similares)

        # 9. Determinar um nível de veracidade (ESTA LÓGICA PRECISA SER IMPLEMENTADA)
        nivel_de_veracidade_numerico = 3  # Exemplo: Artigo considerado inconclusivo
        resultados["nivel_veracidade_numerico"] = nivel_de_veracidade_numerico

        # Converter o nível numérico para texto
        resultados["nivel_veracidade"] = self.converter_nivel_para_texto(nivel_de_veracidade_numerico)

        return resultados

    def verificar_url(self, url):
        """Função principal para verificar a credibilidade de uma notícia a partir da URL."""
        resultados = {"url": url}

        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo:
            resultados["erro"] = "Não foi possível baixar o artigo."
            return resultados

        # 2. Analisar a credibilidade do artigo
        analise_credibilidade = self.analisar_credibilidade(artigo)
        resultados.update(analise_credibilidade)

        return resultados

def analisar_url(url):
    """Função para analisar uma URL específica."""
    verificador = VerificadorNoticiasAvancado()
    resultados = verificador.verificar_url(url)

    return resultados

# Exemplo de uso (para testar no mesmo arquivo)
if __name__ == "__main__":
    url_para

Overwriting verificacao_simples.py


In [2]:
%run verificacao_simples.py

Verificador de Notícias Avançado inicializado!

--- Resultados da Análise ---
Título: N/A
URL: https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp
Fonte: N/A
Data de Publicação: N/A
Número de Palavras: N/A
Alertas: Nenhum
Erro: Artigo não disponível para análise
Verificador de Notícias Avançado inicializado!
Verificador de Notícias Avançado inicializado!
{
    "url": "https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp",
    "erro": "Artigo não disponível para análise"
}


In [1]:
%%writefile verificacao_simples.py


# verificacao_simples.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse
import hashlib
from collections import Counter
import time

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticiasAvancado:
    def __init__(self):
        print("Verificador de Notícias Avançado inicializado!")
        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]

        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe"
        ]

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        domain = parsed_url.netloc
        return domain

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0

        resultado["dominio"] = dominio
        return resultado

    def baixar_artigo(self, url):
        """Baixa e extrai o conteúdo básico de um artigo a partir da URL"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""

            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)

                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': url,
                'imagens': imagens,
                'keywords': keywords
            }

        except Exception as e:
            print(f"Erro ao baixar artigo: {e}")
            return None

    def analisar_linguagem(self, texto):
        """Analisa o tom e a linguagem usada no texto."""
        resultados = {}

        # Análise de sensacionalismo com pesos
        count_sensacionalismo = 0
        palavras_encontradas = []

        for palavra, peso in self.palavras_sensacionalistas.items():
            ocorrencias = len(re.findall(r'\b' + palavra + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_sensacionalismo += ocorrencias * peso
                palavras_encontradas.append(f"{palavra} ({ocorrencias}x)")

        resultados["indice_sensacionalismo"] = count_sensacionalismo
        resultados["palavras_sensacionalistas"] = palavras_encontradas

        if count_sensacionalismo > 3:
            resultados["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista detectado"
        elif count_sensacionalismo > 1.5:
            resultados["alerta_sensacionalismo"] = "Linguagem potencialmente sensacionalista detectada"

        # Análise de incerteza
        count_incerteza = 0
        palavras_incerteza = []

        for palavra in self.palavras_incerteza:
            ocorrencias = len(re.findall(r'\b' + palavra + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_incerteza += ocorrencias
                palavras_incerteza.append(palavra)

        resultados["indice_incerteza"] = count_incerteza
        resultados["palavras_incerteza"] = palavras_incerteza

        if count_incerteza > 3:
            resultados["alerta_incerteza"] = "Alto uso de termos de incerteza detectado"

        # Análise de opinião
        count_opiniao = 0
        expressoes_opiniao = []

        for expressao in self.expressoes_opinativas:
            ocorrencias = len(re.findall(r'\b' + expressao + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_opiniao += ocorrencias
                expressoes_opiniao.append(expressao)

        resultados["indice_opiniao"] = count_opiniao
        resultados["expressoes_opiniao"] = expressoes_opiniao

        if count_opiniao > 2:
            resultados["alerta_opiniao"] = "Texto com forte caráter opinativo detectado"

        # Análise de exclamações (indicativo de sensacionalismo)
        exclamacoes = len(re.findall(r'!', texto))
        resultados["num_exclamacoes"] = exclamacoes

        if exclamacoes > 5:
            resultados["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        return resultados

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}

        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        return resultados

    def buscar_noticias_similares(self, titulo, keywords):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}

        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta
        except Exception as e:
            print(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)

        return resultados

    def analisar_credibilidade(self, artigo):
        """Análise combinada de credibilidade baseada em características do texto e fonte."""
        if not artigo or not artigo.get('conteudo'):
            return {"erro": "Artigo não disponível para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 1. Verificação de fonte/domínio
        info_fonte = self.verificar_fonte(artigo['url'])
        resultados.update(info_fonte)

        # 2. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 3. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 4. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 5. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)

        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 6. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 7. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 8. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', [])
        )
        resultados.update(resultados_similares)

        return resultados

    def verificar_url(self, url):
        """Função principal para verificar a credibilidade de uma notícia a partir da URL."""
        resultados = {"url": url}

        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo:
            resultados["erro"] = "Não foi possível baixar o artigo."
            return resultados

        # 2. Analisar a credibilidade do artigo
        analise_credibilidade = self.analisar_credibilidade(artigo)
        resultados.update(analise_credibilidade)

        return resultados

def analisar_url(url):
    """Função para analisar uma URL específica."""
    verificador = VerificadorNoticiasAvancado()
    resultados = verificador.verificar_url(url)

    return resultados

# Exemplo de uso (para testar no mesmo arquivo)
if __name__ == "__main__":
    url_para_analisar = "https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp"

    analise = analisar_url(url_para_analisar)

    # Imprimir resultados de forma organizada (exemplo)
    print("\n--- Resultados da Análise ---")
    print(f"Título: {analise.get('titulo', 'N/A')}")
    print(f"URL: {analise.get('url', 'N/A')}")
    print(f"Fonte: {analise.get('status_fonte', 'N/A')}")
    print(f"Data de Publicação: {analise.get('data_publicacao', 'N/A')}")
    print(f"Número de Palavras: {analise.get('num_palavras', 'N/A')}")
    print(f"Alertas: {analise.get('alertas', 'Nenhum')}")  # Adapte conforme a estrutura real

    if "erro" in analise:
        print(f"Erro: {analise['erro']}")

def verificar_url(url):
    """Função para analisar uma URL."""
    return analisar_url(url)

print("Verificador de Notícias Avançado inicializado!")

# Exemplo de uso da função verificar_url
url_para_analisar = "https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp"
resultados = verificar_url(url_para_analisar)
print(json.dumps(resultados, indent=4, ensure_ascii=False))

Writing verificacao_simples.py


In [12]:
%%writefile verificacao_simples.py

# verificacao_simples.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse
import hashlib
from collections import Counter
import time

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticiasAvancado:
    def __init__(self):
        print("Verificador de Notícias Avançado inicializado!")
        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]

        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe"
        ]

    def extrair_dominio(self, self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        domain = parsed_url.netloc
        return domain

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0

        resultado["dominio"] = dominio
        return resultado

    def baixar_artigo(self, url):
        """Baixa e extrai o conteúdo básico de um artigo a partir da URL"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""

            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)

                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': url,
                'imagens': imagens,
                'keywords': keywords
            }

        except Exception as e:
            print(f"Erro ao baixar artigo: {e}")
            return None

    def analisar_linguagem(self, texto):
        """Analisa o tom e a linguagem usada no texto."""
        resultados = {}

        # Análise de sensacionalismo com pesos
        count_sensacionalismo = 0
        palavras_encontradas = []

        for palavra, peso in self.palavras_sensacionalistas.items():
            ocorrencias = len(re.findall(r'\b' + palavra + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_sensacionalismo += ocorrencias * peso
                palavras_encontradas.append(f"{palavra} ({ocorrencias}x)")

        resultados["indice_sensacionalismo"] = count_sensacionalismo
        resultados["palavras_sensacionalistas"] = palavras_encontradas

        if count_sensacionalismo > 3:
            resultados["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista detectado"
        elif count_sensacionalismo > 1.5:
            resultados["alerta_sensacionalismo"] = "Linguagem potencialmente sensacionalista detectada"

        # Análise de incerteza
        count_incerteza = 0
        palavras_incerteza = []

        for palavra in self.palavras_incerteza:
            ocorrencias = len(re.findall(r'\b' + palavra + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_incerteza += ocorrencias
                palavras_incerteza.append(palavra)

        resultados["indice_incerteza"] = count_incerteza
        resultados["palavras_incerteza"] = palavras_incerteza

        if count_incerteza > 3:
            resultados["alerta_incerteza"] = "Alto uso de termos de incerteza detectado"

        # Análise de opinião
        count_opiniao = 0
        expressoes_opiniao = []

        for expressao in self.expressoes_opinativas:
            ocorrencias = len(re.findall(r'\b' + expressao + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_opiniao += ocorrencias
                expressoes_opiniao.append(expressao)

        resultados["indice_opiniao"] = count_opiniao
        resultados["expressoes_opiniao"] = expressoes_opiniao

        if count_opiniao > 2:
            resultados["alerta_opiniao"] = "Texto com forte caráter opinativo detectado"

        # Análise de exclamações (indicativo de sensacionalismo)
        exclamacoes = len(re.findall(r'!', texto))
        resultados["num_exclamacoes"] = exclamacoes

        if exclamacoes > 5:
            resultados["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        return resultados

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}

        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        return resultados

    def buscar_noticias_similares(self, titulo, keywords):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}

        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta
        except Exception as e:
            print(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)

        return resultados

    def analisar_credibilidade(self, artigo):
        """Análise combinada de credibilidade baseada em características do texto e fonte."""
        if not artigo or not artigo.get('conteudo'):
            return {"erro": "Artigo não disponível para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 1. Verificação de fonte/domínio
        info_fonte = self.verificar_fonte(artigo['url'])
        resultados.update(info_fonte)

        # 2. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 3. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 4. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 5. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)

        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 6. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 7. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 8. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', [])
        )
        resultados.update(resultados_similares)

        return resultados

    def verificar_url(self, url):
        """Função principal para verificar a credibilidade de uma notícia a partir da URL."""
        resultados = {"url": url}

        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo:
            resultados["erro"] = "Não foi possível baixar o artigo."
            return resultados

        # 2. Analisar a credibilidade do artigo
        analise_credibilidade = self.analisar_credibilidade(artigo)
        resultados.update(analise_credibilidade)

        return resultados

def analisar_url(url):
    """Função para analisar uma URL específica."""
    verificador = VerificadorNoticiasAvancado()
    resultados = verificador.verificar_url(url)

    return resultados

# Exemplo de uso (para testar no mesmo arquivo)
if __name__ == "__main__":
    url_para_analisar = "https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp"

    analise = analisar_url(url_para_analisar)

    # Imprimir resultados de forma organizada (exemplo)
    print("\n--- Resultados da Análise ---")
    print(f"Título: {analise.get('titulo', 'N/A')}")
    print(f"URL: {analise.get('url', 'N/A')}")
    print(f"Fonte: {analise.get('status_fonte', 'N/A')}")
    print(f"Data de Publicação: {analise.get('data_publicacao', 'N/A')}")
    print(f"Número de Palavras: {analise.get('num_palavras', 'N/A')}")
    print(f"Alertas: {analise.get('alertas', 'Nenhum')}")  # Adapte conforme a estrutura real

    if "erro" in analise:
        print(f"Erro: {analise['erro']}")

def verificar_url(url):
    """Função para analisar uma URL."""
    return analisar_url(url)

print("Verificador de Notícias Avançado inicializado!")

# Exemplo de uso da função verificar_url
url_para_analisar = "https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp"
resultados = verificar_url(url_para_analisar)
print(json.dumps(resultados, indent=4, ensure_ascii=False))


Overwriting verificacao_simples.py


In [11]:
python verificacao_simples.py

SyntaxError: invalid syntax (<ipython-input-11-212b1e80bbf7>, line 1)

In [10]:
%run verificacao_simples.py

# verificacao_simples.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse
import hashlib
from collections import Counter
import time

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticiasAvancado:
    def __init__(self):
        print("Verificador de Notícias Avançado inicializado!")
        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]

        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe"
        ]

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        domain = parsed_url.netloc
        return domain

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0

        resultado["dominio"] = dominio
        return resultado

    def baixar_artigo(self, url):
        """Baixa e extrai o conteúdo básico de um artigo a partir da URL"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""

            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)

                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(author_tag)
                else:
                    elements = soup.find_all(author_tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': url,
                'imagens': imagens,
                'keywords': keywords
            }

        except Exception as e:
            print(f"Erro ao baixar artigo: {e}")
            return None

    def analisar_linguagem(self, texto):
        """Analisa o tom e a linguagem usada no texto."""
        resultados = {}

        # Análise de sensacionalismo com pesos
        count_sensacionalismo = 0
        palavras_encontradas = []

        for palavra, peso in self.palavras_sensacionalistas.items():
            ocorrencias = len(re.findall(r'\b' + palavra + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_sensacionalismo += ocorrencias * peso
                palavras_encontradas.append(f"{palavra} ({ocorrencias}x)")

        resultados["indice_sensacionalismo"] = count_sensacionalismo
        resultados["palavras_sensacionalistas"] = palavras_encontradas

        if count_sensacionalismo > 3:
            resultados["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista detectado"
        elif count_sensacionalismo > 1.5:
            resultados["alerta_sensacionalismo"] = "Linguagem potencialmente sensacionalista detectada"

        # Análise de incerteza
        count_incerteza = 0
        palavras_incerteza = []

        for palavra in self.palavras_incerteza:
            ocorrencias = len(re.findall(r'\b' + palavra + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_incerteza += ocorrencias
                palavras_incerteza.append(palavra)

        resultados["indice_incerteza"] = count_incerteza
        resultados["palavras_incerteza"] = palavras_incerteza

        if count_incerteza > 3:
            resultados["alerta_incerteza"] = "Alto uso de termos de incerteza detectado"

        # Análise de opinião
        count_opiniao = 0
        expressoes_opiniao = []

        for expressao in self.expressoes_opinativas:
            ocorrencias = len(re.findall(r'\b' + expressao + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_opiniao += ocorrencias
                expressoes_opiniao.append(expressao)

        resultados["indice_opiniao"] = count_opiniao
        resultados["expressoes_opiniao"] = expressoes_opiniao

        if count_opiniao > 2:
            resultados["alerta_opiniao"] = "Texto com forte caráter opinativo detectado"

        # Análise de exclamações (indicativo de sensacionalismo)
        exclamacoes = len(re.findall(r'!', texto))
        resultados["num_exclamacoes"] = exclamacoes

        if exclamacoes > 5:
            resultados["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        return resultados

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}

        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        return resultados

    def buscar_noticias_similares(self, titulo, keywords):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}

        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta
        except Exception as e:
            print(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)

        return resultados

    def analisar_credibilidade(self, artigo):
        """Análise combinada de credibilidade baseada em características do texto e fonte."""
        if not artigo or not artigo.get('conteudo'):
            return {"erro": "Artigo não disponível para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 1. Verificação de fonte/domínio
        info_fonte = self.verificar_fonte(artigo['url'])
        resultados.update(info_fonte)

        # 2. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 3. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 4. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 5. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)

        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 6. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 7. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 8. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', [])
        )
        resultados.update(resultados_similares)

        return resultados

    def verificar_url(self, url):
        """Função principal para verificar a credibilidade de uma notícia a partir da URL."""
        resultados = {"url": url}

        # 1. Baixar o artigo
        artigo = self.baixar_artigo(url)
        if not artigo:
            resultados["erro"] = "Não foi possível baixar o artigo."
            return resultados

        # 2. Analisar a credibilidade do artigo
        analise_credibilidade = self.analisar_credibilidade(artigo)
        resultados.update(analise_credibilidade)

        return resultados

def analisar_url(url):
    """Função para analisar uma URL específica."""
    verificador = VerificadorNoticiasAvancado()
    resultados = verificador.verificar_url(url)

    return resultados

# Exemplo de uso (para testar no mesmo arquivo)
if __name__ == "__main__":
    url_para_analisar = "https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp"

    analise = analisar_url(url_para_analisar)

    # Imprimir resultados de forma organizada (exemplo)
    print("\n--- Resultados da Análise ---")
    print(f"Título: {analise.get('titulo', 'N/A')}")
    print(f"URL: {analise.get('url', 'N/A')}")
    print(f"Fonte: {analise.get('status_fonte', 'N/A')}")
    print(f"Data de Publicação: {analise.get('data_publicacao', 'N/A')}")
    print(f"Número de Palavras: {analise.get('num_palavras', 'N/A')}")
    print(f"Alertas: {analise.get('alertas', 'Nenhum')}")  # Adapte conforme a estrutura real

    if "erro" in analise:
        print(f"Erro: {analise['erro']}")

def verificar_url(url):
    """Função para analisar uma URL."""
    return analisar_url(url)

print("Verificador de Notícias Avançado inicializado!")

# Exemplo de uso da função verificar_url
url_para_analisar = "https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp"
resultados = verificar_url(url_para_analisar)
print(json.dumps(resultados, indent=4, ensure_ascii=False))


Verificador de notícias avançado - use a função verificar_url(sua_url) para analisar uma notícia
Verificador de Notícias Avançado inicializado!

--- Resultados da Análise ---
Título: N/A
URL: https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp
Fonte: N/A
Data de Publicação: N/A
Número de Palavras: N/A
Alertas: Nenhum
Erro: Artigo não disponível para análise
Verificador de Notícias Avançado inicializado!
Verificador de Notícias Avançado inicializado!
{
    "url": "https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp",
    "erro": "Artigo não disponível para análise"
}


In [9]:
python verificar_noticias.py "https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp"

SyntaxError: invalid syntax (<ipython-input-9-2cd42356226d>, line 1)

In [8]:
pip install transformers newspaper3k beautifulsoup4 requests

In [7]:
%run verificacao_simples.py
verificar_url("https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp")


Verificador de notícias avançado - use a função verificar_url(sua_url) para analisar uma notícia
Verificador de Notícias Avançado inicializado!
Analisando URL: https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp

RELATÓRIO DE VERIFICAÇÃO DE NOTÍCIA
URL: https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp
Domínio: N/A
Status da fonte: N/A
--------------------------------------------------
Título: N/A
Data de publicação: N/A
Autores: N/A
Número de palavras: 0
--------------------------------------------------
MÉTRICAS:
• Citações: 0
• URLs referenciadas: 0
• Índice de sensacionalismo: 0
• Índice de incerteza: 0
• Índice opinativo: 0
• Pontos de exclamação: 0
• Imagens: 0
• Artigos similares 

{'erro': 'Artigo não disponível para análise',
 'pontuacao_credibilidade': 70,
 'avaliacao': 'Boa credibilidade'}

In [6]:
%run verificacao_simples.py

Verificador de notícias avançado - use a função verificar_url(sua_url) para analisar uma notícia


In [5]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [3]:
pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [1]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [34]:
!pip install functions-framework

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.3 MB/s eta 0:00:00


In [33]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [32]:
!pip install google-cloud-functions-framework

ERROR: Could not find a version that satisfies the requirement google-cloud-functions-framework (from versions: none)
ERROR: No matching distribution found for google-cloud-functions-framework


In [31]:
pip install google-cloud-functions-framework

ERROR: Could not find a version that satisfies the requirement google-cloud-functions-framework (from versions: none)
ERROR: No matching distribution found for google-cloud-functions-framework


In [30]:
!python main.py


Traceback (most recent call last):
  File "/content/main.py", line 2, in <module>
    import functions_framework
ModuleNotFoundError: No module named 'functions_framework'


In [29]:
python main.py

SyntaxError: invalid syntax (<ipython-input-29-a5b85dd88f47>, line 1)

In [28]:
%%writefile verificacao_avancada.py
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse, urljoin # Added urljoin
import hashlib
from collections import Counter
import time
import logging # Added logging

# Desabilitar avisos SSL (Use com cautela em produção)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Configurar logging básico
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class VerificadorNoticiasAvancado:
    def __init__(self):
        logging.info("Verificador de Notícias Avançado inicializado!")
        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]

        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe"
        ]

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        try:
            parsed_url = urlparse(url)
            domain = parsed_url.netloc.replace('www.', '') # Remove www. for consistency
            return domain
        except Exception as e:
            logging.error(f"Erro ao extrair domínio de {url}: {e}")
            return None

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {"dominio": dominio} # Initialize with domain

        if not dominio:
             resultado["status_fonte"] = "erro_dominio"
             resultado["pontos_fonte"] = 0
             resultado["alerta_fonte"] = "Não foi possível extrair o domínio da URL."
             return resultado

        # Verificar se o domínio está nas listas
        if any(dominio == fonte for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
        elif any(dominio == fonte for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade."
        else:
            # Check subdomains of reliable sources (e.g., news.google.com)
            is_subdomain_confiavel = False
            for fonte_confiavel in self.fontes_confiaveis:
                 if dominio.endswith('.' + fonte_confiavel):
                      is_subdomain_confiavel = True
                      break
            if is_subdomain_confiavel:
                 resultado["status_fonte"] = "confiável (subdomínio)"
                 resultado["pontos_fonte"] = 15 # Slightly less than main domain
            else:
                 resultado["status_fonte"] = "não categorizada"
                 resultado["pontos_fonte"] = 0

        return resultado

    def baixar_artigo(self, url):
        """Baixa e extrai o conteúdo básico de um artigo a partir da URL"""
        logging.info(f"Tentando baixar artigo de: {url}")
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
                'Accept-Language': 'en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7'
            }
            # Usar verify=False é um risco de segurança, mas necessário para alguns sites com SSL mal configurado.
            # Idealmente, configurar certifi ou similar.
            response = requests.get(url, headers=headers, verify=False, timeout=15) # Increased timeout
            response.raise_for_status() # Levanta exceção para erros HTTP (4xx ou 5xx)

            # Detectar codificação para evitar problemas com caracteres especiais
            response.encoding = response.apparent_encoding
            html_content = response.text

            soup = BeautifulSoup(html_content, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content_tags = ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text', '.story-content', '.main-content']
            content = ""
            found_content = False

            for tag_selector in content_tags:
                elements = soup.select(tag_selector)
                if elements:
                    logging.info(f"Encontrado conteúdo com seletor: {tag_selector}")
                    for element in elements:
                        # Remove elementos indesejados como scripts, styles, menus, footers
                        for unwanted in element.select('script, style, nav, footer, .menu, .footer, .sidebar, .ad, .advertisement, .related-links'):
                            unwanted.decompose()
                        content += element.get_text(separator=' ', strip=True) + " "
                    if len(content.split()) > 50: # Consider content found if it has reasonable length
                         found_content = True
                         break # Stop searching if good content is found

            # Fallback: Pega todos os parágrafos se a busca específica falhar
            if not found_content or len(content.split()) < 50:
                logging.info("Seletores específicos falharam ou conteúdo curto, tentando todos <p>.")
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs if len(p.get_text(strip=True)) > 20]) # Filter short paragraphs

            if not content:
                 logging.warning(f"Não foi possível extrair o conteúdo principal de {url}")
                 # Fallback ainda mais geral: pegar o body todo (pode ser ruidoso)
                 body = soup.find('body')
                 if body:
                      for unwanted in body.select('script, style, nav, footer, header, .menu, .footer, .sidebar, .ad, .advertisement'):
                           unwanted.decompose()
                      content = body.get_text(separator=' ', strip=True)

            # Tenta encontrar a data
            data = None
            date_selectors = [
                'meta[property="article:published_time"]',
                'meta[property="og:published_time"]',
                'meta[name="publication_date"]',
                'meta[name="date"]',
                'time[datetime]'
            ]
            for selector in date_selectors:
                 element = soup.select_one(selector)
                 if element:
                      data = element.get('content') or element.get('datetime') or element.text
                      if data:
                           logging.info(f"Data encontrada: {data}")
                           break

            # Procura por elementos que possam conter autores
            autores = []
            author_selectors = [
                'meta[name="author"]',
                '.author', '.byline', '[rel="author"]', '.autor', '.nome-autor'
                'a[href*="/author/"]', 'span[class*="author"]'
            ]
            processed_authors = set() # Evitar duplicação e processamento excessivo

            for selector in author_selectors:
                 elements = soup.select(selector)
                 for element in elements:
                      autor_text = None
                      if element.name == 'meta':
                           autor_text = element.get('content')
                      else:
                           autor_text = element.get_text(strip=True)

                      if autor_text and len(autor_text) > 2 and len(autor_text) < 100: # Filtros básicos
                           # Limpeza adicional
                           autor_text = re.sub(r'(?i)por\s+|by\s+', '', autor_text).strip()
                           if autor_text and autor_text.lower() not in processed_authors:
                                autores.append(autor_text)
                                processed_authors.add(autor_text.lower())


            # Extrair imagens com URLs absolutas
            imagens = []
            processed_img_urls = set()
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'): # Ignorar imagens embutidas
                    # Garantir URL completa
                    absolute_src = urljoin(url, src) # Constrói URL absoluta
                    if absolute_src not in processed_img_urls:
                         imagens.append({
                             'url': absolute_src,
                             'alt': alt.strip() if alt else ''
                         })
                         processed_img_urls.add(absolute_src)

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': re.compile(r'keywords', re.I)})
            if meta_keywords and meta_keywords.get('content'):
                keywords = [k.strip() for k in meta_keywords['content'].split(',') if k.strip()]

            logging.info(f"Artigo baixado: Título='{titulo[:50]}...', Palavras={len(content.split())}, Imagens={len(imagens)}")
            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': list(autores), # Converter set de volta para lista se necessário
                'url': url,
                'imagens': imagens,
                'keywords': keywords,
                'status': 'sucesso' # Adicionar status
            }

        except requests.exceptions.HTTPError as http_err:
            logging.error(f"Erro HTTP ao baixar {url}: {http_err} - Status: {http_err.response.status_code}")
            return {'status': 'erro_http', 'mensagem': str(http_err), 'status_code': http_err.response.status_code}
        except requests.exceptions.ConnectionError as conn_err:
             logging.error(f"Erro de conexão ao baixar {url}: {conn_err}")
             return {'status': 'erro_conexao', 'mensagem': str(conn_err)}
        except requests.exceptions.Timeout as timeout_err:
            logging.error(f"Timeout ao baixar {url}: {timeout_err}")
            return {'status': 'erro_timeout', 'mensagem': str(timeout_err)}
        except requests.exceptions.RequestException as req_err:
            logging.error(f"Erro genérico de request ao baixar {url}: {req_err}")
            return {'status': 'erro_request', 'mensagem': str(req_err)}
        except Exception as e:
            logging.error(f"Erro inesperado ao processar {url}: {e}", exc_info=True) # Log traceback
            return {'status': 'erro_inesperado', 'mensagem': str(e)}

    def analisar_linguagem(self, texto):
        """Analisa o tom e a linguagem usada no texto."""
        resultados = {}
        texto_lower = texto.lower() # Converter para minúsculas uma vez

        # Análise de sensacionalismo com pesos
        count_sensacionalismo = 0
        palavras_encontradas_sensacionalismo = []
        for palavra, peso in self.palavras_sensacionalistas.items():
            # Usar regex para encontrar palavras inteiras (evitar substrings)
            ocorrencias = len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto_lower))
            if ocorrencias > 0:
                count_sensacionalismo += ocorrencias * peso
                palavras_encontradas_sensacionalismo.append(f"{palavra} ({ocorrencias}x)")

        resultados["indice_sensacionalismo"] = round(count_sensacionalismo, 2)
        resultados["palavras_sensacionalistas"] = palavras_encontradas_sensacionalismo
        if count_sensacionalismo > 5: # Ajustar limiar
            resultados["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista detectado."
        elif count_sensacionalismo > 2: # Ajustar limiar
            resultados["alerta_sensacionalismo"] = "Linguagem potencialmente sensacionalista detectada."

        # Análise de incerteza
        count_incerteza = 0
        palavras_encontradas_incerteza = []
        for palavra in self.palavras_incerteza:
            ocorrencias = len(re.findall(r'\b' + re.escape(palavra) + r'\b', texto_lower))
            if ocorrencias > 0:
                count_incerteza += ocorrencias
                palavras_encontradas_incerteza.append(palavra)

        resultados["indice_incerteza"] = count_incerteza
        resultados["palavras_incerteza"] = list(set(palavras_encontradas_incerteza)) # Lista única
        if count_incerteza > 4: # Ajustar limiar
            resultados["alerta_incerteza"] = "Alto uso de termos de incerteza detectado."

        # Análise de opinião
        count_opiniao = 0
        expressoes_encontradas_opiniao = []
        for expressao in self.expressoes_opinativas:
            ocorrencias = len(re.findall(r'\b' + re.escape(expressao) + r'\b', texto_lower))
            if ocorrencias > 0:
                count_opiniao += ocorrencias
                expressoes_encontradas_opiniao.append(expressao)

        resultados["indice_opiniao"] = count_opiniao
        resultados["expressoes_opiniao"] = list(set(expressoes_encontradas_opiniao)) # Lista única
        if count_opiniao > 2: # Ajustar limiar
            resultados["alerta_opiniao"] = "Texto com possível caráter opinativo detectado."

        # Análise de exclamações
        exclamacoes = texto.count('!')
        resultados["num_exclamacoes"] = exclamacoes
        if exclamacoes > 5:
            resultados["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação."

        return resultados

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}
        num_imagens = len(imagens)
        resultados["num_imagens"] = num_imagens

        if num_imagens == 0:
            # Não necessariamente um alerta negativo, alguns artigos não têm imagens.
            # resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo."
            return resultados

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao > 0 and imagens_sem_descricao == num_imagens:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição (alt text)."
        elif imagens_sem_descricao > num_imagens / 2:
             resultados["alerta_imagens_descricao"] = "Muitas imagens sem descrição (alt text)."

        # Calcular hash básico das URLs de imagens (pode ser usado para verificação reversa)
        # hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        # resultados["hashes_imagens"] = hashes_urls

        return resultados

    def buscar_noticias_similares(self, titulo, keywords):
        """Busca por notícias similares na web (SIMULAÇÃO)."""
        # --- IMPLEMENTAÇÃO SIMULADA ---
        # Em uma aplicação real, você usaria uma API de busca (Google News, Bing News, etc.)
        # ou um motor de busca customizado.
        logging.info(f"Simulando busca por notícias similares para: '{titulo[:50]}...'")
        resultados = {}
        try:
            # Simular tempo de busca
            time.sleep(0.5)

            # Simular resultados baseados em keywords ou título
            num_similares = 0
            if keywords:
                 num_similares = len(keywords) * 2 # Exemplo simples
            else:
                 num_similares = len(titulo.split()) // 5 # Exemplo simples

            num_similares = min(num_similares, 10) # Limitar simulação

            resultados["artigos_similares_encontrados"] = num_similares # Nome mais claro
            if num_similares > 5:
                 resultados["similaridade_detectada"] = "alta"
            elif num_similares > 2:
                 resultados["similaridade_detectada"] = "média"
            else:
                 resultados["similaridade_detectada"] = "baixa"

        except Exception as e:
            logging.error(f"Erro na simulação de busca de notícias similares: {e}")
            resultados["erro_busca_similar"] = str(e)

        return resultados
        # --- FIM DA SIMULAÇÃO ---

    def analisar_credibilidade(self, artigo):
        """Análise combinada de credibilidade baseada em características do texto e fonte."""
        # Verifica se o artigo foi baixado corretamente
        if not artigo or artigo.get('status') != 'sucesso' or not artigo.get('conteudo'):
            logging.warning("Artigo não disponível ou incompleto para análise de credibilidade.")
            # Retorna a informação de erro que veio de baixar_artigo
            return artigo if artigo else {"status": "erro", "mensagem": "Artigo não fornecido para análise."}

        resultados = {}
        conteudo = artigo['conteudo']
        url = artigo['url']

        # 1. Verificação de fonte/domínio
        info_fonte = self.verificar_fonte(url)
        resultados.update(info_fonte)

        # 2. Copiar informações básicas do artigo
        resultados["titulo"] = artigo.get('titulo', 'N/A')
        resultados["data_publicacao"] = artigo.get('data', 'N/A')
        resultados["autores"] = artigo.get('autores', [])
        resultados["url"] = url

        # 3. Análise básica do conteúdo
        palavras = conteudo.split()
        num_palavras = len(palavras)
        resultados["num_palavras"] = num_palavras
        if num_palavras < 150: # Aumentar limiar para considerar notícia
            resultados["alerta_tamanho"] = f"Texto curto ({num_palavras} palavras), pode ser apenas uma nota ou resumo."

        # 4. Análise detalhada da linguagem
        if num_palavras > 10: # Só analisa linguagem se houver texto suficiente
            analise_linguagem = self.analisar_linguagem(conteudo)
            resultados.update(analise_linguagem)

        # 5. Verificação de citações diretas (entre aspas)
        # Regex aprimorado para capturar aspas corretamente
        citacoes = re.findall(r'[“"]([^"”]+)[”"]', conteudo)
        resultados["num_citacoes"] = len(citacoes)
        # Citações não são obrigatórias, remover alerta negativo por falta delas.
        # if len(citacoes) == 0 and num_palavras > 300: # Só alerta em textos longos
        #     resultados["info_fontes"] = "Nenhuma citação direta (entre aspas) encontrada."

        # 6. Verificação de URLs externos (links)
        # Regex mais robusto para URLs
        urls_externos = re.findall(r'https?://[^\s/$.?#].[^\s]*', conteudo)
        # Filtrar links para o mesmo domínio
        dominio_artigo = self.extrair_dominio(url)
        links_externos_validos = [link for link in urls_externos if dominio_artigo not in urlparse(link).netloc]
        resultados["num_links_externos"] = len(links_externos_validos)

        # 7. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 8. Verificação cruzada com outras notícias (Simulação)
        resultados_similares = self.buscar_noticias_similares(
            artigo.get('titulo', ''),
            artigo.get('keywords', [])
        )
        resultados.update(resultados_similares)

        # Marcar que a análise de credibilidade foi feita
        resultados['status_analise'] = 'concluida'

        return resultados

    def calcular_pontuacao_credibilidade(self, resultados_analise):
        """Calcula a pontuação final de credibilidade baseada nos resultados da análise."""

        # Verifica se a análise foi concluída com sucesso
        if resultados_analise.get('status_analise') != 'concluida':
             # Se houve erro antes (ex: download), a pontuação não é calculada
             return {"pontuacao_credibilidade": 0, "avaliacao": "Não foi possível analisar"}

        pontuacao = 50  # Base inicial neutra (escala 0-100)

        # Ajustes baseados na fonte
        pontuacao += resultados_analise.get("pontos_fonte", 0)

        # Fatores que reduzem a pontuação
        if "alerta_tamanho" in resultados_analise:
            pontuacao -= 10
        if "alerta_sensacionalismo" in resultados_analise:
            if "Alto índice" in resultados_analise["alerta_sensacionalismo"]:
                pontuacao -= 25 # Penalidade maior para alto sensacionalismo
            else:
                pontuacao -= 15
        # if "alerta_fontes" in resultados_analise: # Removido alerta negativo de citações
        #     pontuacao -= 5
        if "alerta_opiniao" in resultados_analise:
            pontuacao -= 15
        if "alerta_incerteza" in resultados_analise:
             # Penalidade proporcional ao índice de incerteza
            pontuacao -= min(resultados_analise.get("indice_incerteza", 0) * 2, 15) # Limita a penalidade
        if "alerta_exclamacoes" in resultados_analise:
            pontuacao -= 5
        if "alerta_imagens_descricao" in resultados_analise:
             # Penalidade menor se houver imagens, mas sem descrição
             if "Nenhuma das imagens" in resultados_analise["alerta_imagens_descricao"]:
                  pontuacao -= 5
             else:
                  pontuacao -= 2

        # Fatores que aumentam a pontuação
        num_citacoes = resultados_analise.get("num_citacoes", 0)
        if num_citacoes > 3:
            pontuacao += 10
        elif num_citacoes > 0:
            pontuacao += 5

        num_links_externos = resultados_analise.get("num_links_externos", 0)
        if num_links_externos > 3:
            pontuacao += 10
        elif num_links_externos > 0:
            pontuacao += 5

        num_imagens = resultados_analise.get("num_imagens", 0)
        if num_imagens >= 1 and resultados_analise.get("imagens_sem_descricao", num_imagens) < num_imagens:
             # Bonus se tem imagens E pelo menos uma tem descrição
             pontuacao += 5

        # Bônus por artigos similares (indicativo que a notícia é coberta por outros)
        artigos_similares = resultados_analise.get("artigos_similares_encontrados", 0)
        if artigos_similares > 5:
             pontuacao += 10
        elif artigos_similares > 2:
             pontuacao += 5

        # Bônus por ter autores identificados
        if resultados_analise.get("autores"):
             pontuacao += 5

        # Bônus por ter data de publicação
        if resultados_analise.get("data_publicacao") != 'N/A':
             pontuacao += 3


        # Ajustar pontuação final para ficar entre 0 e 100
        pontuacao = max(0, min(100, pontuacao))
        resultados_analise["pontuacao_credibilidade"] = int(round(pontuacao)) # Arredonda para inteiro

        # Avaliação qualitativa baseada na pontuação
        if pontuacao >= 85:
            resultados_analise["avaliacao"] = "Credibilidade Muito Alta"
        elif pontuacao >= 70:
            resultados_analise["avaliacao"] = "Credibilidade Alta"
        elif pontuacao >= 50:
            resultados_analise["avaliacao"] = "Credibilidade Moderada"
        elif pontuacao >= 30:
            resultados_analise["avaliacao"] = "Credibilidade Baixa"
        else:
            resultados_analise["avaliacao"] = "Credibilidade Muito Baixa"

        return resultados_analise


    def verificar_noticia(self, url):
        """Função principal para verificar uma notícia: baixa, analisa e pontua."""
        logging.info(f"Iniciando verificação completa para: {url}")
        tempo_inicio = time.time()

        # 1. Baixar e extrair informações do artigo
        artigo = self.baixar_artigo(url)

        # Se baixar falhar, retorna o erro imediatamente
        if not artigo or artigo.get('status') != 'sucesso':
            logging.error(f"Falha ao baixar ou processar artigo de {url}.")
            # Retornar um dicionário consistente mesmo em caso de erro
            return {
                "url": url,
                "status_geral": "erro_ao_baixar",
                "mensagem": artigo.get("mensagem", "Erro desconhecido ao baixar artigo."),
                "pontuacao_credibilidade": 0,
                "avaliacao": "Não foi possível analisar",
                "tempo_analise_seg": round(time.time() - tempo_inicio, 2)
            }

        # 2. Analisar a credibilidade com base nas informações extraídas
        resultados_analise = self.analisar_credibilidade(artigo)

        # Se a análise de credibilidade falhar (improvável se o download funcionou, mas por segurança)
        if resultados_analise.get('status_analise') != 'concluida':
             logging.error(f"Falha na etapa de análise de credibilidade para {url}.")
             return {
                 **resultados_analise, # Retorna o que foi possível analisar
                 "url": url,
                 "status_geral": "erro_na_analise",
                 "pontuacao_credibilidade": 0,
                 "avaliacao": "Não foi possível analisar completamente",
                 "tempo_analise_seg": round(time.time() - tempo_inicio, 2)
             }

        # 3. Calcular a pontuação e avaliação final
        resultados_finais = self.calcular_pontuacao_credibilidade(resultados_analise)

        # Adicionar tempo de execução e status geral
        resultados_finais["tempo_analise_seg"] = round(time.time() - tempo_inicio, 2)
        resultados_finais["status_geral"] = "analise_completa"

        logging.info(f"Análise de {url} concluída em {resultados_finais['tempo_analise_seg']:.2f}s. Pontuação: {resultados_finais['pontuacao_credibilidade']}/100")

        return resultados_finais

# --- Helper para formatação de saída (usado no main.py para testes locais) ---
def formatar_resultado_texto(resultado):
    """Formata o dicionário de resultados em um texto legível."""
    if not resultado:
        return "Erro: Resultado da análise está vazio."

    output = []
    output.append("="*50)
    output.append(f"RELATÓRIO DE VERIFICAÇÃO DE NOTÍCIA")
    output.append("="*50)
    output.append(f"URL: {resultado.get('url', 'N/A')}")

    if resultado.get('status_geral', '').startswith('erro'):
        output.append("-" * 50)
        output.append(f"⚠️ ERRO NA ANÁLISE: {resultado.get('status_geral')}")
        output.append(f"Mensagem: {resultado.get('mensagem', 'Detalhe não disponível')}")
        if 'status_code' in resultado:
             output.append(f"Status HTTP: {resultado.get('status_code')}")
        output.append("="*50)
        return "\n".join(output)

    # Se a análise foi completa
    output.append(f"Domínio: {resultado.get('dominio', 'N/A')} ({resultado.get('status_fonte', 'N/A')})")
    output.append("-" * 50)

    output.append(f"Título: {resultado.get('titulo', 'N/A')}")
    output.append(f"Data de publicação: {resultado.get('data_publicacao', 'N/A')}")
    autores = resultado.get('autores', [])
    output.append(f"Autores: {', '.join(autores) if autores else 'N/A'}")
    output.append(f"Número de palavras: {resultado.get('num_palavras', 'N/A')}")

    # Métricas quantitativas
    output.append("-" * 50)
    output.append("MÉTRICAS:")
    output.append(f"• Citações diretas: {resultado.get('num_citacoes', 0)}")
    output.append(f"• Links externos: {resultado.get('num_links_externos', 0)}")
    output.append(f"• Índice de sensacionalismo: {resultado.get('indice_sensacionalismo', 0):.2f}")
    if resultado.get('palavras_sensacionalistas'):
        output.append(f"  - Termos: {', '.join(resultado.get('palavras_sensacionalistas', []))}")
    output.append(f"• Índice de incerteza: {resultado.get('indice_incerteza', 0)}")
    if resultado.get('palavras_incerteza'):
        output.append(f"  - Termos: {', '.join(resultado.get('palavras_incerteza', []))}")
    output.append(f"• Índice opinativo: {resultado.get('indice_opiniao', 0)}")
    output.append(f"• Pontos de exclamação: {resultado.get('num_exclamacoes', 0)}")
    output.append(f"• Imagens: {resultado.get('num_imagens', 0)} (Sem descrição: {resultado.get('imagens_sem_descricao', 0)})")
    output.append(f"• Artigos similares (simulado): {resultado.get('artigos_similares_encontrados', 'N/A')}")

    # Alertas
    alertas = {k:v for k, v in resultado.items() if k.startswith("alerta_")}
    if alertas:
        output.append("-" * 50)
        output.append("ALERTAS:")
        for key, msg in alertas.items():
            output.append(f"⚠️ {msg}")

    # Pontuação final
    output.append("-" * 50)
    output.append(f"PONTUAÇÃO DE CREDIBILIDADE: {resultado.get('pontuacao_credibilidade', 'N/A')}/100")
    output.append(f"AVALIAÇÃO: {resultado.get('avaliacao', 'N/A')}")
    output.append(f"Tempo de análise: {resultado.get('tempo_analise_seg', 'N/A'):.2f} segundos")
    output.append("=" * 50)

    return "\n".join(output)


# Removido o bloco if __name__ == "__main__": daqui,
# a execução local será feita pelo main.py

Overwriting verificacao_avancada.py


In [27]:
%%writefile main.py
# -*- coding: utf-8 -*-
import functions_framework
import json
import logging
import os # Para obter variáveis de ambiente se necessário

# Importar a classe e a função de formatação do outro arquivo
from verificacao_avancada import VerificadorNoticiasAvancado, formatar_resultado_texto

# Configura o logging para Cloud Functions
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Instanciar o verificador globalmente?
# Pró: Reutiliza o objeto e listas carregadas entre invocações (se a instância for mantida "quente")
# Contra: Pode manter estado indesejado (embora esta classe pareça stateless)
# Alternativa: Instanciar dentro da função para garantir estado limpo a cada requisição.
# Vamos instanciar dentro da função por simplicidade e garantia de limpeza.
# verificador_global = VerificadorNoticiasAvancado()


@functions_framework.http
def handle_webhook_request(request):
    """
    Responde a requisições HTTP vindas do Dialogflow CX/ES.
    Espera um JSON com a estrutura do Dialogflow e extrai um parâmetro 'url'.
    """
    try:
        request_json = request.get_json(silent=True)
        logging.info(f"Recebido request do Dialogflow: {json.dumps(request_json, indent=2)}")

        # Verificar se o JSON foi recebido e tem a estrutura esperada
        if not request_json:
            logging.error("Request sem corpo JSON.")
            return create_dialogflow_response("Erro: Não recebi dados na requisição."), 400

        # --- Extrair a URL do parâmetro do Dialogflow ---
        # A localização exata do parâmetro pode variar (Dialogflow ES vs CX)
        # Tentativa comum para ES/CX: sessionInfo.parameters ou queryResult.parameters
        url_noticia = None
        if 'sessionInfo' in request_json and 'parameters' in request_json['sessionInfo']:
            url_noticia = request_json['sessionInfo']['parameters'].get('url') # Ajuste 'url' se o nome do parâmetro for diferente
        elif 'queryResult' in request_json and 'parameters' in request_json['queryResult']:
             url_noticia = request_json['queryResult']['parameters'].get('url') # Ajuste 'url' se o nome do parâmetro for diferente

        if not url_noticia:
            logging.warning("Parâmetro 'url' não encontrado na requisição do Dialogflow.")
            # Tenta pegar de uma possível mensagem de texto direto (menos comum para parâmetros)
            if 'text' in request_json.get('message', {}):
                 possible_url = request_json['message']['text']
                 # Validação básica se parece uma URL
                 if isinstance(possible_url, str) and possible_url.startswith(('http://', 'https://')):
                      url_noticia = possible_url
                      logging.info(f"URL encontrada no texto da mensagem: {url_noticia}")

        # Se ainda não encontrou a URL, retorna erro amigável
        if not url_noticia or not isinstance(url_noticia, str):
            logging.error("URL da notícia não fornecida ou inválida na requisição.")
            return create_dialogflow_response("Por favor, me envie a URL da notícia que você quer verificar."), 400

        # --- Executar a Verificação ---
        logging.info(f"Iniciando verificação para a URL: {url_noticia}")
        # Instanciar o verificador a cada requisição
        verificador = VerificadorNoticiasAvancado()
        resultado_analise = verificador.verificar_noticia(url_noticia)

        # --- Formatar a Resposta para o Dialogflow ---
        texto_resposta = gerar_resposta_dialogflow(resultado_analise)
        logging.info(f"Enviando resposta para Dialogflow: {texto_resposta[:200]}...") # Loga início da resposta
        return create_dialogflow_response(texto_resposta)

    except Exception as e:
        logging.exception("Erro inesperado ao processar a requisição do webhook.") # Loga o traceback completo
        # Retorna uma mensagem de erro genérica para o usuário
        return create_dialogflow_response("Desculpe, ocorreu um erro interno ao tentar verificar a notícia. Tente novamente mais tarde."), 500

def gerar_resposta_dialogflow(resultado):
    """Gera uma string de resposta amigável baseada no resultado da análise."""

    if not resultado:
         return "Desculpe, não consegui obter um resultado para a análise."

    status = resultado.get('status_geral', 'erro')

    if status.startswith('erro'):
        msg_erro = resultado.get('mensagem', 'Não foi possível completar a análise.')
        if status == 'erro_ao_baixar':
             # Erros comuns de download
             if '404' in msg_erro:
                  return f"Não consegui encontrar a página da notícia (Erro 404). Verifique se a URL está correta: {resultado.get('url')}"
             elif 'Timeout' in msg_erro:
                  return f"A página demorou muito para responder. O site pode estar fora do ar ou lento. Tente novamente mais tarde. URL: {resultado.get('url')}"
             elif 'Connection refused' in msg_erro or 'SSL' in msg_erro:
                  return f"Tive problemas para conectar ao site da notícia. Pode ser um problema temporário ou de segurança do site. URL: {resultado.get('url')}"
             else:
                  return f"Não consegui acessar a notícia para analisar. Verifique a URL ou tente mais tarde. Detalhe: {msg_erro[:100]}" # Limita tamanho do erro técnico
        else:
             return f"Ocorreu um problema durante a análise. Detalhe: {msg_erro[:100]}"

    # Análise completa
    pontuacao = resultado.get('pontuacao_credibilidade', 0)
    avaliacao = resultado.get('avaliacao', 'Indeterminada')
    dominio = resultado.get('dominio', 'Domínio desconhecido')
    status_fonte = resultado.get('status_fonte', 'não categorizada')

    resposta = []
    resposta.append(f"Análise da notícia em '{dominio}':")
    resposta.append(f"Pontuação de Credibilidade: {pontuacao}/100 ({avaliacao}).")

    # Adiciona detalhes sobre a fonte
    if status_fonte == 'confiável' or status_fonte == 'confiável (subdomínio)':
        resposta.append("✅ A fonte é considerada confiável.")
    elif status_fonte == 'suspeita':
        resposta.append(f"⚠️ Atenção: Esta fonte ('{dominio}') é conhecida por notícias satíricas ou de baixa credibilidade.")
    else: # não categorizada ou erro
        resposta.append("ℹ️ A reputação desta fonte não está na minha lista principal.")

    # Adiciona os principais alertas, se houver
    alertas_importantes = []
    if "alerta_sensacionalismo" in resultado:
         alertas_importantes.append("linguagem sensacionalista")
    if "alerta_opiniao" in resultado:
         alertas_importantes.append("forte caráter opinativo")
    if "alerta_incerteza" in resultado:
         alertas_importantes.append("uso de termos de incerteza")
    if "alerta_tamanho" in resultado:
        alertas_importantes.append("texto muito curto")

    if alertas_importantes:
         resposta.append(f"🚨 Pontos de atenção detectados: {'; '.join(alertas_importantes)}.")

    # Conclusão geral
    if pontuacao < 30:
        resposta.append("Recomendo muita cautela e buscar outras fontes antes de confiar nesta notícia.")
    elif pontuacao < 50:
        resposta.append("É bom verificar esta informação em outras fontes mais estabelecidas.")
    elif pontuacao < 70:
         resposta.append("Parece razoável, mas fique atento aos pontos levantados.")
    else:
        resposta.append("A notícia parece ter boa credibilidade com base na análise.")

    # Limitar o tamanho total da resposta para Dialogflow (geralmente tem limites)
    return " ".join(resposta)[:4000] # Limite generoso, ajuste se necessário

def create_dialogflow_response(text_message):
    """Cria a estrutura JSON de resposta padrão do Dialogflow."""
    response_data = {
        "fulfillment_response": {
            "messages": [
                {
                    "text": {
                        # O texto deve ser uma lista
                        "text": [text_message]
                    }
                }
            ]
        }
    }
    # Retorna a string JSON, status HTTP e Content-Type
    return json.dumps(response_data), 200, {'Content-Type': 'application/json; charset=utf-8'}


# --- Bloco para Execução Local (Testes) ---
if __name__ == "__main__":
    print("Executando Verificador de Notícias localmente...")
    # Exemplo de URLs para teste:
    url_confiavel = "https://g1.globo.com/economia/noticia/2023/10/26/bc-mantera-cortes-de-05-ponto-percentual-nas-proximas-reunioes-indicam-comunicado-e-declaracoes-de-campos-neto.ghtml" # Exemplo G1
    url_suspeita = "https://www.theonion.com/study-finds-working-from-home-increases-productivity-am-1849809884" # Exemplo The Onion (Sátira)
    url_nao_categorizada = "https://www.exemploqualquer.com/noticia-teste" # Exemplo inválido ou não categorizado
    url_erro_404 = "https://g1.globo.com/pagina-que-nao-existe-12345"
    url_curta = "https://www.tecmundo.com.br/voxel" # Página principal, não artigo

    urls_para_testar = [
         #url_confiavel,
         url_suspeita,
         #url_erro_404,
         #url_curta
         # Adicione outras URLs que queira testar
    ]

    if not urls_para_testar:
        try:
             url_input = input("Digite a URL da notícia para verificar: ")
             urls_para_testar.append(url_input.strip())
        except EOFError: # Handle case where input is redirected and empty
             print("Nenhuma URL fornecida para teste.")
             urls_para_testar = []


    verificador_local = VerificadorNoticiasAvancado()

    for url in urls_para_testar:
        if not url: continue
        print(f"\n--- Testando URL: {url} ---")
        try:
            resultado = verificador_local.verificar_noticia(url)
            # Usar a função de formatação para exibir o resultado no console
            print(formatar_resultado_texto(resultado))
            # Simular como seria a resposta para o Dialogflow
            print("\n--- Resposta Simulada para Dialogflow ---")
            print(gerar_resposta_dialogflow(resultado))
            print("----------------------------------------")

        except Exception as e:
            print(f"Erro inesperado durante o teste local da URL {url}: {e}")
            logging.exception("Erro no teste local") # Loga o traceback

    print("\nTestes locais concluídos.")

Writing main.py


In [26]:
%run verificacao_simples.py
verificar_url("https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp")


Verificador de notícias avançado - use a função verificar_url(sua_url) para analisar uma notícia
Verificador de Notícias Avançado inicializado!
Analisando URL: https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp

RELATÓRIO DE VERIFICAÇÃO DE NOTÍCIA
URL: https://www.msn.com/pt-br/noticias/brasil/casa-branca-aponta-que-suposta-verdade-sobre-a-origem-da-covid-19-tem-rela%C3%A7%C3%A3o-com-vazamento-em-laborat%C3%B3rio-na-china/ar-AA1DbpaW?ocid=BingNewsSerp
Domínio: N/A
Status da fonte: N/A
--------------------------------------------------
Título: N/A
Data de publicação: N/A
Autores: N/A
Número de palavras: 0
--------------------------------------------------
MÉTRICAS:
• Citações: 0
• URLs referenciadas: 0
• Índice de sensacionalismo: 0
• Índice de incerteza: 0
• Índice opinativo: 0
• Pontos de exclamação: 0
• Imagens: 0
• Artigos similares 

{'erro': 'Artigo não disponível para análise',
 'pontuacao_credibilidade': 70,
 'avaliacao': 'Boa credibilidade'}

In [25]:
%run verificacao_simples.py

Verificador de notícias avançado - use a função verificar_url(sua_url) para analisar uma notícia


In [24]:
%run verificacao_avancada.py

In [21]:
!python "C:\\Users\\Geraldo\\Documents\\ChatBot_NewsgamesIA\\verificacao_avancada.py"

python3: can't open file '/content/C:\\Users\\Geraldo\\Documents\\ChatBot_NewsgamesIA\\verificacao_avancada.py': [Errno 2] No such file or directory


In [20]:
!python "C:\\Users\\Geraldo\\Documents\\ChatBot_NewsgamesIA\\verificacao_avancada.py"

python3: can't open file '/content/C:\\Users\\Geraldo\\Documents\\ChatBot_NewsgamesIA\\verificacao_avancada.py': [Errno 2] No such file or directory


In [19]:
python verificacao_avancada.py

SyntaxError: invalid syntax (<ipython-input-19-c824c7471ad6>, line 1)

In [18]:
%%writefile verificacao_avancada.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse
import hashlib
from collections import Counter
import time

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticiasAvancado:
    def __init__(self):
        print("Verificador de Notícias Avançado inicializado!")
        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]

        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe"
        ]

    def verificar_url_simples(self, url):
        """Executa uma verificação simples em uma URL de notícia."""
        try:
            # Extrair domínio
            parsed_url = urlparse(url)
            dominio = parsed_url.netloc

            # Tentar acessar a URL
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Levantar exceção para status HTTP de erro

            # Verificar status HTTP
            if response.status_code == 200:
                print(f"URL acessada com sucesso: {url}")
                print(f"Domínio: {dominio}")
                print(f"Tamanho do conteúdo: {len(response.text)} caracteres.")
                return {
                    "status": "sucesso",
                    "dominio": dominio,
                    "tamanho_conteudo": len(response.text),
                    "mensagem": "URL acessada com sucesso"
                }
            else:
                print(f"Erro ao acessar a URL. Código HTTP: {response.status_code}")
                return {
                    "status": "erro",
                    "mensagem": f"Erro ao acessar a URL. Código HTTP: {response.status_code}"
                }

        except requests.exceptions.RequestException as e:
            print(f"Erro ao acessar a URL: {e}")
            return {
                "status": "erro",
                "mensagem": str(e)
            }

    # Outros métodos como "extrair_dominio", "verificar_fonte", "baixar_artigo", etc.
    # permanecem iguais.

    def verificar_noticia(self, url):
        """Função principal para verificar uma notícia."""
        print(f"Analisando URL: {url}")

        # Tentar uma verificação simples primeiro
        resultado_simples = self.verificar_url_simples(url)
        if resultado_simples["status"] == "erro":
            return resultado_simples  # Retorna o erro direto se a verificação simples falhar

        # Se a verificação simples for bem-sucedida, proceder para análise avançada
        artigo = self.baixar_artigo(url)

        if not artigo:
            return {"status": "erro", "mensagem": "Não foi possível baixar o artigo"}

        resultados = self.analisar_credibilidade(artigo)

        # Calcular pontuação de credibilidade (0-100)
        pontuacao = 70  # Base inicial neutro

        # Ajustes baseados na fonte
        pontuacao += resultados.get("pontos_fonte", 0)

        # Fatores que reduzem a pontuação
        if "alerta_tamanho" in resultados:
            pontuacao -= 15
        if "alerta_sensacionalismo" in resultados:
            if "Alto índice" in resultados["alerta_sensacionalismo"]:
                pontuacao -= 20
            else:
                pontuacao -= 10
        if "alerta_fontes" in resultados:
            pontuacao -= 10
        if "alerta_opiniao" in resultados:
            pontuacao -= 15
        if "alerta_incerteza" in resultados and resultados["indice_incerteza"] > 3:
            pontuacao -= 10
        if "alerta_exclamacoes" in resultados:
            pontuacao -= 5
        if "alerta_imagens_descricao" in resultados:
            pontuacao -= 5

        # Fatores que aumentam a pontuação
        if resultados.get("num_citacoes", 0) > 3:
            pontuacao += 10
        elif resultados.get("num_citacoes", 0) > 0:
            pontuacao += 5
        if resultados.get("num_urls", 0) > 2:
            pontuacao += 5
        if resultados.get("num_imagens", 0) > 1:
            pontuacao += 5
        if resultados.get("artigos_similares", 0) > 3:
            pontuacao += 5

        # Ajustar pontuação final
        pontuacao = max(0, min(100, pontuacao))
        resultados["pontuacao_credibilidade"] = pontuacao

        # Avaliação qualitativa
        if pontuacao >= 85:
            resultados["avaliacao"] = "Alta credibilidade"
        elif pontuacao >= 70:
            resultados["avaliacao"] = "Boa credibilidade"
        elif pontuacao >= 50:
            resultados["avaliacao"] = "Credibilidade moderada"
        elif pontuacao >= 30:
            resultados["avaliacao"] = "Credibilidade questionável"
        else:
            resultados["avaliacao"] = "Baixa credibilidade"

        return resultados

Writing verificacao_avancada.py


In [16]:
%run verificacao_simples.py

Verificador de notícias avançado - use a função verificar_url(sua_url) para analisar uma notícia


In [15]:
import os
print(os.listdir())

['.config', 'drive', 'verificacao_simples.py', 'sample_data']


In [14]:
import os
print(os.getcwd())

/content


In [13]:
%run verificacao_avancada.py

Exception: File `'verificacao_avancada.py'` not found.

In [12]:
!pip install requests beautifulsoup4

In [17]:
%run verificacao_avancada.py

Exception: File `'verificacao_avancada.py'` not found.

In [ ]:
# -*- coding: utf-8 -*-
import functions_framework
import json
import logging
import os # Para obter variáveis de ambiente se necessário

# Importar a classe e a função de formatação do outro arquivo
from verificacao_avancada import VerificadorNoticiasAvancado, formatar_resultado_texto

# Configura o logging para Cloud Functions
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Instanciar o verificador globalmente?
# Pró: Reutiliza o objeto e listas carregadas entre invocações (se a instância for mantida "quente")
# Contra: Pode manter estado indesejado (embora esta classe pareça stateless)
# Alternativa: Instanciar dentro da função para garantir estado limpo a cada requisição.
# Vamos instanciar dentro da função por simplicidade e garantia de limpeza.
# verificador_global = VerificadorNoticiasAvancado()


@functions_framework.http
def handle_webhook_request(request):
    """
    Responde a requisições HTTP vindas do Dialogflow CX/ES.
    Espera um JSON com a estrutura do Dialogflow e extrai um parâmetro 'url'.
    """
    try:
        request_json = request.get_json(silent=True)
        logging.info(f"Recebido request do Dialogflow: {json.dumps(request_json, indent=2)}")

        # Verificar se o JSON foi recebido e tem a estrutura esperada
        if not request_json:
            logging.error("Request sem corpo JSON.")
            return create_dialogflow_response("Erro: Não recebi dados na requisição."), 400

        # --- Extrair a URL do parâmetro do Dialogflow ---
        # A localização exata do parâmetro pode variar (Dialogflow ES vs CX)
        # Tentativa comum para ES/CX: sessionInfo.parameters ou queryResult.parameters
        url_noticia = None
        if 'sessionInfo' in request_json and 'parameters' in request_json['sessionInfo']:
            url_noticia = request_json['sessionInfo']['parameters'].get('url') # Ajuste 'url' se o nome do parâmetro for diferente
        elif 'queryResult' in request_json and 'parameters' in request_json['queryResult']:
             url_noticia = request_json['queryResult']['parameters'].get('url') # Ajuste 'url' se o nome do parâmetro for diferente

        if not url_noticia:
            logging.warning("Parâmetro 'url' não encontrado na requisição do Dialogflow.")
            # Tenta pegar de uma possível mensagem de texto direto (menos comum para parâmetros)
            if 'text' in request_json.get('message', {}):
                 possible_url = request_json['message']['text']
                 # Validação básica se parece uma URL
                 if isinstance(possible_url, str) and possible_url.startswith(('http://', 'https://')):
                      url_noticia = possible_url
                      logging.info(f"URL encontrada no texto da mensagem: {url_noticia}")

        # Se ainda não encontrou a URL, retorna erro amigável
        if not url_noticia or not isinstance(url_noticia, str):
            logging.error("URL da notícia não fornecida ou inválida na requisição.")
            return create_dialogflow_response("Por favor, me envie a URL da notícia que você quer verificar."), 400

        # --- Executar a Verificação ---
        logging.info(f"Iniciando verificação para a URL: {url_noticia}")
        # Instanciar o verificador a cada requisição
        verificador = VerificadorNoticiasAvancado()
        resultado_analise = verificador.verificar_noticia(url_noticia)

        # --- Formatar a Resposta para o Dialogflow ---
        texto_resposta = gerar_resposta_dialogflow(resultado_analise)
        logging.info(f"Enviando resposta para Dialogflow: {texto_resposta[:200]}...") # Loga início da resposta
        return create_dialogflow_response(texto_resposta)

    except Exception as e:
        logging.exception("Erro inesperado ao processar a requisição do webhook.") # Loga o traceback completo
        # Retorna uma mensagem de erro genérica para o usuário
        return create_dialogflow_response("Desculpe, ocorreu um erro interno ao tentar verificar a notícia. Tente novamente mais tarde."), 500

def gerar_resposta_dialogflow(resultado):
    """Gera uma string de resposta amigável baseada no resultado da análise."""

    if not resultado:
         return "Desculpe, não consegui obter um resultado para a análise."

    status = resultado.get('status_geral', 'erro')

    if status.startswith('erro'):
        msg_erro = resultado.get('mensagem', 'Não foi possível completar a análise.')
        if status == 'erro_ao_baixar':
             # Erros comuns de download
             if '404' in msg_erro:
                  return f"Não consegui encontrar a página da notícia (Erro 404). Verifique se a URL está correta: {resultado.get('url')}"
             elif 'Timeout' in msg_erro:
                  return f"A página demorou muito para responder. O site pode estar fora do ar ou lento. Tente novamente mais tarde. URL: {resultado.get('url')}"
             elif 'Connection refused' in msg_erro or 'SSL' in msg_erro:
                  return f"Tive problemas para conectar ao site da notícia. Pode ser um problema temporário ou de segurança do site. URL: {resultado.get('url')}"
             else:
                  return f"Não consegui acessar a notícia para analisar. Verifique a URL ou tente mais tarde. Detalhe: {msg_erro[:100]}" # Limita tamanho do erro técnico
        else:
             return f"Ocorreu um problema durante a análise. Detalhe: {msg_erro[:100]}"

    # Análise completa
    pontuacao = resultado.get('pontuacao_credibilidade', 0)
    avaliacao = resultado.get('avaliacao', 'Indeterminada')
    dominio = resultado.get('dominio', 'Domínio desconhecido')
    status_fonte = resultado.get('status_fonte', 'não categorizada')

    resposta = []
    resposta.append(f"Análise da notícia em '{dominio}':")
    resposta.append(f"Pontuação de Credibilidade: {pontuacao}/100 ({avaliacao}).")

    # Adiciona detalhes sobre a fonte
    if status_fonte == 'confiável' or status_fonte == 'confiável (subdomínio)':
        resposta.append("✅ A fonte é considerada confiável.")
    elif status_fonte == 'suspeita':
        resposta.append(f"⚠️ Atenção: Esta fonte ('{dominio}') é conhecida por notícias satíricas ou de baixa credibilidade.")
    else: # não categorizada ou erro
        resposta.append("ℹ️ A reputação desta fonte não está na minha lista principal.")

    # Adiciona os principais alertas, se houver
    alertas_importantes = []
    if "alerta_sensacionalismo" in resultado:
         alertas_importantes.append("linguagem sensacionalista")
    if "alerta_opiniao" in resultado:
         alertas_importantes.append("forte caráter opinativo")
    if "alerta_incerteza" in resultado:
         alertas_importantes.append("uso de termos de incerteza")
    if "alerta_tamanho" in resultado:
        alertas_importantes.append("texto muito curto")

    if alertas_importantes:
         resposta.append(f"🚨 Pontos de atenção detectados: {'; '.join(alertas_importantes)}.")

    # Conclusão geral
    if pontuacao < 30:
        resposta.append("Recomendo muita cautela e buscar outras fontes antes de confiar nesta notícia.")
    elif pontuacao < 50:
        resposta.append("É bom verificar esta informação em outras fontes mais estabelecidas.")
    elif pontuacao < 70:
         resposta.append("Parece razoável, mas fique atento aos pontos levantados.")
    else:
        resposta.append("A notícia parece ter boa credibilidade com base na análise.")

    # Limitar o tamanho total da resposta para Dialogflow (geralmente tem limites)
    return " ".join(resposta)[:4000] # Limite generoso, ajuste se necessário

def create_dialogflow_response(text_message):
    """Cria a estrutura JSON de resposta padrão do Dialogflow."""
    response_data = {
        "fulfillment_response": {
            "messages": [
                {
                    "text": {
                        # O texto deve ser uma lista
                        "text": [text_message]
                    }
                }
            ]
        }
    }
    # Retorna a string JSON, status HTTP e Content-Type
    return json.dumps(response_data), 200, {'Content-Type': 'application/json; charset=utf-8'}


# --- Bloco para Execução Local (Testes) ---
if __name__ == "__main__":
    print("Executando Verificador de Notícias localmente...")
    # Exemplo de URLs para teste:
    url_confiavel = "https://g1.globo.com/economia/noticia/2023/10/26/bc-mantera-cortes-de-05-ponto-percentual-nas-proximas-reunioes-indicam-comunicado-e-declaracoes-de-campos-neto.ghtml" # Exemplo G1
    url_suspeita = "https://www.theonion.com/study-finds-working-from-home-increases-productivity-am-1849809884" # Exemplo The Onion (Sátira)
    url_nao_categorizada = "https://www.exemploqualquer.com/noticia-teste" # Exemplo inválido ou não categorizado
    url_erro_404 = "https://g1.globo.com/pagina-que-nao-existe-12345"
    url_curta = "https://www.tecmundo.com.br/voxel" # Página principal, não artigo

    urls_para_testar = [
         #url_confiavel,
         url_suspeita,
         #url_erro_404,
         #url_curta
         # Adicione outras URLs que queira testar
    ]

    if not urls_para_testar:
        try:
             url_input = input("Digite a URL da notícia para verificar: ")
             urls_para_testar.append(url_input.strip())
        except EOFError: # Handle case where input is redirected and empty
             print("Nenhuma URL fornecida para teste.")
             urls_para_testar = []


    verificador_local = VerificadorNoticiasAvancado()

    for url in urls_para_testar:
        if not url: continue
        print(f"\n--- Testando URL: {url} ---")
        try:
            resultado = verificador_local.verificar_noticia(url)
            # Usar a função de formatação para exibir o resultado no console
            print(formatar_resultado_texto(resultado))
            # Simular como seria a resposta para o Dialogflow
            print("\n--- Resposta Simulada para Dialogflow ---")
            print(gerar_resposta_dialogflow(resultado))
            print("----------------------------------------")

        except Exception as e:
            print(f"Erro inesperado durante o teste local da URL {url}: {e}")
            logging.exception("Erro no teste local") # Loga o traceback

    print("\nTestes locais concluídos.")

In [10]:
%%writefile verificacao_simples.py
import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3
import json
from urllib.parse import urlparse
import hashlib
from collections import Counter
import time

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticiasAvancado:
    def __init__(self):
        print("Verificador de Notícias Avançado inicializado!")
        # Carregar listas de fontes confiáveis e não confiáveis
        self.fontes_confiaveis = [
            'g1.globo.com', 'bbc.com', 'bbc.co.uk', 'reuters.com', 'apnews.com',
            'nytimes.com', 'washingtonpost.com', 'estadao.com.br', 'folha.uol.com.br',
            'cnn.com', 'valor.globo.com', 'info.abril.com.br', 'oglobo.globo.com',
            'dw.com', 'npr.org', 'tecmundo.com.br', 'canaltech.com.br', 'olhardigital.com.br'
        ]

        self.fontes_suspeitas = [
            'theonion.com', 'sensacionalista.com.br', 'breitbart.com', 'infowars.com',
            'clickhole.com', 'nationalreport.net', 'worldnewsdailyreport.com',
            'dailycurrant.com', 'empirenews.net'
        ]

        # Palavras para análise linguística
        self.palavras_sensacionalistas = {
            "chocante": 2, "inacreditável": 2, "surpreendente": 1, "sensacional": 2,
            "impressionante": 1, "alarmante": 1.5, "exclusivo": 1, "urgente": 1.5,
            "shocking": 2, "unbelievable": 2, "amazing": 1, "surprising": 1,
            "imperdível": 1.5, "incrível": 1.5, "absurdo": 1.5, "escandaloso": 2,
            "bomba": 1.5, "inédito": 1, "revelado": 1, "segredo": 1.5,
            "polêmico": 1.5, "controverso": 1, "escândalo": 1.5
        }

        self.palavras_incerteza = [
            "talvez", "possivelmente", "provavelmente", "pode ser", "especula-se",
            "acredita-se", "sugere", "alega", "supostamente", "rumores",
            "maybe", "possibly", "probably", "might be", "it is speculated",
            "it is believed", "suggests", "allegedly", "supposedly", "rumors"
        ]

        self.expressoes_opinativas = [
            "eu acho", "na minha opinião", "parece que", "acredito que",
            "i think", "in my opinion", "it seems that", "i believe"
        ]

    def extrair_dominio(self, url):
        """Extrai o domínio base da URL."""
        parsed_url = urlparse(url)
        domain = parsed_url.netloc
        return domain

    def verificar_fonte(self, url):
        """Verifica o domínio da URL contra as listas de fontes."""
        dominio = self.extrair_dominio(url)
        resultado = {}

        # Verificar se o domínio está nas listas
        if any(dominio.endswith(fonte) for fonte in self.fontes_confiaveis):
            resultado["status_fonte"] = "confiável"
            resultado["pontos_fonte"] = 20
        elif any(dominio.endswith(fonte) for fonte in self.fontes_suspeitas):
            resultado["status_fonte"] = "suspeita"
            resultado["pontos_fonte"] = -30
            resultado["alerta_fonte"] = "Fonte conhecida por conteúdo satírico ou de baixa credibilidade"
        else:
            resultado["status_fonte"] = "não categorizada"
            resultado["pontos_fonte"] = 0

        resultado["dominio"] = dominio
        return resultado

    def baixar_artigo(self, url):
        """Baixa e extrai o conteúdo básico de um artigo a partir da URL"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem mais abrangente)
            content = ""

            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body', '.news-text']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)

                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(author_tag)
                else:
                    elements = soup.find_all(author_tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            # Remover duplicações nos autores
            autores = list(dict.fromkeys(autores))

            # Extrair imagens
            imagens = []
            for img in soup.find_all('img'):
                src = img.get('src')
                alt = img.get('alt', '')
                if src and not src.startswith('data:'):
                    # Garantir URL completa
                    if src.startswith('/'):
                        parsed_url = urlparse(url)
                        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                        src = base_url + src
                    imagens.append({
                        'url': src,
                        'alt': alt
                    })

            # Extrair palavras-chave
            keywords = []
            meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
            if meta_keywords:
                keywords = meta_keywords.get('content', '').split(',')
                keywords = [k.strip() for k in keywords]

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': url,
                'imagens': imagens,
                'keywords': keywords
            }

        except Exception as e:
            print(f"Erro ao baixar artigo: {e}")
            return None

    def analisar_linguagem(self, texto):
        """Analisa o tom e a linguagem usada no texto."""
        resultados = {}

        # Análise de sensacionalismo com pesos
        count_sensacionalismo = 0
        palavras_encontradas = []

        for palavra, peso in self.palavras_sensacionalistas.items():
            ocorrencias = len(re.findall(r'\b' + palavra + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_sensacionalismo += ocorrencias * peso
                palavras_encontradas.append(f"{palavra} ({ocorrencias}x)")

        resultados["indice_sensacionalismo"] = count_sensacionalismo
        resultados["palavras_sensacionalistas"] = palavras_encontradas

        if count_sensacionalismo > 3:
            resultados["alerta_sensacionalismo"] = "Alto índice de linguagem sensacionalista detectado"
        elif count_sensacionalismo > 1.5:
            resultados["alerta_sensacionalismo"] = "Linguagem potencialmente sensacionalista detectada"

        # Análise de incerteza
        count_incerteza = 0
        palavras_incerteza = []

        for palavra in self.palavras_incerteza:
            ocorrencias = len(re.findall(r'\b' + palavra + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_incerteza += ocorrencias
                palavras_incerteza.append(palavra)

        resultados["indice_incerteza"] = count_incerteza
        resultados["palavras_incerteza"] = palavras_incerteza

        if count_incerteza > 3:
            resultados["alerta_incerteza"] = "Alto uso de termos de incerteza detectado"

        # Análise de opinião
        count_opiniao = 0
        expressoes_opiniao = []

        for expressao in self.expressoes_opinativas:
            ocorrencias = len(re.findall(r'\b' + expressao + r'\b', texto.lower()))
            if ocorrencias > 0:
                count_opiniao += ocorrencias
                expressoes_opiniao.append(expressao)

        resultados["indice_opiniao"] = count_opiniao
        resultados["expressoes_opiniao"] = expressoes_opiniao

        if count_opiniao > 2:
            resultados["alerta_opiniao"] = "Texto com forte caráter opinativo detectado"

        # Análise de exclamações (indicativo de sensacionalismo)
        exclamacoes = len(re.findall(r'!', texto))
        resultados["num_exclamacoes"] = exclamacoes

        if exclamacoes > 5:
            resultados["alerta_exclamacoes"] = "Uso excessivo de pontos de exclamação"

        return resultados

    def verificar_imagens(self, imagens):
        """Analisa as imagens associadas ao artigo."""
        resultados = {}

        if not imagens:
            resultados["alerta_imagens"] = "Nenhuma imagem encontrada no artigo"
            return resultados

        resultados["num_imagens"] = len(imagens)

        # Verificar descrições das imagens (alt text)
        imagens_sem_descricao = sum(1 for img in imagens if not img.get('alt'))
        resultados["imagens_sem_descricao"] = imagens_sem_descricao

        if imagens_sem_descricao == len(imagens) and len(imagens) > 0:
            resultados["alerta_imagens_descricao"] = "Nenhuma das imagens possui descrição adequada"

        # Calcular hash básico das URLs de imagens para potencial comparação futura
        hashes_urls = [hashlib.md5(img['url'].encode()).hexdigest()[:8] for img in imagens]
        resultados["hashes_imagens"] = hashes_urls

        return resultados

    def buscar_noticias_similares(self, titulo, keywords):
        """Busca por notícias similares na web usando palavras-chave do artigo."""
        resultados = {}

        try:
            # Preparar termos de busca
            termos_busca = titulo
            if keywords:
                termos_busca += " " + " ".join(keywords[:3])  # Limitar a 3 keywords

            # Simular resultados para demonstração
            # Em uma implementação real, você usaria uma API como Google News ou similar
            resultados["artigos_similares"] = 5  # Simulando 5 artigos similares
            resultados["similaridade"] = "média"  # baixa, média, alta
        except Exception as e:
            print(f"Erro ao buscar notícias similares: {e}")
            resultados["erro_busca"] = str(e)

        return resultados

    def analisar_credibilidade(self, artigo):
        """Análise combinada de credibilidade baseada em características do texto e fonte."""
        if not artigo or not artigo.get('conteudo'):
            return {"erro": "Artigo não disponível para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # 1. Verificação de fonte/domínio
        info_fonte = self.verificar_fonte(artigo['url'])
        resultados.update(info_fonte)

        # 2. Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # 3. Análise básica do conteúdo
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # 4. Análise detalhada da linguagem
        analise_linguagem = self.analisar_linguagem(conteudo)
        resultados.update(analise_linguagem)

        # 5. Verificação de citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)

        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # 6. Verificação de URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        # 7. Análise de imagens
        resultados_imagens = self.verificar_imagens(artigo.get('imagens', []))
        resultados.update(resultados_imagens)

        # 8. Verificação cruzada com outras notícias
        resultados_similares = self.buscar_noticias_similares(
            artigo['titulo'],
            artigo.get('keywords', [])
        )
        resultados.update(resultados_similares)

        return resultados

    def verificar_noticia(self, url):
        """Função principal para verificar uma notícia."""
        print(f"Analisando URL: {url}")
        artigo = self.baixar_artigo(url)

        if not artigo:
            return {"status": "erro", "mensagem": "Não foi possível baixar o artigo"}

        resultados = self.analisar_credibilidade(artigo)

        # Calcular pontuação de credibilidade (0-100)
        pontuacao = 70  # Base inicial neutro

        # Ajustes baseados na fonte
        pontuacao += resultados.get("pontos_fonte", 0)

        # Fatores que reduzem a pontuação
        if "alerta_tamanho" in resultados:
            pontuacao -= 15
        if "alerta_sensacionalismo" in resultados:
            if "Alto índice" in resultados["alerta_sensacionalismo"]:
                pontuacao -= 20
            else:
                pontuacao -= 10
        if "alerta_fontes" in resultados:
            pontuacao -= 10
        if "alerta_opiniao" in resultados:
            pontuacao -= 15
        if "alerta_incerteza" in resultados and resultados["indice_incerteza"] > 3:
            pontuacao -= 10
        if "alerta_exclamacoes" in resultados:
            pontuacao -= 5
        if "alerta_imagens_descricao" in resultados:
            pontuacao -= 5

        # Fatores que aumentam a pontuação
        if resultados.get("num_citacoes", 0) > 3:
            pontuacao += 10
        elif resultados.get("num_citacoes", 0) > 0:
            pontuacao += 5
        if resultados.get("num_urls", 0) > 2:
            pontuacao += 5
        if resultados.get("num_imagens", 0) > 1:
            pontuacao += 5
        if resultados.get("artigos_similares", 0) > 3:
            pontuacao += 5

        # Ajustar pontuação final
        pontuacao = max(0, min(100, pontuacao))
        resultados["pontuacao_credibilidade"] = pontuacao

        # Avaliação qualitativa
        if pontuacao >= 85:
            resultados["avaliacao"] = "Alta credibilidade"
        elif pontuacao >= 70:
            resultados["avaliacao"] = "Boa credibilidade"
        elif pontuacao >= 50:
            resultados["avaliacao"] = "Credibilidade moderada"
        elif pontuacao >= 30:
            resultados["avaliacao"] = "Credibilidade questionável"
        else:
            resultados["avaliacao"] = "Baixa credibilidade"

        return resultados


# Função para demonstração com formatação de saída melhorada
def verificar_url(url):
    verificador = VerificadorNoticiasAvancado()
    tempo_inicio = time.time()
    resultado = verificador.verificar_noticia(url)
    tempo_execucao = time.time() - tempo_inicio

    print("\n" + "="*50)
    print(f"RELATÓRIO DE VERIFICAÇÃO DE NOTÍCIA")
    print("="*50)
    print(f"URL: {url}")
    print(f"Domínio: {resultado.get('dominio', 'N/A')}")
    print(f"Status da fonte: {resultado.get('status_fonte', 'N/A')}")
    print("-"*50)

    print(f"Título: {resultado.get('titulo', 'N/A')}")
    print(f"Data de publicação: {resultado.get('data_publicacao', 'N/A')}")
    print(f"Autores: {', '.join(resultado.get('autores', ['N/A']))}")
    print(f"Número de palavras: {resultado.get('num_palavras', 0)}")

    # Métricas quantitativas
    print("-"*50)
    print("MÉTRICAS:")
    print(f"• Citações: {resultado.get('num_citacoes', 0)}")
    print(f"• URLs referenciadas: {resultado.get('num_urls', 0)}")
    print(f"• Índice de sensacionalismo: {resultado.get('indice_sensacionalismo', 0)}")
    if resultado.get('palavras_sensacionalistas'):
        print(f"  - Termos: {', '.join(resultado.get('palavras_sensacionalistas', []))}")
    print(f"• Índice de incerteza: {resultado.get('indice_incerteza', 0)}")
    if resultado.get('indice_incerteza', 0) > 0:
        print(f"  - Termos: {', '.join(resultado.get('palavras_incerteza', []))}")
    print(f"• Índice opinativo: {resultado.get('indice_opiniao', 0)}")
    print(f"• Pontos de exclamação: {resultado.get('num_exclamacoes', 0)}")
    print(f"• Imagens: {resultado.get('num_imagens', 0)}")
    print(f"• Artigos similares encontrados: {resultado.get('artigos_similares', 0)}")

    # Alertas
    alertas = [k for k in resultado.keys() if k.startswith("alerta_")]
    if alertas:
        print("-"*50)
        print("ALERTAS:")
        for alerta in alertas:
            print(f"⚠️ {resultado[alerta]}")

    # Pontuação final
    print("-"*50)
    print(f"PONTUAÇÃO DE CREDIBILIDADE: {resultado.get('pontuacao_credibilidade', 0)}/100")
    print(f"AVALIAÇÃO: {resultado.get('avaliacao', 'N/A')}")
    print(f"Tempo de análise: {tempo_execucao:.2f} segundos")
    print("="*50)

    return resultado


# Execução principal
if __name__ == "__main__":
    print("Verificador de notícias avançado - use a função verificar_url(sua_url) para analisar uma notícia")

Overwriting verificacao_simples.py


In [9]:
%run verificacao_simples.py
verificar_url("https://www.oficinadanet.com.br/iphone/61164-iphone-16e-made-in-brazil-montagem-confirmada-foxconn-jundiai ")


Verificador de notícias lite - use a função verificar_url(sua_url) para analisar uma notícia
Verificador de Notícias Lite inicializado!
Analisando URL: https://www.oficinadanet.com.br/iphone/61164-iphone-16e-made-in-brazil-montagem-confirmada-foxconn-jundiai 

=== RESULTADO DA VERIFICAÇÃO ===
URL: https://www.oficinadanet.com.br/iphone/61164-iphone-16e-made-in-brazil-montagem-confirmada-foxconn-jundiai 
Título: Confirmado! iPhone 16e agora é “Made in Brazil”
Data de publicação: None
Autores: Adalton Bonaventura, Adalton Bonaventura
Número de palavras: 893
Número de citações: 2
Número de URLs referenciadas: 0

Pontuação de credibilidade: 70/100
Avaliação: Credibilidade moderada


{'titulo': 'Confirmado! iPhone 16e agora é “Made in Brazil”',
 'data_publicacao': None,
 'autores': ['Adalton Bonaventura', 'Adalton Bonaventura'],
 'url': 'https://www.oficinadanet.com.br/iphone/61164-iphone-16e-made-in-brazil-montagem-confirmada-foxconn-jundiai ',
 'num_palavras': 893,
 'indice_sensacionalismo': 2,
 'num_citacoes': 2,
 'num_urls': 0,
 'pontuacao_credibilidade': 70,
 'avaliacao': 'Credibilidade moderada'}

In [8]:
%run verificacao_simples.py

Verificador de notícias lite - use a função verificar_url(sua_url) para analisar uma notícia


In [7]:
!pip install lxml[html_clean]

In [6]:
%%writefile verificacao_simples.py

import requests
from bs4 import BeautifulSoup
import re
import ssl
import urllib3

# Desabilitar avisos SSL
urllib3.disable_warnings()

class VerificadorNoticiasLite:
    def __init__(self):
        print("Verificador de Notícias Lite inicializado!")

    def baixar_artigo(self, url):
        """Baixa e extrai o conteúdo básico de um artigo a partir da URL"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, verify=False, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')

            # Tenta extrair o título
            titulo = soup.title.text.strip() if soup.title else "Título não encontrado"

            # Tenta extrair o conteúdo do artigo (abordagem simples)
            # Procura por elementos que geralmente contêm o conteúdo principal
            content = ""

            # Procura por conteúdo em elementos comuns
            for tag in ['article', 'main', '.content', '.post', '.entry', '#content', '.article-body']:
                if tag.startswith('.') or tag.startswith('#'):
                    elements = soup.select(tag)
                else:
                    elements = soup.find_all(tag)

                if elements:
                    for element in elements:
                        content += element.get_text(separator=' ', strip=True) + " "
                    break

            # Se não encontrou conteúdo nos elementos específicos, pega todos os parágrafos
            if not content:
                paragraphs = soup.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])

            # Tenta encontrar a data
            data = None
            # Procura por metadados comuns
            for meta in soup.find_all('meta'):
                if meta.get('property') in ['article:published_time', 'og:published_time', 'publication_date']:
                    data = meta.get('content')
                    break

            # Procura por elementos que possam conter autores
            autores = []
            for author_tag in ['author', 'byline', '.author', '.byline', '[rel=author]']:
                if author_tag.startswith('.') or author_tag.startswith('['):
                    elements = soup.select(author_tag)
                else:
                    elements = soup.find_all(author_tag)

                if elements:
                    for element in elements:
                        autor = element.get_text(strip=True)
                        if autor and len(autor) < 100:  # Evitar pegar conteúdo grande por engano
                            autores.append(autor)

            return {
                'titulo': titulo,
                'conteudo': content,
                'data': data,
                'autores': autores,
                'url': url
            }

        except Exception as e:
            print(f"Erro ao baixar artigo: {e}")
            return None

    def analisar_credibilidade(self, artigo):
        """Análise básica de credibilidade baseada em características do texto."""
        if not artigo or not artigo.get('conteudo'):
            return {"erro": "Artigo não disponível para análise"}

        resultados = {}
        conteudo = artigo['conteudo']

        # Copiar informações básicas
        resultados["titulo"] = artigo['titulo']
        resultados["data_publicacao"] = artigo['data']
        resultados["autores"] = artigo['autores']
        resultados["url"] = artigo['url']

        # Tamanho do texto (textos muito curtos podem ser suspeitos)
        palavras = len(conteudo.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # Verificar presença de palavras sensacionalistas
        palavras_sensacionalistas = ["chocante", "inacreditável", "surpreendente", "sensacional",
                                    "impressionante", "alarmante", "exclusivo", "urgente",
                                    "shocking", "unbelievable", "amazing", "surprising"]

        count_sensacionalismo = 0
        for palavra in palavras_sensacionalistas:
            count_sensacionalismo += len(re.findall(r'\b' + palavra + r'\b', conteudo.lower()))

        resultados["indice_sensacionalismo"] = count_sensacionalismo
        if count_sensacionalismo > 2:
            resultados["alerta_sensacionalismo"] = "Possível linguagem sensacionalista detectada"

        # Verificar presença de fontes ou citações
        citacoes = re.findall(r'"([^"]+)"', conteudo)
        resultados["num_citacoes"] = len(citacoes)

        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        # Verificar URLs externos (referências)
        urls = re.findall(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', conteudo)
        resultados["num_urls"] = len(urls)

        return resultados

    def verificar_noticia(self, url):
        """Função principal para verificar uma notícia."""
        print(f"Analisando URL: {url}")
        artigo = self.baixar_artigo(url)

        if not artigo:
            return {"status": "erro", "mensagem": "Não foi possível baixar o artigo"}

        resultados = self.analisar_credibilidade(artigo)

        # Calcular pontuação básica de credibilidade (0-100)
        pontuacao = 70  # Base inicial neutro

        # Fatores que reduzem a pontuação
        if "alerta_tamanho" in resultados:
            pontuacao -= 15
        if "alerta_sensacionalismo" in resultados:
            pontuacao -= 15
        if "alerta_fontes" in resultados:
            pontuacao -= 10

        # Fatores que aumentam a pontuação
        if resultados.get("num_citacoes", 0) > 3:
            pontuacao += 10
        if resultados.get("num_urls", 0) > 2:
            pontuacao += 5

        # Ajustar pontuação final
        pontuacao = max(0, min(100, pontuacao))
        resultados["pontuacao_credibilidade"] = pontuacao

        # Avaliação qualitativa
        if pontuacao >= 80:
            resultados["avaliacao"] = "Alta credibilidade"
        elif pontuacao >= 60:
            resultados["avaliacao"] = "Credibilidade moderada"
        elif pontuacao >= 40:
            resultados["avaliacao"] = "Credibilidade questionável"
        else:
            resultados["avaliacao"] = "Baixa credibilidade"

        return resultados


# Função para demonstração
def verificar_url(url):
    verificador = VerificadorNoticiasLite()
    resultado = verificador.verificar_noticia(url)

    print("\n=== RESULTADO DA VERIFICAÇÃO ===")
    print(f"URL: {url}")

    if "status" in resultado and resultado["status"] == "erro":
        print(f"ERRO: {resultado['mensagem']}")
        return

    print(f"Título: {resultado.get('titulo', 'N/A')}")
    print(f"Data de publicação: {resultado.get('data_publicacao', 'N/A')}")
    print(f"Autores: {', '.join(resultado.get('autores', ['N/A']))}")
    print(f"Número de palavras: {resultado.get('num_palavras', 0)}")
    print(f"Número de citações: {resultado.get('num_citacoes', 0)}")
    print(f"Número de URLs referenciadas: {resultado.get('num_urls', 0)}")

    # Mostrar alertas se houver
    alertas = [k for k in resultado.keys() if k.startswith("alerta_")]
    if alertas:
        print("\nALERTAS:")
        for alerta in alertas:
            print(f"- {resultado[alerta]}")

    print(f"\nPontuação de credibilidade: {resultado.get('pontuacao_credibilidade', 0)}/100")
    print(f"Avaliação: {resultado.get('avaliacao', 'N/A')}")

    return resultado


# Execução principal
if __name__ == "__main__":
    print("Verificador de notícias lite - use a função verificar_url(sua_url) para analisar uma notícia")

Overwriting verificacao_simples.py


In [5]:
%run verificacao_simples.py

ImportError: lxml.html.clean module is now a separate project lxml_html_clean.
Install lxml[html_clean] or lxml_html_clean directly.

In [4]:
!pip install newspaper3k langdetect textblob nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 89.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 10.7 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=cda521c32c7fac17bc3369e822c8fc7d678dc176a347603a770d114e0b36bb61
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=ed7aecee885b682eedb4a0ffc065b920

In [3]:
%run verificacao_simples.py

ModuleNotFoundError: No module named 'newspaper'

In [2]:
%%writefile verificacao_simples.py

from newspaper import Article
from langdetect import detect
from textblob import TextBlob
import re
import nltk
import ssl

# Desabilitar verificação SSL para download do NLTK (se necessário)
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Baixar recursos necessários do NLTK
try:
    nltk.data.find('punkt')
except LookupError:
    nltk.download('punkt')

class VerificadorNoticias:
    def __init__(self):
        print("Verificador de Notícias inicializado!")

    def baixar_artigo(self, url):
        """Baixa e processa o conteúdo de um artigo a partir da URL."""
        try:
            article = Article(url)
            article.download()
            article.parse()
            article.nlp()  # Realiza processamento de linguagem natural
            return article
        except Exception as e:
            print(f"Erro ao baixar artigo: {e}")
            return None

    def analisar_credibilidade(self, article):
        """Análise básica de credibilidade baseada em características do texto."""
        if not article:
            return {"erro": "Artigo não disponível para análise"}

        resultados = {}

        # Verificar idioma
        try:
            idioma = detect(article.text)
            resultados["idioma"] = idioma
        except:
            resultados["idioma"] = "desconhecido"

        # Extrair informações básicas
        resultados["titulo"] = article.title
        resultados["data_publicacao"] = article.publish_date
        resultados["autores"] = article.authors

        # Tamanho do texto (textos muito curtos podem ser suspeitos)
        palavras = len(article.text.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # Análise de sentimento (textos muito polarizados podem indicar viés)
        if resultados["idioma"] == "en":  # TextBlob funciona melhor com inglês
            blob = TextBlob(article.text)
            polaridade = blob.sentiment.polarity
            subjetividade = blob.sentiment.subjectivity
            resultados["polaridade"] = round(polaridade, 2)
            resultados["subjetividade"] = round(subjetividade, 2)

            if abs(polaridade) > 0.6:
                resultados["alerta_polaridade"] = "Texto com alta polaridade emocional"
            if subjetividade > 0.7:
                resultados["alerta_subjetividade"] = "Texto com alto grau de subjetividade"

        # Verificar presença de palavras sensacionalistas
        palavras_sensacionalistas = ["chocante", "inacreditável", "surpreendente", "sensacional",
                                    "impressionante", "alarmante", "exclusivo", "urgente",
                                    "shocking", "unbelievable", "amazing", "surprising"]

        count_sensacionalismo = 0
        for palavra in palavras_sensacionalistas:
            count_sensacionalismo += len(re.findall(palavra, article.text.lower()))

        resultados["indice_sensacionalismo"] = count_sensacionalismo
        if count_sensacionalismo > 2:
            resultados["alerta_sensacionalismo"] = "Possível linguagem sensacionalista detectada"

        # Verificar presença de fontes ou citações - CORREÇÃO SIMPLIFICADA
        # Use uma abordagem mais simples para encontrar citações - apenas aspas duplas
        citacoes = re.findall(r'"([^"]+)"', article.text)
        resultados["num_citacoes"] = len(citacoes)

        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        return resultados

    def verificar_noticia(self, url):
        """Função principal para verificar uma notícia."""
        print(f"Analisando URL: {url}")
        artigo = self.baixar_artigo(url)

        if not artigo:
            return {"status": "erro", "mensagem": "Não foi possível baixar o artigo"}

        resultados = self.analisar_credibilidade(artigo)

        # Calcular pontuação básica de credibilidade (0-100)
        pontuacao = 70  # Base inicial neutro

        # Fatores que reduzem a pontuação
        if "alerta_tamanho" in resultados:
            pontuacao -= 15
        if "alerta_polaridade" in resultados:
            pontuacao -= 10
        if "alerta_subjetividade" in resultados:
            pontuacao -= 10
        if "alerta_sensacionalismo" in resultados:
            pontuacao -= 15
        if "alerta_fontes" in resultados:
            pontuacao -= 10

        # Ajustar pontuação final
        pontuacao = max(0, min(100, pontuacao))
        resultados["pontuacao_credibilidade"] = pontuacao

        # Avaliação qualitativa
        if pontuacao >= 80:
            resultados["avaliacao"] = "Alta credibilidade"
        elif pontuacao >= 60:
            resultados["avaliacao"] = "Credibilidade moderada"
        elif pontuacao >= 40:
            resultados["avaliacao"] = "Credibilidade questionável"
        else:
            resultados["avaliacao"] = "Baixa credibilidade"

        return resultados


# Função para demonstração
def verificar_url(url):
    verificador = VerificadorNoticias()
    resultado = verificador.verificar_noticia(url)

    print("\n=== RESULTADO DA VERIFICAÇÃO ===")
    print(f"URL: {url}")

    if "status" in resultado and resultado["status"] == "erro":
        print(f"ERRO: {resultado['mensagem']}")
        return

    print(f"Título: {resultado.get('titulo', 'N/A')}")
    print(f"Data de publicação: {resultado.get('data_publicacao', 'N/A')}")
    print(f"Autores: {', '.join(resultado.get('autores', ['N/A']))}")
    print(f"Idioma: {resultado.get('idioma', 'N/A')}")
    print(f"Número de palavras: {resultado.get('num_palavras', 0)}")

    # Mostrar alertas se houver
    alertas = [k for k in resultado.keys() if k.startswith("alerta_")]
    if alertas:
        print("\nALERTAS:")
        for alerta in alertas:
            print(f"- {resultado[alerta]}")

    print(f"\nPontuação de credibilidade: {resultado.get('pontuacao_credibilidade', 0)}/100")
    print(f"Avaliação: {resultado.get('avaliacao', 'N/A')}")

    return resultado


# Definir a função como global para uso após rodar o script
def main():
    print("Verificador de notícias - use a função verificar_url(sua_url) para analisar uma notícia")

# Execução principal
if __name__ == "__main__":
    main()

Writing verificacao_simples.py


In [1]:
%run verificacao_simples.py

Exception: File `'verificacao_simples.py'` not found.

In [ ]:
%run verificacao_simples.py

SyntaxError: invalid syntax (verificacao_simples.py, line 90)

In [ ]:
%%writefile verificacao_simples.py
from newspaper import Article
from langdetect import detect
from textblob import TextBlob
import re
import nltk
import ssl

# Desabilitar verificação SSL para download do NLTK (se necessário)
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Baixar recursos necessários do NLTK
try:
    nltk.data.find('punkt')
except LookupError:
    nltk.download('punkt')

class VerificadorNoticias:
    def __init__(self):
        print("Verificador de Notícias inicializado!")

    def baixar_artigo(self, url):
        """Baixa e processa o conteúdo de um artigo a partir da URL."""
        try:
            article = Article(url)
            article.download()
            article.parse()
            article.nlp()  # Realiza processamento de linguagem natural
            return article
        except Exception as e:
            print(f"Erro ao baixar artigo: {e}")
            return None

    def analisar_credibilidade(self, article):
        """Análise básica de credibilidade baseada em características do texto."""
        if not article:
            return {"erro": "Artigo não disponível para análise"}

        resultados = {}

        # Verificar idioma
        try:
            idioma = detect(article.text)
            resultados["idioma"] = idioma
        except:
            resultados["idioma"] = "desconhecido"

        # Extrair informações básicas
        resultados["titulo"] = article.title
        resultados["data_publicacao"] = article.publish_date
        resultados["autores"] = article.authors

        # Tamanho do texto (textos muito curtos podem ser suspeitos)
        palavras = len(article.text.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # Análise de sentimento (textos muito polarizados podem indicar viés)
        if resultados["idioma"] == "en":  # TextBlob funciona melhor com inglês
            blob = TextBlob(article.text)
            polaridade = blob.sentiment.polarity
            subjetividade = blob.sentiment.subjectivity
            resultados["polaridade"] = round(polaridade, 2)
            resultados["subjetividade"] = round(subjetividade, 2)

            if abs(polaridade) > 0.6:
                resultados["alerta_polaridade"] = "Texto com alta polaridade emocional"
            if subjetividade > 0.7:
                resultados["alerta_subjetividade"] = "Texto com alto grau de subjetividade"

        # Verificar presença de palavras sensacionalistas
        palavras_sensacionalistas = ["chocante", "inacreditável", "surpreendente", "sensacional",
                                    "impressionante", "alarmante", "exclusivo", "urgente",
                                    "shocking", "unbelievable", "amazing", "surprising"]

        count_sensacionalismo = 0
        for palavra in palavras_sensacionalistas:
            count_sensacionalismo += len(re.findall(palavra, article.text.lower()))

        resultados["indice_sensacionalismo"] = count_sensacionalismo
        if count_sensacionalismo > 2:
            resultados["alerta_sensacionalismo"] = "Possível linguagem sensacionalista detectada"

        # Verificar presença de fontes ou citações - CORREÇÃO DA EXPRESSÃO REGULAR
        padrao_citacao = re.compile(r'"[^"]+"|\\"[^\\"]+\\"|\\'[^\\']+\\'')
        citacoes = padrao_citacao.findall(article.text)
        resultados["num_citacoes"] = len(citacoes)

        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        return resultados

    def verificar_noticia(self, url):
        """Função principal para verificar uma notícia."""
        print(f"Analisando URL: {url}")
        artigo = self.baixar_artigo(url)

        if not artigo:
            return {"status": "erro", "mensagem": "Não foi possível baixar o artigo"}

        resultados = self.analisar_credibilidade(artigo)

        # Calcular pontuação básica de credibilidade (0-100)
        pontuacao = 70  # Base inicial neutro

        # Fatores que reduzem a pontuação
        if "alerta_tamanho" in resultados:
            pontuacao -= 15
        if "alerta_polaridade" in resultados:
            pontuacao -= 10
        if "alerta_subjetividade" in resultados:
            pontuacao -= 10
        if "alerta_sensacionalismo" in resultados:
            pontuacao -= 15
        if "alerta_fontes" in resultados:
            pontuacao -= 10

        # Ajustar pontuação final
        pontuacao = max(0, min(100, pontuacao))
        resultados["pontuacao_credibilidade"] = pontuacao

        # Avaliação qualitativa
        if pontuacao >= 80:
            resultados["avaliacao"] = "Alta credibilidade"
        elif pontuacao >= 60:
            resultados["avaliacao"] = "Credibilidade moderada"
        elif pontuacao >= 40:
            resultados["avaliacao"] = "Credibilidade questionável"
        else:
            resultados["avaliacao"] = "Baixa credibilidade"

        return resultados


# Função para demonstração
def verificar_url(url):
    verificador = VerificadorNoticias()
    resultado = verificador.verificar_noticia(url)

    print("\n=== RESULTADO DA VERIFICAÇÃO ===")
    print(f"URL: {url}")

    if "status" in resultado and resultado["status"] == "erro":
        print(f"ERRO: {resultado['mensagem']}")
        return

    print(f"Título: {resultado.get('titulo', 'N/A')}")
    print(f"Data de publicação: {resultado.get('data_publicacao', 'N/A')}")
    print(f"Autores: {', '.join(resultado.get('autores', ['N/A']))}")
    print(f"Idioma: {resultado.get('idioma', 'N/A')}")
    print(f"Número de palavras: {resultado.get('num_palavras', 0)}")

    # Mostrar alertas se houver
    alertas = [k for k in resultado.keys() if k.startswith("alerta_")]
    if alertas:
        print("\nALERTAS:")
        for alerta in alertas:
            print(f"- {resultado[alerta]}")

    print(f"\nPontuação de credibilidade: {resultado.get('pontuacao_credibilidade', 0)}/100")
    print(f"Avaliação: {resultado.get('avaliacao', 'N/A')}")

    return resultado


# Para tornar a função verificar_url disponível quando o script é executado
# Cria uma instância global para permitir uso direto após executar o script
verificador_global = VerificadorNoticias()

if __name__ == "__main__":
    print("Verificador de notícias - use a função verificar_url(sua_url) para analisar uma notícia")
    # Exemplo:
    # verificar_url("https://www.exemplo.com/noticia")
else:
    # Isso garante que a função esteja disponível mesmo quando importada
    print("Módulo de verificação de notícias carregado.")

Overwriting verificacao_simples.py


In [ ]:
%run verificacao_simples.py
verificar_url("https://www.oficinadanet.com.br/iphone/61164-iphone-16e-made-in-brazil-montagem-confirmada-foxconn-jundiai")

SyntaxError: unterminated string literal (detected at line 91) (verificacao_simples.py, line 91)

NameError: name 'verificar_url' is not defined

In [ ]:
%%writefile verificacao_simples.py

from newspaper import Article
from langdetect import detect
from textblob import TextBlob
import re
import nltk
import ssl

# Desabilitar verificação SSL para download do NLTK (se necessário)
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Baixar recursos necessários do NLTK
try:
    nltk.data.find('punkt')
except LookupError:
    nltk.download('punkt')

class VerificadorNoticias:
    def __init__(self):
        print("Verificador de Notícias inicializado!")

    def baixar_artigo(self, url):
        """Baixa e processa o conteúdo de um artigo a partir da URL."""
        try:
            article = Article(url)
            article.download()
            article.parse()
            article.nlp()  # Realiza processamento de linguagem natural
            return article
        except Exception as e:
            print(f"Erro ao baixar artigo: {e}")
            return None

    def analisar_credibilidade(self, article):
        """Análise básica de credibilidade baseada em características do texto."""
        if not article:
            return {"erro": "Artigo não disponível para análise"}

        resultados = {}

        # Verificar idioma
        try:
            idioma = detect(article.text)
            resultados["idioma"] = idioma
        except:
            resultados["idioma"] = "desconhecido"

        # Extrair informações básicas
        resultados["titulo"] = article.title
        resultados["data_publicacao"] = article.publish_date
        resultados["autores"] = article.authors

        # Tamanho do texto (textos muito curtos podem ser suspeitos)
        palavras = len(article.text.split())
        resultados["num_palavras"] = palavras
        if palavras < 100:
            resultados["alerta_tamanho"] = "Texto muito curto, pode não ser um artigo completo"

        # Análise de sentimento (textos muito polarizados podem indicar viés)
        if resultados["idioma"] == "en":  # TextBlob funciona melhor com inglês
            blob = TextBlob(article.text)
            polaridade = blob.sentiment.polarity
            subjetividade = blob.sentiment.subjectivity
            resultados["polaridade"] = round(polaridade, 2)
            resultados["subjetividade"] = round(subjetividade, 2)

            if abs(polaridade) > 0.6:
                resultados["alerta_polaridade"] = "Texto com alta polaridade emocional"
            if subjetividade > 0.7:
                resultados["alerta_subjetividade"] = "Texto com alto grau de subjetividade"

        # Verificar presença de palavras sensacionalistas
        palavras_sensacionalistas = ["chocante", "inacreditável", "surpreendente", "sensacional",
                                    "impressionante", "alarmante", "exclusivo", "urgente",
                                    "shocking", "unbelievable", "amazing", "surprising"]

        count_sensacionalismo = 0
        for palavra in palavras_sensacionalistas:
            count_sensacionalismo += len(re.findall(palavra, article.text.lower()))

        resultados["indice_sensacionalismo"] = count_sensacionalismo
        if count_sensacionalismo > 2:
            resultados["alerta_sensacionalismo"] = "Possível linguagem sensacionalista detectada"

        # Verificar presença de fontes ou citações
        padrao_citacao = re.compile(r'"[^"]+"|'[^']+'|"[^"]+"')
        citacoes = padrao_citacao.findall(article.text)
        resultados["num_citacoes"] = len(citacoes)

        if len(citacoes) == 0:
            resultados["alerta_fontes"] = "Nenhuma citação direta encontrada"

        return resultados

    def verificar_noticia(self, url):
        """Função principal para verificar uma notícia."""
        print(f"Analisando URL: {url}")
        artigo = self.baixar_artigo(url)

        if not artigo:
            return {"status": "erro", "mensagem": "Não foi possível baixar o artigo"}

        resultados = self.analisar_credibilidade(artigo)

        # Calcular pontuação básica de credibilidade (0-100)
        pontuacao = 70  # Base inicial neutro

        # Fatores que reduzem a pontuação
        if "alerta_tamanho" in resultados:
            pontuacao -= 15
        if "alerta_polaridade" in resultados:
            pontuacao -= 10
        if "alerta_subjetividade" in resultados:
            pontuacao -= 10
        if "alerta_sensacionalismo" in resultados:
            pontuacao -= 15
        if "alerta_fontes" in resultados:
            pontuacao -= 10

        # Ajustar pontuação final
        pontuacao = max(0, min(100, pontuacao))
        resultados["pontuacao_credibilidade"] = pontuacao

        # Avaliação qualitativa
        if pontuacao >= 80:
            resultados["avaliacao"] = "Alta credibilidade"
        elif pontuacao >= 60:
            resultados["avaliacao"] = "Credibilidade moderada"
        elif pontuacao >= 40:
            resultados["avaliacao"] = "Credibilidade questionável"
        else:
            resultados["avaliacao"] = "Baixa credibilidade"

        return resultados


# Função para demonstração
def verificar_url(url):
    verificador = VerificadorNoticias()
    resultado = verificador.verificar_noticia(url)

    print("\n=== RESULTADO DA VERIFICAÇÃO ===")
    print(f"URL: {url}")

    if "status" in resultado and resultado["status"] == "erro":
        print(f"ERRO: {resultado['mensagem']}")
        return

    print(f"Título: {resultado.get('titulo', 'N/A')}")
    print(f"Data de publicação: {resultado.get('data_publicacao', 'N/A')}")
    print(f"Autores: {', '.join(resultado.get('autores', ['N/A']))}")
    print(f"Idioma: {resultado.get('idioma', 'N/A')}")
    print(f"Número de palavras: {resultado.get('num_palavras', 0)}")

    # Mostrar alertas se houver
    alertas = [k for k in resultado.keys() if k.startswith("alerta_")]
    if alertas:
        print("\nALERTAS:")
        for alerta in alertas:
            print(f"- {resultado[alerta]}")

    print(f"\nPontuação de credibilidade: {resultado.get('pontuacao_credibilidade', 0)}/100")
    print(f"Avaliação: {resultado.get('avaliacao', 'N/A')}")

    return resultado


# Para testar, descomente e adicione uma URL
if __name__ == "__main__":
    print("Verificador de notícias - use a função verificar_url(sua_url) para analisar uma notícia")
    # Exemplo:
    # verificar_url("https://www.exemplo.com/noticia")

Overwriting verificacao_simples.py


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 9.8 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=89bd43714792922f535ccdab78f83501511e1ba60634deca710b0ce2b97986ef
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for langdetect: filename=langdet

In [ ]:
%%writefile requirements.txt
requests                # Para requisições web e acesso a APIs
beautifulsoup4          # Para extração de conteúdo de páginas web
newspaper3k             # Biblioteca para extração de artigos de notícias
nltk                    # Processamento de linguagem natural
scikit-learn            # Aprendizado de máquina para classificação
transformers            # Modelos de linguagem transformers (BERT, etc.)
spacy                   # Processamento avançado de linguagem natural
pandas                  # Análise de dados
python-dotenv           # Gerenciamento de chaves de API
langdetect              # Detecção de idioma
textblob                # Análise de sentimento
faiss-cpu               # Busca por similaridade vetorial (opcional)
tqdm                    # Barras de progresso para processos longos


Overwriting requirements.txt


In [ ]:
!pip install -r requirements.txt

In [ ]:
!dir requirements.txt

requirements.txt


In [ ]:
%%writefile requirements.txt
# Dependências básicas para chatbot
numpy
pandas
scikit-learn
nltk
tensorflow
transformers
flask # Para API web se necessário
requests
python-dotenv # Para variáveis de ambiente

Writing requirements.txt


In [ ]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
%run verificacao_simples.py

verificacao_simples.py 


In [ ]:
%%writefile verificacao_simples.py
print("verificacao_simples.py ")


Writing verificacao_simples.py


In [ ]:
%cd C:/Users/Geraldo/Documents/ChatBot_NewsgamesIA

[Errno 2] No such file or directory: 'C:/Users/Geraldo/Documents/ChatBot_NewsgamesIA'
/content


In [ ]:
import os
os.listdir("C:/Users/Geraldo/Documents/ChatBot_NewsgamesIA")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Geraldo/Documents/ChatBot_NewsgamesIA'

In [ ]:
import os
os.path.exists("C:/Users/Geraldo/Documents/ChatBot_NewsgamesIA/verificacao_simples.py")


False

In [ ]:
%run "C:/Users/Geraldo/Documents/ChatBot_NewsgamesIA/verificacao_simples.py"

Exception: File `'C:/Users/Geraldo/Documents/ChatBot_NewsgamesIA/verificacao_simples.py'` not found.

In [ ]:
!ls

drive  sample_data


In [ ]:
%run verificacao_simples.py

Exception: File `'verificacao_simples.py'` not found.

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 6.7 MB/s e

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64" (from torch) (from versions: none)
ERROR: No matching distribution found for nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64"


In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128
ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch


In [ ]:
pip uninstall torch torchvision torchaudio

In [ ]:
nvcc --version

NameError: name 'nvcc' is not defined

In [ ]:
nvcc --version

NameError: name 'nvcc' is not defined

In [ ]:
import torch

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU detected by PyTorch.")

PyTorch Version: 2.6.0+cu118
CUDA Available: False
No GPU detected by PyTorch.


In [ ]:
pip uninstall torch torchvision torchaudio

Found existing installation: torch 2.6.0+cu118
Uninstalling torch-2.6.0+cu118:
  Would remove:
    /usr/local/bin/torchfrtrace
    /usr/local/bin/torchrun
    /usr/local/lib/python3.11/dist-packages/functorch/*
    /usr/local/lib/python3.11/dist-packages/torch-2.6.0+cu118.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torch/*
    /usr/local/lib/python3.11/dist-packages/torchgen/*
Proceed (Y/n)? y
  Successfully uninstalled torch-2.6.0+cu118
Found existing installation: torchvision 0.21.0+cu118
Uninstalling torchvision-0.21.0+cu118:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/torchvision-0.21.0+cu118.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libcudart.60cfec8e.so.11.0
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libjpeg.1c1c4b09.so.8
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libnvjpeg.70530407.so.11
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libpng16.0364a1db.so.16
    /usr/local

In [ ]:
nvcc --version

NameError: name 'nvcc' is not defined

In [ ]:
import torch

print(f"PyTorch CUDA Version: {torch.version.cuda}")
print(f"CUDA Available: {torch.cuda.is_available()}")

PyTorch CUDA Version: 11.8
CUDA Available: False


In [ ]:
import torch
import torchvision

print(f"PyTorch CUDA Version: {torch.version.cuda}")
print(f"torchvision CUDA Version: {torchvision._C._CUDA_VERSION}")
print(f"CUDA Available: {torch.cuda.is_available()}")

PyTorch CUDA Version: 11.8


AttributeError: module 'torchvision' has no attribute '_C'

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.6.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (27 kB)
Using cached https://download.pytorch.org/whl/cu118/torch-2.6.0%2Bcu118-cp311-cp311-linux_x86_64.whl (848.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 16.8 MB/s eta 0:00:00


In [ ]:
pip uninstall torch torchvision -y

Found existing installation: torch 2.6.0+cu118
Uninstalling torch-2.6.0+cu118:
  Successfully uninstalled torch-2.6.0+cu118
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124


In [ ]:
import torch
import torchvision

print(f"PyTorch CUDA Available: {torch.cuda.is_available()}")
print(f"PyTorch CUDA Version: {torch.version.cuda}")
print(f"torchvision Version: {torchvision.__version__}")

RuntimeError: Detected that PyTorch and torchvision were compiled with different CUDA major versions. PyTorch has CUDA Version=11.8 and torchvision has CUDA Version=12.4. Please reinstall the torchvision that matches your PyTorch install.

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
%run verificacao_simples.py

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
Detected that PyTorch and torchvision were compiled with different CUDA major versions. PyTorch has CUDA Version=11.8 and torchvision has CUDA Version=12.4. Please reinstall the torchvision that matches your PyTorch install.

In [ ]:
pip install torchvision --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
%run verificacao_simples.py

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
Detected that PyTorch and torchvision were compiled with different CUDA major versions. PyTorch has CUDA Version=11.8 and torchvision has CUDA Version=12.4. Please reinstall the torchvision that matches your PyTorch install.

In [ ]:
import torch

if torch.cuda.is_available():
    print(f"GPU Detected: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU Detected.")

No GPU Detected.


In [ ]:
import torch
print(torch.cuda.is_available())

False


In [ ]:
import datasets
print(datasets.__version__)

3.5.0


In [ ]:
%run verificacao_simples.py


In [ ]:
%%writefile verificacao_simples.py
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset

# 1. Carregar Dataset
dataset = load_dataset("liar")  # Exemplo: Dataset LIAR (classificação de declarações)

# 2. Pré-Processamento
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    # Use a coluna correta do dataset: 'statement'
    return tokenizer(examples["statement"], truncation=True, padding="max_length")

encoded_dataset = dataset.map(preprocess_function, batched=True)

# Verificar o número de classes no dataset
labels = set(dataset["train"]["label"])
num_classes = len(labels)
print(f"Número de classes no dataset: {num_classes}")

# 3. Carregar Modelo Pré-Treinado
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_classes)

# Criar um Data Collator para lidar com padding dinâmico
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 4. Definir Configurações de Treino
training_args = TrainingArguments(
    output_dir="./results",  # Diretório para salvar os resultados
    logging_steps=500,  # Fazer logs a cada 500 steps
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",  # Diretório para salvar os logs
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    data_collator=data_collator,  # Substituindo o tokenizador
)

# 5. Treinar o Modelo
trainer.train()

# 6. Função de Verificação
def verificar_noticia(noticia):
    inputs = tokenizer(noticia, return_tensors="pt", truncation=True, padding="max_length")
    outputs = model(**inputs)
    predicted_class = outputs.logits.argmax().item()
    return f"Classificação: {predicted_class}"

# Teste com uma notícia
noticia_teste = "O presidente disse que a inflação caiu."
resultado = verificar_noticia(noticia_teste)
print(f"Resultado: {resultado}")

Overwriting verificacao_simples.py


In [ ]:
with open("verificacao_simples.py", "w") as f:
    f.write("""
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset

# 1. Carregar Dataset
dataset = load_dataset("liar")  # Exemplo: Dataset LIAR (classificação de declarações)

# 2. Pré-Processamento
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    # Use a coluna correta do dataset: 'statement'
    return tokenizer(examples["statement"], truncation=True, padding="max_length")

encoded_dataset = dataset.map(preprocess_function, batched=True)

# Verificar o número de classes no dataset
labels = set(dataset["train"]["label"])
num_classes = len(labels)
print(f"Número de classes no dataset: {num_classes}")

# 3. Carregar Modelo Pré-Treinado
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_classes)

# Criar um Data Collator para lidar com padding dinâmico
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 4. Definir Configurações de Treino
training_args = TrainingArguments(
    output_dir="./results",  # Diretório para salvar os resultados
    logging_steps=500,  # Fazer logs a cada 500 steps
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",  # Diretório para salvar os logs
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    data_collator=data_collator,  # Substituindo o tokenizador
)

# 5. Treinar o Modelo
trainer.train()

# 6. Função de Verificação
def verificar_noticia(noticia):
    inputs = tokenizer(noticia, return_tensors="pt", truncation=True, padding="max_length")
    outputs = model(**inputs)
    predicted_class = outputs.logits.argmax().item()
    return f"Classificação: {predicted_class}"

# Teste com uma notícia
noticia_teste = "O presidente disse que a inflação caiu."
resultado = verificar_noticia(noticia_teste)
print(f"Resultado: {resultado}")
""")

In [ ]:
%run /C:/Users/Geraldo/Documents/ChatBot_NewsgamesIA/verificacao_simples.py

Exception: File `'/C:/Users/Geraldo/Documents/ChatBot_NewsgamesIA/verificacao_simples.py'` not found.

In [ ]:
pip install datasets

In [ ]:
%%writefile verificacao_simples.py

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset

# 1. Carregar Dataset
dataset = load_dataset("liar")  # Exemplo: Dataset LIAR (classificação de declarações)

# 2. Pré-Processamento
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    # Use a coluna correta do dataset: 'statement'
    return tokenizer(examples["statement"], truncation=True, padding="max_length")

encoded_dataset = dataset.map(preprocess_function, batched=True)

# Verificar o número de classes no dataset
labels = set(dataset["train"]["label"])
num_classes = len(labels)
print(f"Número de classes no dataset: {num_classes}")

# 3. Carregar Modelo Pré-Treinado
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_classes)

# Criar um Data Collator para lidar com padding dinâmico
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 4. Definir Configurações de Treino
training_args = TrainingArguments(
    output_dir="./results",  # Diretório para salvar os resultados
    logging_steps=500,  # Fazer logs a cada 500 steps
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",  # Diretório para salvar os logs
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    data_collator=data_collator,  # Substituindo o tokenizador
)

# 5. Treinar o Modelo
trainer.train()

# 6. Função de Verificação
def verificar_noticia(noticia):
    inputs = tokenizer(noticia, return_tensors="pt", truncation=True, padding="max_length")
    outputs = model(**inputs)
    predicted_class = outputs.logits.argmax().item()
    return f"Classificação: {predicted_class}"

# Teste com uma notícia
noticia_teste = "O presidente disse que a inflação caiu."
resultado = verificar_noticia(noticia_teste)
print(f"Resultado: {resultado}")

In [ ]:
!pip install --upgrade datasets torch

In [ ]:
!pip install --upgrade datasets torch

In [ ]:
%run verificacao_simples.py


In [ ]:
%%writefile verificacao_simples.py

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# 1. Carregar Dataset
dataset = load_dataset("liar")  # Exemplo: Dataset LIAR (classificação de declarações)

# 2. Pré-Processamento
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    # Use a coluna correta do dataset: 'statement'
    return tokenizer(examples["statement"], truncation=True, padding="max_length")

encoded_dataset = dataset.map(preprocess_function, batched=True)

# 3. Carregar Modelo Pré-Treinado
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# 4. Definir Configurações de Treino
training_args = TrainingArguments(
    output_dir="./results",  # Diretório para salvar os resultados
    logging_steps=500,  # Fazer logs a cada 500 steps
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",  # Diretório para salvar os logs
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
)

# 5. Treinar o Modelo
trainer.train()

# 6. Função de Verificação
def verificar_noticia(noticia):
    inputs = tokenizer(noticia, return_tensors="pt", truncation=True, padding="max_length")
    outputs = model(**inputs)
    predicted_class = outputs.logits.argmax().item()
    return "Verdadeira" if predicted_class == 1 else "Falsa"

# Teste com uma notícia
noticia_teste = "O presidente disse que a inflação caiu."
resultado = verificar_noticia(noticia_teste)
print(f"Resultado: {resultado}")

In [ ]:
%run verificacao_simples.py